In [1]:
from datetime import datetime
import re
import pandas as pd
from modules import lesewert_screenshots as ls
#from modules import screenshot_powerBI as sp
import importlib
importlib.reload(ls)
import locale
locale.setlocale(locale.LC_TIME, "de_DE") 
from tqdm import tqdm

In [11]:
#df aus Zwischenspeicher laden
# na_val = ['-1.#IND', '1.#QNAN', '1.#IND', '-1.#QNAN', '#N/A N/A', '#N/A', 'N/A', \
#              'n/a', '', '#NA', 'NULL', 'null',  '-NaN', 'nan', '-nan', '']

# df_kum = pd.read_clipboard(decimal=",", \
#                  keep_default_na=False,\
#                  na_values= na_val)
# df.to_pickle("./data/k.pkl")

In [8]:
df = pd.read_pickle("./data/k.pkl")
dfnk = pd.read_pickle("./data/nk.pkl")
df_scans = pd.read_pickle("./data/scans.pkl")
df_orte = pd.read_pickle("./data/handlungsorte_raw.pkl")
# die df_Datei erweitert u.a. mit Darstellungsformen
df_kum = pd.read_pickle("./data/kum.pkl")
df_position = pd.read_pickle("./data/positionen.pkl")
df_scans = pd.merge(dfnk[["ArtikelId", "Wochentag"]], df_scans, how="left", \
                    left_on="ArtikelId", right_on="TeilartikelVeroeffentlichungId")
df_neue_ressorts = pd.read_pickle("./data/neue_ressorts.pkl")
dfnk = dfnk.merge(df_neue_ressorts, on="SplitId")
df_volltext_sport_bremen = pd.read_pickle("./data/volltext_sport_bremen.pkl")

In [5]:
#df_nk = pd.read_clipboard(decimal=",")
#df_nk.to_pickle("./data/nk.pkl")

In [6]:
#df_nk = pd.read_pickle("./data/nk.pkl")
#df = pd.read_pickle("./data/k.pkl")
#df_scans = pd.read_pickle("./data/scans.pkl")
# Werte in float
#df["LW"] = df["LW"].str.replace(",", ".").astype(float)
#df["BW"] = df["BW"].str.replace(",", ".").astype(float)
#df["DW"] = df["DW"].str.replace(",", ".").astype(float)

# data_url_nk = "./data/NK.csv"
# df_nk = pd.read_csv(data_url_nk ,\
#                  keep_default_na=False,\
#                  na_values= na_val)
# df_position = pd.read_excel("./data/abfrage_sonderauswertung.xlsx" , sheet_name = "Positionsangaben",\
#                  keep_default_na=False,\
#                  na_values= na_val)

# df_all = pd.read_excel("./data/abfrage_sonderauswertung.xlsx" , sheet_name = "alle_artikel_k",\
#                  keep_default_na=False,\
#                  na_values= na_val)

# df_artikel_nb = pd.read_csv("./data/kum_mit_nb.csv")
# df_content = pd.read_csv("./data/all_content.csv")

In [63]:
sql_volltext_sport = """

declare @Von DATE = '2021-10-10',
@Bis DATE = '2021-12-31', 
@Wellestart INTEGER = 2, 
@Welleende INTEGER = 4, 
@Ausgabestart INTEGER = 1, 
@Ausgabeende INTEGER = 3;


SELECT ti.TeilartikelId, ti.Dachzeilentext, ti.Ueberschrifttext, ti.Untertiteltext, ti.Artikeltext
FROM Teilartikelinhalte ti
JOIN vw_Artikellesewerte_Kumuliert_AktiveUndInaktiveWellen kum
ON ti.TeilartikelId = kum.SplitId
WHERE kum.Erscheinungsdatum BETWEEN @Von AND @Bis
AND kum.Wellennummer BETWEEN @Wellestart AND @Welleende
AND kum.Seitentitel = 'Regionalsport'
 

"""

In [65]:
import pandas as pd

from datetime import datetime, timedelta
import pyodbc
import pymssql

In [66]:
server = 'tcp:lw-azuresql-1.database.windows.net'
database = 'LesewertMandantenDB_WKM'
username = 'Mehrwertmacher'
password = 'fW18ObHgb8qVhlQUsNpx'
driver= '{ODBC Driver 17 for SQL Server}'

In [67]:
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [68]:
df_volltext_sport = pd.read_sql(sql_volltext_sport, cnxn)

In [24]:
df_volltext_sport.to_pickle("./data/volltext_sport_bremen.pkl")

# Sonntag - Weserstrand

In [168]:
dfs = df_kum[df_kum["Wochentag"]=="Sonntag"]
dfs.Ressort.value_counts()

Politik           132
Bremen            106
Sport              90
Weser-Strand       72
Vermischtes        52
Niedersachsen      51
Kinderzeitung      46
Wirtschaft         43
Regionalsport      42
Kultur             39
Leserforum         38
Titelseite         34
Rätsel & Roman      7
Name: Ressort, dtype: int64

## Ressort-Vergleich Sonntag/Samstag 

In [6]:
def compare_ressorts(df): 
    cols = ["Ressort", "LW_Sonntag", "LW_Woche", "Anzahl_Sonntag", "Anzahl_Woche"]
    new_df = pd.DataFrame(columns=cols)
    
    all_ressorts = df.Ressort.unique()
    
    df_sonntag = df[df["Wochentag"]=="Sonntag"]
    df_week = df[df["Wochentag"]!="Sonntag"]
    
    for res in all_ressorts: 
        df_s = df_sonntag[df_sonntag["Ressort"]==res]
        df_w = df_week[df_week["Ressort"]==res]
        
        # Lesewerte
        lw_s = df_s.LW.mean()
        lw_w = df_w.LW.mean()
        
        # Anzahl Artikel
        anzahl_sonntag = df_s.shape[0]
        anzahl_week =df_w.shape[0]
        
        new_ser = pd.Series([res, lw_s, lw_w, anzahl_sonntag, anzahl_week], index=cols)
    
        new_df = new_df.append(new_ser, ignore_index=True)
    
    return new_df

df_com = compare_ressorts(df_kum)

In [8]:
df_com.to_csv("./data/vergleich_ressorts_sonntag.csv", decimal=",", index=False)

In [9]:
df_com

,Ressort,LW_Sonntag,LW_Woche,Anzahl_Sonntag,Anzahl_Woche
0,Lokales,NaN,11.399607,0,2645
1,Lokalsport,NaN,3.547519,0,772
2,Wirtschaft,21.077678,15.091081,43,554
3,Niedersachsen,18.790651,20.704426,51,332
4,Bremen,21.058471,22.772679,106,697
5,Kinderzeitung,10.423980,9.681953,46,308
6,Titelseite,32.181722,28.991875,34,381
7,Vermischtes,34.304587,31.236650,52,405
8,Sport,12.121243,12.189512,90,591
9,Medizin,NaN,20.192735,0,30


# Sonderauswertungen'

## Sortierung Ressorts

In [5]:
group_ressorts = df.groupby(by="Ressort", as_index=False).agg(
    {"Seite": "mean"}).sort_values(["Seite"])
counter = 10
for res in group_ressorts.Ressort.unique():
    print(counter)
    group_ressorts.loc[group_ressorts["Ressort"]
                       == res, "Sortierung"] = counter
    counter += 10
group_ressorts.to_clipboard(decimal=",", index=False)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230


In [38]:
df.LW.mean()

14.620337506918197

## Lokale 1 Weserkurier identifizieren

In [23]:


df["Ressort_neu"] = df["Ressort"]
df.loc[(df["Ressort"]=="Lokales") & (df["Seitentitel"]!="Erste Lokalseite"), "Ressort_neu"] = "restl. Lokalseiten"
df.loc[(df["Ressort"]=="Lokales") & (df["Seitentitel"]=="Erste Lokalseite"), "Ressort_neu"] = "Erste Lokalseiten"

def find_bremen_1(df): 
    for dat in df.Datum.unique(): 
        df_ = df[(df["Datum"]==dat) & (df["Ressort"]=="Bremen")]
        erste_seite = df_.Seite.min()
        df.loc[(df.Seite==erste_seite) & (df.Datum==dat), "Ressort_neu"] = "Erste Bremenseite"
        df.loc[(df.Ressort=="Bremen") & (df.Ressort_neu!="Erste Bremenseite"), "Ressort_neu"] = "restl. Bremenseiten"
df_Ressort_neu = find_bremen_1(df)        

In [31]:
df[df.Ressort.isin(["Bremen"])][["Artikel", "Ausgabe", "Ressort", "Ressort_neu", "Seitentitel"]]
df[["SplitId", "Ressort_neu"]].to_csv("./data/ressort_neu.csv", index=False)

In [ ]:
li = ["Bremen", "Lokales"]
df[df["Ressort"].isin()]

## Durchschnittliche Anzahl Artikel pro Erscheinungstag

In [17]:
def average_articles_per_day(df, date_col="Datum"):
    list_with_articlenumber = []
    for day in df[date_col].unique():
        df_ = df[df[date_col] == day]
        list_with_articlenumber.append(df_.shape[0])
    average = np.mean(list_with_articlenumber)
    print(list_with_articlenumber)
    return average


print(average_articles_per_day(df))
print(f'Unit-Test: {df.groupby("Datum").agg({"ArtikelId":"count"}).mean()}.')

# oder einfacher: df.groupby("Datum").agg({"ArtikelId":"count"}).mean()

[213, 179, 169, 195, 178, 190, 184, 189, 188, 236, 195, 200, 211, 218, 166, 202, 188, 213, 189, 181, 204, 219, 172, 206, 188, 204, 174, 208, 167, 191, 194, 187, 176, 175, 170, 193, 192, 185, 165, 203, 192, 209, 162, 209, 167, 194, 167, 186, 108, 108, 96, 105, 93, 110, 132]
179.9090909090909
Unit-Test: ArtikelId    179.909091
dtype: float64.


## Tagesausgabe mit dem höchsten Lesewert


In [15]:
def edition_with_highest_value(df, date_col="Datum"):
    start_val = 0
    best_day = ""
    all_dates = df[date_col].unique()
    for date in all_dates:
        mean_average = df[df[date_col] == date]["LW"].mean()
        if mean_average > start_val:
            start_val = mean_average
            best_day = date
    return best_day, start_val


best_day, lesewert = edition_with_highest_value(df)
print(f"Bester Tag: {best_day}, Wert: {round(lesewert, 2)}")

Bester Tag: 28.11.2021, Wert: 19.04


## Leser pro Wochentag

In [24]:
def readers_per_day(df): 
    cols = ["Datum", "Anzahl_Leser", "Wochentag"]
    new_df = pd.DataFrame(columns=cols)
    for date in df.Erscheinungsdatum.unique(): 
        df_ = df[df["Erscheinungsdatum"]==date]
        number_reader = df_.WellenteilnahmenId.nunique()
        wochentag = df_.iloc[0]["Wochentag"]
        
        new_ser = pd.Series([date, number_reader, wochentag], index=cols)
        new_df = new_df.append(new_ser, ignore_index=True)
    return new_df

df_reader = readers_per_day(df_scans)
df_reader.groupby("Wochentag", as_index=False).agg({"Anzahl_Leser":"mean"}).to_csv("./data/leser_pro_wochentag.csv", \
                                                                                  decimal=",", index=False)

In [20]:
df_scans

,ArtikelId,Wochentag,Id,Dateidatum,Treffer,TeilnehmerDurchlesewert,Erfassungsdatum,Erfassungsuhrzeit,Abgeschlossen,WellenteilnahmenId,TeilartikelVeroeffentlichungId,Erscheinungsdatum,Seitennummer,Id.1
0,34105,Montag,14273,18.10.2021 08:01,True,87.465942,"Montag, 18. Oktober 2021",10:01:22,False,238,34105,"Montag, 18. Oktober 2021",NaN,3
1,34105,Montag,13190,18.10.2021 06:36,True,19.618528,"Montag, 18. Oktober 2021",08:36:25,False,215,34105,"Montag, 18. Oktober 2021",NaN,3
2,34105,Montag,13353,18.10.2021 06:51,True,87.465942,"Montag, 18. Oktober 2021",08:51:19,False,203,34105,"Montag, 18. Oktober 2021",NaN,3
3,34105,Montag,41643,22.10.2021 19:13,True,15.531335,"Freitag, 22. Oktober 2021",21:13:35,False,255,34105,"Montag, 18. Oktober 2021",NaN,3
4,34105,Montag,15004,18.10.2021 09:36,True,88.555855,"Montag, 18. Oktober 2021",11:36:11,False,35,34105,"Montag, 18. Oktober 2021",NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279491,52960,Sonntag,250356,28.11.2021 08:17,True,96.062996,"Sonntag, 28. November 2021",08:50:29,False,132,52960,"Sonntag, 28. November 2021",NaN,3
279492,52960,Sonntag,252101,28.11.2021 10:50,True,96.587929,"Sonntag, 28. November 2021",11:49:59,False,204,52960,"Sonntag, 28. November 2021",NaN,3
279493,52960,Sonntag,250976,28.11.2021 08:46,True,96.062996,"Sonntag, 28. November 2021",09:46:24,False,218,52960,"Sonntag, 28. November 2021",NaN,3
279494,52960,Sonntag,250032,28.11.2021 08:16,True,97.637794,"Sonntag, 28. November 2021",08:20:42,False,209,52960,"Sonntag, 28. November 2021",NaN,3


## Ressorts nichtkum im Vergleich


In [90]:
def create_ressorts_lw(df): 
    cols = ["Ressort", "Lesewert WK", "Lesewert ACH", "Lesewert NOR", "Sortierung"]
    new_df = pd.DataFrame(columns=cols)
    
    for ressort in df.Ressort_neu.unique(): 
        df_res = df[df["Ressort_neu"]==ressort]
        lw_wk = df_res[df_res["Ausgabe"]=="WK"].LW.mean()
        lw_ach = df_res[df_res["Ausgabe"]=="ACH"].LW.mean()
        lw_nor = df_res[df_res["Ausgabe"]=="NOR"].LW.mean()
        sorter = df_res.Seite.mean()
        new_ser = pd.Series([ressort, lw_wk, lw_ach, lw_nor, sorter], index=cols)
        new_df = new_df.append(new_ser, ignore_index=True)
    
    return new_df

df_ressorts_nk = create_ressorts_lw(dfnk)
        

In [92]:
df_ressorts_nk.to_csv("./data/ressorts_vergleich.csv", index=False, decimal=",")

## Lesewert nach Wortanzahl

In [9]:
dfb = dfnk[dfnk["Ressort"]=="Bremen"]

In [10]:
dfb

,SplitId,ArtikelId,Artikel,ErscheinungsId,AusgabeId,Datum,Seite,BW,DW,LW,...,Ausgabeteil,Wochentag,offen/codiert,Spalte,Ressort,Ressort_neu.1,Worte,Wortzahl_neu,Wortzahl_neu_col,Ressort_neu
546,19435,43851,Patientenverfügung schnell griffbereit,270,1,"Dienstag, 9. November 2021",10,13.235294,80.925352,10.710708,...,Mantel,Dienstag,codiert,Gemeinsam In Bremen,Bremen,restl. Bremenseiten,92,unter 500,unter 100,restl. Bremenseiten
547,19435,43975,Patientenverfügung schnell griffbereit,271,3,"Dienstag, 9. November 2021",10,15.517241,71.379428,11.076118,...,Mantel,Dienstag,codiert,Gemeinsam In Bremen,Bremen,restl. Bremenseiten,92,unter 500,unter 100,restl. Bremenseiten
548,19435,44155,Patientenverfügung schnell griffbereit,272,2,"Dienstag, 9. November 2021",10,7.407407,69.969040,5.182892,...,Mantel,Dienstag,codiert,Gemeinsam In Bremen,Bremen,restl. Bremenseiten,92,unter 500,unter 100,restl. Bremenseiten
585,20147,45388,32-Jähriger wehrt sich gegen Festnahme,279,1,"Freitag, 12. November 2021",11,35.483871,83.756467,29.720037,...,Mantel,Freitag,codiert,Bremen,Bremen,restl. Bremenseiten,134,unter 500,100 bis 299,restl. Bremenseiten
586,20147,45510,32-Jähriger wehrt sich gegen Festnahme,280,3,"Freitag, 12. November 2021",11,33.333333,94.427243,31.475748,...,Mantel,Freitag,codiert,Bremen,Bremen,restl. Bremenseiten,134,unter 500,100 bis 299,restl. Bremenseiten
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22340,23626,53452,Save the date: 7. Dezember 17.30 Uhr,333,1,"Dienstag, 30. November 2021",10,1.785714,4.687500,0.083705,...,Mantel,Dienstag,codiert,Gemeinsam In Bremen,Bremen,restl. Bremenseiten,84,unter 500,unter 100,restl. Bremenseiten
22341,23626,53647,Save the date: 7. Dezember 17.30 Uhr,334,3,"Dienstag, 30. November 2021",10,3.571429,20.399305,0.728547,...,Mantel,Dienstag,codiert,Gemeinsam In Bremen,Bremen,restl. Bremenseiten,84,unter 500,unter 100,restl. Bremenseiten
22347,25064,56703,Save the date: 11. Dezember 9 Uhr,354,1,"Dienstag, 7. Dezember 2021",10,3.703704,81.559402,3.020719,...,Mantel,Dienstag,codiert,Gemeinsam In Bremen,Bremen,restl. Bremenseiten,58,unter 500,unter 100,restl. Bremenseiten
22348,25064,56892,Save the date: 11. Dezember 9 Uhr,355,3,"Dienstag, 7. Dezember 2021",10,5.882353,55.610559,3.271209,...,Mantel,Dienstag,codiert,Gemeinsam In Bremen,Bremen,restl. Bremenseiten,58,unter 500,unter 100,restl. Bremenseiten


In [14]:
alle_seiten

array(['unter 100', '100 bis 299', '300 bis 499', '500 bis 699',
       '700 bis 899', 'mehr als 1300', '900 bis 1099'], dtype=object)

In [20]:
dfb = dfb[dfb["Platzierungen"]=="AA"]

In [25]:
alle_seiten = dfb.Wortzahl_neu_col.unique()
cols = ["Wortzahl", "LW ACH", "LW NOR", "LW WK", "DW ACH", "DW NOR", "DW WK", "Sortierung", "Anzahl"]
df_w = pd.DataFrame(columns=cols)
for word in dfb.Wortzahl_neu_col.unique():
    if word == "unter 100": 
        sortierung = 1
    elif word == '100 bis 299': 
        sortierung = 2
    elif word == '300 bis 499': 
        sortierung = 3
    elif word == '500 bis 699': 
        sortierung = 4
    elif word == '700 bis 899': 
        sortierung = 5
    elif word == '900 bis 1099': 
        sortierung = 6
    elif word == "1100 bis 1299": 
        sortierung = 7
    elif word == "mehr als 1300": 
        sortierung = 8
    else: 
        sortierung = 9
    
    
    df_ = dfb[dfb["Wortzahl_neu_col"]==word]
    anzahl = round(df_.shape[0] / 3)
    lw_ach = df_.loc[df_["Ausgabe"]=="ACH"].LW.mean()
    lw_nor = df_.loc[df_["Ausgabe"]=="NOR"].LW.mean()
    lw_wk = df_.loc[df_["Ausgabe"]=="WK"].LW.mean()
    dw_ach = df_.loc[df_["Ausgabe"]=="ACH"].DW.mean()
    dw_nor = df_.loc[df_["Ausgabe"]=="NOR"].DW.mean()
    dw_wk = df_.loc[df_["Ausgabe"]=="WK"].DW.mean()
    
    
    ser_data = [word, lw_ach, lw_nor, lw_wk,dw_ach, dw_nor, dw_wk, sortierung, anzahl]
    new_ser = pd.Series(ser_data, index=cols)
    df_w = df_w.append(new_ser, ignore_index=True)
    
df_w

,Wortzahl,LW ACH,LW NOR,LW WK,DW ACH,DW NOR,DW WK,Sortierung,Anzahl
0,700 bis 899,22.349928,28.795628,33.287470,61.808851,73.238077,78.286411,5,62
1,500 bis 699,21.773961,27.445183,33.748907,61.334846,71.298376,76.344615,4,120
2,300 bis 499,15.119020,20.675053,22.334998,55.657086,67.917239,73.787764,3,18
3,mehr als 1300,21.529019,22.739687,25.517839,67.890310,70.132629,77.469469,8,3
4,900 bis 1099,15.014974,23.332586,27.005683,59.476566,70.862688,77.896984,6,13
5,100 bis 299,32.925130,31.418375,35.750388,82.312826,87.640731,89.375970,2,1


In [26]:
df_w.loc[df_w["Wortzahl"]=="mehr als 1300", "Wortzahl"] = "mehr als 1100" 

In [27]:
df_w.to_csv("./data/wortzahl_bremen_aufmacher_dw.csv", index=False, decimal=",")

In [128]:
dfb.columns

Index(['SplitId', 'ArtikelId', 'Artikel', 'ErscheinungsId', 'AusgabeId',
       'Datum', 'Seite', 'BW', 'DW', 'LW', 'Seitentitel', 'Platzierungen',
       'Darstellungsformen', 'Beschreibung', 'Handlungsorte', 'Akteure',
       'Themen', 'Ausgabe', 'Layerlink', 'Ausgabeteil', 'Wochentag',
       'offen/codiert', 'Spalte', 'Ressort', 'Ressort_neu.1', 'Worte',
       'Wortzahl_neu', 'Ressort_neu'],
      dtype='object')

## Artikel < LW10 > LW30

In [22]:
def create_u10_and_ü30(df, u=10, ü=30): 
    anzahl_u = df[df["LW"]<u].shape[0]
    anzahl_ü = df[df["LW"]>ü].shape[0]
    for num in [anzahl_u, anzahl_ü]:
        
        percentage = round(num/df.shape[0]*100,1)
        if num == anzahl_u: 
            print(f"Unter LW{u}: {num} Artikel ({percentage}%)")
        else: 
            print(f"Über LW{ü}: {num} Artikel ({percentage}%)")
        
create_u10_and_ü30(df)

Unter LW10: 4503 Artikel (45.5%)
Über LW30: 1217 Artikel (12.3%)


In [29]:
d = pd.read_clipboard()

In [34]:
d = d[d["Frage"]!="Gibt es Themen, die Sie besonders interessieren, die aber in Ihrer Süddeutschen zu kurz kommen oder ganz fehlen?"]

In [37]:
d.groupby(by=["Frage", "Antwort"] ,as_index=False).agg({"Anzahl":"sum"}).to_clipboard()

In [33]:
d.to_clipboard()

## Check ob Lokalteile andere Werte zwischen kumuliert und nicht kumuliert aufweisen

In [26]:
def check_lokalteile(df, dfnk): 
    for aus in dfnk.AusgabeId.unique(): 
        df_ = df[(df["AusgabeId"]==aus) & (df["Ressort"]=="Lokales")]
        print(f"Lokales Ausgabe kum {aus}, Anzahl {df_.shape}, LW Ø {df_.LW.mean()}")
        dfnk_ = dfnk[(dfnk["AusgabeId"]==aus) & (dfnk["Ressort"]=="Lokales")]
        print(f"Lokales Ausgabe nichtkum{aus}, Anzahl {dfnk_.shape}, LW Ø {dfnk_.LW.mean()}")
check_lokalteile(df, dfnk)

Lokales Ausgabe kum 2, Anzahl (1304, 23), LW Ø 11.033987068744668
Lokales Ausgabe nichtkum2, Anzahl (1304, 24), LW Ø 11.033987068744668
Lokales Ausgabe kum 3, Anzahl (1049, 23), LW Ø 13.221685727313433
Lokales Ausgabe nichtkum3, Anzahl (1049, 24), LW Ø 13.221685727313432
Lokales Ausgabe kum 1, Anzahl (292, 23), LW Ø 6.4866189020103455
Lokales Ausgabe nichtkum1, Anzahl (292, 24), LW Ø 6.486618902010346


In [23]:
dfnk

,SplitId,ArtikelId,Artikel,ErscheinungsId,AusgabeId,Datum,Seite,BW,DW,LW,...,Handlungsorte,Akteure,Themen,Ausgabe,Layerlink,Ausgabeteil,Wochentag,offen/codiert,Spalte,Ressort
0,15207,34105,Farbspur auf der Fahrbahn,209,2,"Montag, 18. Oktober 2021",28,19.047619,62.602181,11.924225,...,NaN,NaN,"Persönliche Finanzen, Brand-, Hausrat- und Geb...",ACH,https://mein.lesewert.de/1022/Coding/Page/Deta...,Lokales ACH,Montag,codiert,NaN,Lokales
1,15209,34106,Von Erntemaschine erfasst,209,2,"Montag, 18. Oktober 2021",28,30.952381,66.586538,20.610119,...,NaN,NaN,"News, Polizei, Feuerwehr und Unfall, Kriminali...",ACH,https://mein.lesewert.de/1022/Coding/Page/Deta...,Lokales ACH,Montag,codiert,NaN,Lokales
2,15423,34577,Gegenverkehr übersehen,212,2,"Dienstag, 19. Oktober 2021",32,8.163265,48.291926,3.942198,...,NaN,NaN,"News, Polizei, Feuerwehr und Unfall, Verkehr/M...",ACH,https://mein.lesewert.de/1022/Coding/Page/Deta...,Lokales ACH,Dienstag,codiert,NaN,Lokales
3,15430,34601,Stromstörung in Bierden,212,2,"Dienstag, 19. Oktober 2021",30,16.326531,89.634616,14.634223,...,NaN,NaN,"Persönliche Finanzen, Haustechnik, private Hei...",ACH,https://mein.lesewert.de/1022/Coding/Page/Deta...,Lokales ACH,Dienstag,codiert,NaN,Lokales
4,15597,34926,56-Jährige fährt über Verkehrsinsel,215,2,"Mittwoch, 20. Oktober 2021",32,22.222222,64.861997,14.413777,...,NaN,NaN,"News, Polizei, Feuerwehr und Unfall, Verkehr/M...",ACH,https://mein.lesewert.de/1022/Coding/Page/Deta...,Lokales ACH,Mittwoch,codiert,NaN,Lokales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22501,23331,52857,NaN,328,3,"Sonntag, 28. November 2021",8,43.137255,90.053474,38.846597,...,NaN,NaN,Society/Promis/Klatsch & Tratsch,NOR,https://mein.lesewert.de/1022/Coding/Page/Deta...,Mantel,Sonntag,codiert,Vermischtes,Vermischtes
22502,23364,52869,NaN,328,3,"Sonntag, 28. November 2021",8,37.254902,88.382376,32.926767,...,NaN,NaN,Society/Promis/Klatsch & Tratsch,NOR,https://mein.lesewert.de/1022/Coding/Page/Deta...,Mantel,Sonntag,codiert,Vermischtes,Vermischtes
22503,23326,52884,NaN,329,2,"Sonntag, 28. November 2021",8,47.727273,92.943986,44.359630,...,NaN,NaN,Society/Promis/Klatsch & Tratsch,ACH,https://mein.lesewert.de/1022/Coding/Page/Deta...,Mantel,Sonntag,codiert,Vermischtes,Vermischtes
22504,23331,52906,NaN,329,2,"Sonntag, 28. November 2021",8,47.727273,84.693431,40.421865,...,NaN,NaN,Society/Promis/Klatsch & Tratsch,ACH,https://mein.lesewert.de/1022/Coding/Page/Deta...,Mantel,Sonntag,codiert,Vermischtes,Vermischtes


## Ressorts nach Geschlecht

In [8]:
df_g = pd.read_pickle("./data/nk_geschlecht.pkl")

In [16]:
df_g.columns

Index(['TeilartikelVeroeffentlichungId', 'TeilartikelId', 'Platzierung',
       'LW_w', 'BW_w', 'DW_w', 'LW_m', 'BW_m', 'DW_m', 'LW_40bis59',
       'BW_40bis59', 'DW_40bis59', 'LW_60bis79', 'BW_60bis79', 'DW_60bis79'],
      dtype='object')

In [17]:
df_merge = pd.merge(df_g, df[["ArtikelId", "Ressort"]],
                    left_on="TeilartikelVeroeffentlichungId", right_on="ArtikelId", )

In [21]:
df_group = df_merge.groupby("Ressort", as_index=False).agg({
    "LW_w": "mean", "LW_m": "mean"})
df_group

,Ressort,LW_w,LW_m
0,Auto,4.464312,15.925142
1,Beruf & Karriere,13.400934,14.012071
2,Einblicke,22.832660,22.273341
3,Gut zu wissen,18.932958,19.522087
4,Hamburg / Schleswig-Holstein,21.430791,22.274083
5,Immobilienmarkt,5.339525,12.267466
6,Kinderzeitung,5.426673,6.783284
7,Kultur,10.214907,10.413151
8,Leute,22.466561,20.721497
9,Lokales Hamburg & Umland,27.096016,25.885175


## LW nach Wochentagen


In [107]:
def create_df_number_of_days(df):
    all_weekdays = df.Wochentag.unique()
    dict_fin = {}
    for day in all_weekdays:
        df_ = df[df["Wochentag"] == day]
        anzahl_tage = df_.Datum.nunique()
        dict_fin[day] = anzahl_tage
    df_new = pd.DataFrame(pd.Series(dict_fin).reset_index())
    return df_new


df_anzahl_wochentage = create_df_number_of_days(df)
df_anzahl_wochentage.columns = ["Wochentag", "Anzahl"]

df_wt = df.groupby(by="Wochentag", as_index=False).agg(
    {"LW": "mean", "ArtikelId": "count"})
df_wt = df_wt.merge(df_anzahl_wochentage, on="Wochentag")


def sorter(el): 
    if el == "Montag": 
        return 1
    elif el == "Dienstag": 
        return 2
    elif el == "Mittwoch": 
        return 3
    elif el == "Donnerstag": 
        return 4
    elif el == "Freitag": 
        return 5
    elif el == "Samstag": 
        return 6
    else: 
        return 7
df_wt["Sorter"] = df_wt["Wochentag"].apply(sorter)
df_wt["Anzahl"] = df_wt["ArtikelId"] / df_wt["Anzahl"]
df_wt = df_wt.sort_values(by="Sorter")
df_wt.drop("ArtikelId", axis=1, inplace=True)
df_wt.to_clipboard(index=False ,decimal=",")

In [105]:
df_wt

,Wochentag,LW,ArtikelId,Anzahl,Sorter
4,Montag,14.296687,1533,8,1
0,Dienstag,14.538577,1552,8,2
3,Mittwoch,15.035629,1409,8,3
1,Donnerstag,13.890518,1632,8,4
2,Freitag,15.064839,1429,8,5
5,Samstag,13.594358,1588,8,6
6,Sonntag,17.576498,752,7,7


## Leserzahl nach Stunden

In [2]:
df_scans["Stunde"] = df_scans["Erfassungsuhrzeit"].str[:2].astype(int)
df_group = df_scans.groupby("Stunde", as_index=False)

all_hours = list(range(24))
all_cols = all_hours.copy()
all_cols.insert(0, "Datum")
all_cols

all_date = df_scans.Erscheinungsdatum.unique()

from collections import defaultdict
new_df = pd.DataFrame(columns=all_cols)

for dat in all_date:
    new_dict = defaultdict(int)
    df_ = df_scans[df_scans["Erscheinungsdatum"] == dat]
    new_dict["Datum"] = dat
    for hour in all_hours:
        df__ = df_[df_["Stunde"] == hour].WellenteilnahmenId.nunique()
        new_dict[hour] = df__

    new_ser = pd.Series(new_dict)

    new_df = new_df.append(new_ser, ignore_index=True)
    
nn = new_df.mean()
df_fin = pd.DataFrame([nn]).transpose().reset_index()
df_fin.columns = ["Uhrzeit", "Leser"]

NameError: name 'df_scans' is not defined

In [1]:
df_fin.to_clipboard(index=False)

NameError: name 'df_fin' is not defined

In [73]:
df_fin

,Uhrzeit,Leser
0,0,0.275862
1,1,0.120690
2,2,0.051724
3,3,0.000000
4,4,0.000000
5,5,0.137931
6,6,4.568966
7,7,7.155172
8,8,7.793103
9,9,6.948276


## Gelesene Artikel pro Leser und pro Wochentag

In [3]:
# Split_Id und Wochentag an die Scans mergen
df_merge = pd.merge(df_scans, \
                    dfnk[["ArtikelId", "Wochentag", "SplitId"]], \
                    left_on="TeilartikelVeroeffentlichungId", right_on="ArtikelId", \
                   how="left")

In [15]:
# bei den Scans laufen smämtliche Scanzeilen ein, auch auf Artikel, die später mit NB gekennzeichnet wurden
# daher müssen alle Scans raus, die auf Artikel gesetzt wurden, die sich nicht mehr in der bereinigten nichtkumulierten
# Datei wiederfinden. 
df_merge_bereinigt = df_merge[~df_merge["SplitId"].isnull()]

In [16]:
df_merge_bereinigt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 279496 entries, 0 to 284994
Data columns (total 15 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Id                              279496 non-null  int64  
 1   Dateidatum                      279496 non-null  object 
 2   Treffer                         279496 non-null  bool   
 3   TeilnehmerDurchlesewert         277078 non-null  float64
 4   Erfassungsdatum                 279496 non-null  object 
 5   Erfassungsuhrzeit               279496 non-null  object 
 6   Abgeschlossen                   279496 non-null  bool   
 7   WellenteilnahmenId              279496 non-null  int64  
 8   TeilartikelVeroeffentlichungId  279496 non-null  int64  
 9   Erscheinungsdatum               279496 non-null  object 
 10  Seitennummer                    0 non-null       float64
 11  Id.1                            279496 non-null  int64  
 12  ArtikelId       

In [25]:
def articles_per_reader(df): 
    new_df = pd.DataFrame(columns=["Datum", "Wochentag", "gelesene_Artikel"])
    for date in df.Erscheinungsdatum.unique(): 
        print(date)
        df_ = df[df["Erscheinungsdatum"]==date]
        wochentag = df_.iloc[0].Wochentag
        print(wochentag)
        
        print(f"Anzahl Leser: {df_.WellenteilnahmenId.nunique()}")
        for reader in df_.WellenteilnahmenId.unique():
            df_reader = df_[df_.WellenteilnahmenId==reader]
            number_articles = df_reader.SplitId.nunique()
            
            new_ser = pd.Series([date, wochentag, number_articles], index=new_df.columns)
            
            
            
            new_df = new_df.append(new_ser, ignore_index=True)
    
        print("-------------")
    
    return new_df
    
    
df_artikel_pro_tag = articles_per_reader(df_merge_bereinigt)

Dienstag, 26. Oktober 2021
Dienstag
Anzahl Leser: 183
-------------
Dienstag, 7. Dezember 2021
Dienstag
Anzahl Leser: 153
-------------
Donnerstag, 9. Dezember 2021
Donnerstag
Anzahl Leser: 160
-------------
Donnerstag, 11. November 2021
Donnerstag
Anzahl Leser: 168
-------------
Montag, 18. Oktober 2021
Montag
Anzahl Leser: 147
-------------
Mittwoch, 1. Dezember 2021
Mittwoch
Anzahl Leser: 160
-------------
Sonntag, 21. November 2021
Sonntag
Anzahl Leser: 141
-------------
Sonntag, 5. Dezember 2021
Sonntag
Anzahl Leser: 144
-------------
Sonntag, 14. November 2021
Sonntag
Anzahl Leser: 160
-------------
Samstag, 27. November 2021
Samstag
Anzahl Leser: 154
-------------
Mittwoch, 24. November 2021
Mittwoch
Anzahl Leser: 168
-------------
Mittwoch, 17. November 2021
Mittwoch
Anzahl Leser: 168
-------------
Sonntag, 7. November 2021
Sonntag
Anzahl Leser: 161
-------------
Mittwoch, 20. Oktober 2021
Mittwoch
Anzahl Leser: 168
-------------
Mittwoch, 10. November 2021
Mittwoch
Anzahl Lese

In [35]:
df_artikel_pro_Wochentag = df_artikel_pro_tag.groupby(by=["Wochentag"], as_index=False).agg({"gelesene_Artikel":"median"})

In [36]:
df_artikel_pro_Wochentag

,Wochentag,gelesene_Artikel
0,Dienstag,25.0
1,Donnerstag,24.0
2,Freitag,24.0
3,Mittwoch,23.0
4,Montag,24.0
5,Samstag,23.0
6,Sonntag,20.0


In [33]:
df_artikel_pro_Wochentag

,Wochentag,gelesene_Artikel
0,Dienstag,30.687870
1,Donnerstag,30.300831
2,Freitag,29.730208
3,Mittwoch,28.878810
4,Montag,29.277444
5,Samstag,29.621118
6,Sonntag,23.920076


In [37]:
df_artikel_pro_Wochentag.to_csv("./data/gelesen_artikel_pro_wochentag.csv", index=False, decimal=",")

In [28]:
df_artikel_pro_tag

,Datum,Wochentag,gelesene_Artikel
0,"Dienstag, 26. Oktober 2021",Dienstag,101
1,"Dienstag, 26. Oktober 2021",Dienstag,91
2,"Dienstag, 26. Oktober 2021",Dienstag,11
3,"Dienstag, 26. Oktober 2021",Dienstag,90
4,"Dienstag, 26. Oktober 2021",Dienstag,57
...,...,...,...
8985,"Mittwoch, 8. Dezember 2021",Mittwoch,3
8986,"Mittwoch, 8. Dezember 2021",Mittwoch,8
8987,"Mittwoch, 8. Dezember 2021",Mittwoch,8
8988,"Sonntag, 31. Oktober 2021",Samstag,1


In [24]:
df_artikel_pro_tag[df_artikel_pro_tag["Wochentag"]]

,Datum,Wochentag,gelesene_Artikel
0,"Dienstag, 26. Oktober 2021",Dienstag,101
1,"Dienstag, 26. Oktober 2021",Dienstag,91
2,"Dienstag, 26. Oktober 2021",Dienstag,11
3,"Dienstag, 26. Oktober 2021",Dienstag,90
4,"Dienstag, 26. Oktober 2021",Dienstag,57
...,...,...,...
9049,"Samstag, 16. Oktober 2021",NaN,0
9050,"Sonntag, 31. Oktober 2021",Samstag,1
9051,"Sonntag, 31. Oktober 2021",Samstag,10
9052,"Freitag, 15. Oktober 2021",NaN,0


# Darstellungsformen und Platzierungen

## Platzierungen 

In [4]:
df.Platzierungen.value_counts()

TS1    5596
AA     1994
SK     1031
TS2     739
ÜA      248
ZA      227
TS3      60
Name: Platzierungen, dtype: int64

In [5]:
# Platzierungen Klarnamen
platzierung_dict = {"AA": "Aufmacher", "ZA": "Zweitaufmacher", "SK": "Seitenkeller", "ÜA": "Überaufmacher", "TS1":"Einspalter", 
                    "TS2":"Zweispalter", "TS3":"Dreispalter", "TS4":"Vierspalter", "TS5": "Fünfspalter", "TS6": "Sechsspalter"}


In [6]:
# Darstellungsformen sortieren
df["PL_basis"] = ""

def format_darstellungsformen_basis(df): 
    for i in df.index: 
        ds = df.at[i, "Platzierungen"]
        if ds == "AA": 
            df.at[i, "PL_basis"] = "Aufmacher"
        elif ds == "SK": 
            df.at[i, "PL_basis"] = "Seitenkeller"
        elif ds == "TS1": 
            df.at[i, "PL_basis"] = "Einspalter"
        else: 
            df.at[i, "PL_basis"] = "andere"
            
    return df

df = format_darstellungsformen_basis(df)

## Darstellungsformen

In [7]:
df.Darstellungsformen.unique()

array(['NA', 'AK, NA', 'HG', 'BF', 'AK, HG', 'AK, BF', 'HG, NA',
       'HG, NA, RK', 'BN', 'HG, KM', 'RP', 'AK, RP', 'AK, BN', 'FF', 'BB',
       'LB', 'GT', 'SA', 'KM', 'RK', 'KI', 'ZU', 'KM, GA', 'IV', 'HG, SA',
       'AK, HG, NA', 'HG, KI, NA', 'BF, KI', 'HG, ZU', 'BN, HG', 'BF, HG',
       'HG, IV', 'AK, RK', nan, 'BF, SE', 'RP, SE', 'KI, RP', 'KI, SA',
       'ES, GA', 'KM, SE', 'AK, BB', 'SE, SA', 'HG, SE', 'IV, SE',
       'RK, SE', 'KI, RP, SE'], dtype=object)

In [99]:
def sortierung_darstellungsformen(df): 
    
    # BB Bildbericht in Bildnachricht überführen
    df.loc[df.Darstellungsformen == "BB", "Darstellungsformen"] = "BN"
    
    df["DF"] = ""
    
    # mit den wichtige_df klopfen wir die Einträge ab, die zwei DF tragen, z.B. AK und KM. 
    # für die Statistik kann ich nur einen verwenden, daher lege ich mit wichtige_df fest, welche DF
    # im Fall von zwei DFs bei einem Artikel genommen wird. 
    # alle_df hingegen ist die Referenzliste, wenn ich nur einen Eintrag habe. 
    # Ansonsten kommt es zu häufig vor, dass z.B. Artikel nur als GA (Gastartikel) gelabelt sind, aber 
    # als nichts anderes. Wenn ich da nur die wichtige_df nutze, würden diese Artikel keine DF erhalten. 
    wichtige_df = ["BF", "RP", "IV", "KM", "BF", "LB", "SA", "NA", "RK", "ZU", "BN", "ES", \
                  # Serie und Gastartikel nehme ich raus, da es da immer wieder zu doppelmeldungen kommt.
                  # "SE", "GA"
                  #"SE", "GT", "FF"
                  ]
    alle_df = ["BF", "RP", "IV", "KM", "BF", "LB", "SA", "NA", "RK", "ZU", "BN", "ES", "SE", "GA", "HG" ,"GT", "FF"]
    klarnamen_dict = darstellung_dict = {"NA": "Nachricht", "AK": "Ankündigung", "SA": "Struktur. Artikel", 
                                         "ZU": "Zitattext/Umfrage",  "SE":"Serie", "GA": "Gastartikel",\
                                         "FF": "Freies Format", 'LB':'Leserbrief','ES': 'Essay', \
                        'RK': 'Rezension/Kritik', 'KM': 'Komm./Kolumne', 'BB': 'Bildbericht', 'BN': 'Bildnachricht', \
                        'GT': 'Geschichtstext', 'KI': 'Karikatur/Illustration/Grafik', 'ST': 'Serviceelement/Tabelle', \
                        'HG': 'Hintergrundkasten', 'IV': 'Interview', "RP": "Reportage/Porträt",\
                                         "BF":"Bericht/Feature", "BI": "Bildnachricht" }
    
    df.loc[df["Darstellungsformen"].isnull(), "Darstellungsformen"] = ""
    for i in df.index: 
        darstellungsform = df.at[i, "Darstellungsformen"]
        
            
        if len(darstellungsform)>3: 
            
            dar_split = darstellungsform.split(", ")
            
            if (dar_split[0] in wichtige_df) and (dar_split[1] in wichtige_df): 
                print(f"Achtung, {dar_split}")
            
            for el in range(0, len(dar_split)): 
                try:
                    if dar_split[el] in wichtige_df: 
                        df.at[i, "DF"] = klarnamen_dict[dar_split[el]]
                except: 
                    print(f"DDDDD {darstellungsform}, {dar_split}, und noch el : {el}")

        else: 
            if darstellungsform in alle_df: 
                df.at[i, "DF"] = klarnamen_dict[darstellungsform]
            else: 
                print(f"Achtugn, {darstellungsform} wurde nicht gefunden.")
        
        check_beschreibung = df.at[i, "Beschreibung"] 
        if check_beschreibung == "Ihre Meinung": 
            df.at[i, "Darstellungsformen"] = "LB"
            df.at[i, "DF"] = "Leserbrief"
                
    return df
df_darstellungsformen = sortierung_darstellungsformen(df)

Achtugn, KI wurde nicht gefunden.
Achtugn, KI wurde nicht gefunden.
Achtugn,  wurde nicht gefunden.
Achtugn,  wurde nicht gefunden.
Achtugn,  wurde nicht gefunden.
Achtugn, KI wurde nicht gefunden.
Achtugn,  wurde nicht gefunden.


In [101]:
df_darstellungsformen[["SplitId", "DF"]].to_csv("./data/darstellungsformen.csv", index=False)

## Darstellungsformen und Platzierungen nach LW und Ressort in Tabelle

In [52]:
df_kum[["Darstellungsformen", "DF"]]

,Darstellungsformen,DF
0,NA,Nachricht
1,NA,Nachricht
2,NA,Nachricht
3,NA,Nachricht
4,NA,Nachricht
...,...,...
9890,NA,Nachricht
9891,NA,Nachricht
9892,NA,Nachricht
9893,NA,Nachricht


In [68]:
def create_table(df, col):
    
    df_ = df.pivot_table(index="Ressort", columns=col, values="LW")
    df_ = df_.reset_index()
    return(df_)

df_pl = create_table(df_kum, "PL")
df_df = create_table(df_kum, "DF")
    
    

In [69]:
df_pl = df_pl.round(1).applymap(str).replace("nan", "-").replace("\.", ",", regex=True)
df_df = df_df.round(1).applymap(str).replace("nan", "-").replace("\.", ",", regex=True)

In [70]:
df_pl = df_pl[["Ressort", "Aufmacher", "Zweitaufmacher", "Überaufmacher", "Seitenkeller", "Einspalter", "Zweispalter", "Dreispalter"]]
df_df = df_df[["Ressort", "Aufmacher", "Zweitaufmacher", "Überaufmacher", "Seitenkeller", "Einspalter", "Zweispalter", "Dreispalter"]]

KeyError: "['Aufmacher', 'Zweitaufmacher', 'Überaufmacher', 'Seitenkeller', 'Einspalter', 'Zweispalter', 'Dreispalter'] not in index"

In [71]:
df_pl.to_csv("./data/platzierung_pro_ressort.csv", decimal=",")
df_df.to_csv("./data/darstellungsformen_pro_ressort.csv", decimal=",")

In [41]:
t = df_kum[(df_kum["Ressort"]=="Titelseite") & \
           (df_kum["Platzierungen"]=="TS2") & \
          (df_kum["DF"]!="Hintergrundkasten") & \
          (df_kum["Wochentag"]!="Sonntag")]
#t.sort_values(by="Datum")
t.LW.mean()

8.095090605207377

In [42]:
df_pl

PL,Aufmacher,Zweitaufmacher,Überaufmacher,Seitenkeller,Einspalter,Zweispalter,Dreispalter
Ressort,,,,,,,
Beruf & Karriere,"10,0","10,5",-,"8,2","6,4",-,-
Bremen,"27,1","26,4","25,7","24,9","20,3","12,5","21,6"
Dossier,"21,6",-,-,"13,9",-,-,-
Immobilien & Wohnen,"5,3","5,8",-,"7,0","4,2",-,-
Kinderzeitung,"14,1","10,7","13,4","10,8","7,7","10,7","2,9"
Kultur,"14,5","17,8","16,2","10,5","7,3","6,8","13,0"
Leserforum,"14,6","14,7",-,"10,7","12,4","11,8","11,6"
Lokales,"18,6","10,1","17,4","14,9","8,5","8,7","9,8"
Lokalsport,"5,6","3,8","18,8","4,2","2,3","2,9","2,2"


In [79]:
df_titel_ohne_sonntag = df_kum[(df_kum["Ressort"]=="Titelseite") & (df_kum["Wochentag"]!="Sonntag")]

In [82]:
df_titel_ohne_sonntag.LW.mean()

28.991874941289858

In [83]:
ohne_t = df_titel_ohne_sonntag[(df_tdf_titel_ohne_sonntagitel["Platzierungen"]!="TS2") | \
                  ((df_titel_ohne_sonntag["Platzierungen"]=="TS2") &(df_titel_ohne_sonntag["DF"]=="Hintergrundkasten"))]

In [84]:
ohne_t.LW.mean()

36.83759902415115

# Handlungsorte / Regionen

## Handlungsorte sortieren

In [87]:
orte_weserkurier = ["Bremen", "Bremerhaven", \
                    # Landkreise
                    "Osterholz", "Verden", "Diepholz", "Wesermarsch", "Oldenburg", "Rotenburg", \
                    "Cuxhaven"]

In [48]:
orte_deutschland =[ 'Aach', 'Aachen', 'Aalen', 'Abenberg', 'Abensberg', 'Achern', 'Achim', 'Adelsheim', 'Adenau',
                     'Adorf/Vogtl.', 'Ahaus', 'Ahlen', 'Ahrensburg', 'Aichach', 'Aichtal', 'Aken (Elbe)', 'Albstadt',
                     'Alfeld (Leine)', 'Allendorf (Lumda)', 'Allstedt', 'Alpirsbach', 'Alsdorf', 'Alsfeld',
                     'Alsleben (Saale)', 'Altdorf bei Nürnberg', 'Altena', 'Altenberg', 'Altenburg', 
                     'Altenkirchen (Westerwald)', 'Altensteig', 'Altentreptow', 'Altlandsberg', 'Altötting', 'Alzenau', 
                     'Alzey', 'Amberg', 'Amöneburg', 'Amorbach', 'Andernach', 'Angermünde', 'Anklam',
                     'Annaberg-Buchholz', 'Annaburg', 'Annweiler am Trifels', 'Ansbach', 'Apolda', 'Arendsee (Altmark)',
                     'Arneburg', 'Arnis', 'Arnsberg', 'Arnstadt', 'Arnstein', 'Arnstein', 'Artern/Unstrut', 'Arzberg', 
                     'Aschaffenburg', 'Aschersleben', 'Asperg', 'Aßlar', 'Attendorn', 'Aub', 'Aue', 'Auerbach in der Oberpfalz',
                     'Auerbach/Vogtl.', 'Augsburg', 'Augustusburg', 'Aulendorf', 'Auma-Weidatal', 'Aurich', 'Babenhausen',
                     'Bacharach', 'Backnang', 'Bad Aibling', 'Bad Arolsen', 'Bad Belzig', 'Bad Bentheim', 'Bad Bergzabern', 
                     'Bad Berka', 'Bad Berleburg', 'Bad Berneck im Fichtelgebirge', 'Bad Bevensen', 'Bad Bibra', 
                     'Bad Blankenburg', 'Bad Bramstedt', 'Bad Breisig', 'Bad Brückenau', 'Bad Buchau', 'Bad Camberg', 
                     'Bad Colberg-Heldburg', 'Bad Doberan', 'Bad Driburg', 'Bad Düben', 'Bad Dürkheim', 'Bad Dürrenberg',
                     'Bad Dürrheim', 'Bad Elster', 'Bad Ems', 'Baden-Baden', 'Bad Fallingbostel', 'Bad Frankenhausen/Kyffhäuser',
                     'Bad Freienwalde (Oder)', 'Bad Friedrichshall', 'Bad Gandersheim', 'Bad Gottleuba-Berggießhübel',
                     'Bad Griesbach im Rottal', 'Bad Harzburg', 'Bad Herrenalb', 'Bad Hersfeld', 'Bad Homburg vor der Höhe', 
                     'Bad Honnef', 'Bad Hönningen', 'Bad Iburg', 'Bad Karlshafen', 'Bad Kissingen', 'Bad König', 
                     'Bad Königshofen im Grabfeld', 'Bad Köstritz', 'Bad Kötzting', 'Bad Kreuznach', 'Bad Krozingen', 
                     'Bad Laasphe', 'Bad Langensalza', 'Bad Lauchstädt', 'Bad Lausick', 'Bad Lauterberg im Harz', 
                     'Bad Liebenstein', 'Bad Liebenwerda', 'Bad Liebenzell', 'Bad Lippspringe', 'Bad Lobenstein', 
                     'Bad Marienberg (Westerwald)', 'Bad Mergentheim', 'Bad Münder am Deister', 'Bad Münstereifel', 
                     'Bad Muskau', 'Bad Nauheim', 'Bad Nenndorf', 'Bad Neuenahr-Ahrweiler', 'Bad Neustadt an der Saale', 
                     'Bad Oeynhausen', 'Bad Oldesloe', 'Bad Orb', 'Bad Pyrmont', 'Bad Rappenau', 'Bad Reichenhall', 
                     'Bad Rodach', 'Bad Sachsa', 'Bad Säckingen', 'Bad Salzdetfurth', 'Bad Salzuflen', 'Bad Salzungen',
                     'Bad Saulgau', 'Bad Schandau', 'Bad Schmiedeberg', 'Bad Schussenried', 'Bad Schwalbach', 'Bad Schwartau',
                     'Bad Segeberg', 'Bad Sobernheim', 'Bad Soden am Taunus', 'Bad Soden-Salmünster', 'Bad Sooden-Allendorf', 
                     'Bad Staffelstein', 'Bad Sulza', 'Bad Sülze', 'Bad Teinach-Zavelstein', 'Bad Tennstedt', 'Bad Tölz', 
                     'Bad Urach', 'Bad Vilbel', 'Bad Waldsee', 'Bad Wildbad', 'Bad Wildungen', 'Bad Wilsnack', 'Bad Wimpfen', 
                     'Bad Windsheim', 'Bad Wörishofen', 'Bad Wünnenberg', 'Bad Wurzach', 'Baesweiler', 'Baiersdorf', 'Balingen', 
                     'Ballenstedt', 'Balve', 'Bamberg', 'Barby', 'Bargteheide', 'Barmstedt', 'Bärnau', 'Barntrup', 'Barsinghausen',
                     'Barth', 'Baruth/Mark', 'Bassum', 'Battenberg (Eder)', 'Baumholder', 'Baunach', 'Baunatal', 'Bautzen', 
                     'Bayreuth', 'Bebra', 'Beckum', 'Bedburg', 'Beelitz', 'Beerfelden', 'Beeskow', 'Beilngries', 'Beilstein', 
                     'Belgern-Schildau', 'Bendorf', 'Bensheim', 'Berching', 'Berga/Elster', 'Bergen', 'Bergen auf Rügen', 
                     'Bergheim', 'Bergisch Gladbach', 'Bergkamen', 'Bergneustadt', 'Berka/Werra', 'Berlin', 'Bernau bei Berlin', 
                     'Bernburg (Saale)', 'Bernkastel-Kues', 'Bernsdorf', 'Bernstadt a. d. Eigen', 'Bersenbrück', 'Besigheim', 
                     'Betzdorf', 'Betzenstein', 'Beverungen', 'Bexbach', 'Biberach an der Riß', 'Biedenkopf', 'Bielefeld',
                     'Biesenthal', 'Bietigheim-Bissingen', 'Billerbeck', 'Bingen am Rhein', 'Birkenfeld', 
                     'Bischofsheim an der Rhön', 'Bischofswerda', 'Bismark (Altmark)', 'Bitburg', 'Bitterfeld-Wolfen', 
                     'Blankenburg (Harz)', 'Blankenhain', 'Blaubeuren', 'Blaustein', 'Bleckede', 'Bleicherode', 'Blieskastel', 
                     'Blomberg', 'Blumberg', 'Bobingen', 'Böblingen', 'Bocholt', 'Bochum', 'Bockenem', 'Bodenwerder', 'Bogen',
                     'Böhlen', 'Boizenburg/Elbe', 'Bonn', 'Bonndorf im Schwarzwald', 'Bönnigheim', 'Bopfingen', 'Boppard',
                     'Borgentreich', 'Borgholzhausen', 'Borken', 'Borken', 'Borkum', 'Borna', 'Bornheim', 'Bottrop', 'Boxberg',
                     'Brackenheim', 'Brake (Unterweser)', 'Brakel', 'Bramsche', 'Brandenburg an der Havel', 'Brand-Erbisdorf',
                     'Brandis', 'Braubach', 'Braunfels', 'Braunlage', 'Bräunlingen', 'Braunsbedra', 'Braunschweig', 'Breckerfeld',
                     'Bredstedt', 'Breisach am Rhein', 'Bremen', 'Bremerhaven', 'Bremervörde', 'Bretten', 'Breuberg', 'Brilon', 
                     'Brotterode-Trusetal', 'Bruchköbel', 'Bruchsal', 'Brück', 'Brüel', 'Brühl', 'Brunsbüttel', 'Brüssow', 
                     'Buchen (Odenwald)', 'Buchholz in der Nordheide', 'Buchloe', 'Bückeburg', 'Buckow (Märkische Schweiz)', 
                     'Büdelsdorf', 'Büdingen', 'Bühl', 'Bünde', 'Büren', 'Burg', 'Burgau', 'Burgbernheim', 'Burgdorf', 'Bürgel', 
                     'Burghausen', 'Burgkunstadt', 'Burglengenfeld', 'Burgstädt', 'Burg Stargard', 'Burgwedel', 'Burladingen', 
                     'Burscheid', 'Bürstadt', 'Buttelstedt', 'Buttstädt', 'Butzbach', 'Bützow', 'Buxtehude', 'Calau',
                     'Calbe (Saale)', 'Calw', 'Castrop-Rauxel', 'Celle', 'Cham', 'Chemnitz', 'Clausthal-Zellerfeld', 'Clingen', 
                     'Cloppenburg', 'Coburg', 'Cochem', 'Coesfeld', 'Colditz', 'Coswig', 'Coswig (Anhalt)', 'Cottbus', 'Crailsheim',
                     'Creglingen', 'Creußen', 'Creuzburg', 'Crimmitschau', 'Crivitz', 'Cuxhaven', 'Dachau', 'Dahlen', 'Dahme/Mark',
                     'Dahn', 'Damme', 'Dannenberg (Elbe)', 'Dargun', 'Darmstadt', 'Dassel', 'Dassow', 'Datteln', 'Daun', 
                     'Deggendorf', 'Deidesheim', 'Delbrück', 'Delitzsch', 'Delmenhorst', 'Demmin', 'Dessau-Roßlau', 'Detmold', 
                     'Dettelbach', 'Dieburg', 'Diemelstadt', 'Diepholz', 'Dierdorf', 'Dietenheim', 'Dietfurt an der Altmühl', 
                     'Dietzenbach', 'Diez', 'Dillenburg', 'Dillingen an der Donau', 'Dillingen/Saar', 'Dingelstädt', 'Dingolfing',
                     'Dinkelsbühl', 'Dinklage', 'Dinslaken', 'Dippoldiswalde', 'Dissen am Teutoburger Wald', 'Ditzingen', 'Döbeln',
                     'Doberlug-Kirchhain', 'Döbern', 'Dohna', 'Dömitz', 'Dommitzsch', 'Donaueschingen', 'Donauwörth', 'Donzdorf',
                     'Dorfen', 'Dormagen', 'Dornburg-Camburg', 'Dornhan', 'Dornstetten', 'Dorsten', 'Dortmund', 'Dransfeld',
                     'Drebkau', 'Dreieich', 'Drensteinfurt', 'Dresden', 'Drolshagen', 'Duderstadt', 'Duisburg', 'Dülmen',
                     'Düren', 'Düsseldorf', 'Ebeleben', 'Eberbach', 'Ebermannstadt', 'Ebern', 'Ebersbach an der Fils', 
                     'Ebersbach-Neugersdorf', 'Ebersberg', 'Eberswalde', 'Eckartsberga', 'Eckernförde', 'Edenkoben', 'Egeln',
                     'Eggenfelden', 'Eggesin', 'Ehingen (Donau)', 'Ehrenfriedersdorf', 'Eibelstadt', 'Eibenstock', 'Eichstätt', 
                     'Eilenburg', 'Einbeck', 'Eisenach', 'Eisenberg', 'Eisenberg (Pfalz)', 'Eisenhüttenstadt', 'Eisfeld', 'Eisleben',
                     'Eislingen/Fils', 'Ellingen', 'Ellrich', 'Ellwangen (Jagst)', 'Elmshorn', 'Elsdorf', 'Elsfleth', 'Elsterberg', 
                     'Elsterwerda', 'Elstra', 'Elterlein', 'Eltmann', 'Eltville am Rhein', 'Elzach', 'Elze', 'Emden', 'Emmelshausen',
                     'Emmendingen', 'Emmerich am Rhein', 'Emsdetten', 'Endingen am Kaiserstuhl', 'Engen', 'Enger', 'Ennepetal', 
                     'Ennigerloh', 'Eppelheim', 'Eppingen', 'Eppstein', 'Erbach', 'Erbach (Odenwald)', 'Erbendorf', 'Erding', 
                     'Erftstadt', 'Erfurt', 'Erkelenz', 'Erkner', 'Erkrath', 'Erlangen', 'Erlenbach am Main', 'Erlensee', 'Erwitte',
                     'Eschborn', 'Eschenbach in der Oberpfalz', 'Eschershausen', 'Eschwege', 'Eschweiler', 'Esens', 'Espelkamp',
                     'Essen', 'Esslingen am Neckar', 'Ettenheim', 'Ettlingen', 'Euskirchen', 'Eutin', 'Falkenberg/Elster', 
                     'Falkensee', 'Falkenstein/Harz', 'Falkenstein/Vogtl.', 'Fehmarn', 'Fellbach', 'Felsberg', 'Feuchtwangen', 
                     'Filderstadt', 'Finsterwalde', 'Fladungen', 'Flensburg', 'Flöha', 'Flörsheim am Main', 'Florstadt', 'Forchheim',
                     'Forchtenberg', 'Forst (Lausitz)', 'Frankenau', 'Frankenberg (Eder)', 'Frankenberg/Sa.', 'Frankenthal (Pfalz)', 
                     'Frankfurt am Main', 'Frankfurt (Oder)', 'Franzburg', 'Frauenstein', 'Frechen', 'Freiberg am Neckar', 'Freiberg',
                     'Freiburg im Breisgau', 'Freilassing', 'Freinsheim', 'Freising', 'Freital', 'Freren', 'Freudenberg', 
                     'Freudenberg', 'Freudenstadt', 'Freyburg (Unstrut)', 'Freystadt', 'Freyung', 'Fridingen an der Donau', 
                     'Friedberg', 'Friedberg', 'Friedland', 'Friedland', 'Friedrichroda', 'Friedrichsdorf', 'Friedrichshafen',
                     'Friedrichstadt', 'Friedrichsthal', 'Friesack', 'Friesoythe', 'Fritzlar', 'Frohburg', 'Fröndenberg/Ruhr', 
                     'Fulda', 'Fürstenau', 'Fürstenberg/Havel', 'Fürstenfeldbruck', 'Fürstenwalde/Spree', 'Fürth', 'Furth im Wald', 
                     'Furtwangen im Schwarzwald', 'Füssen', 'Gadebusch', 'Gaggenau', 'Gaildorf', 'Gammertingen', 'Garbsen', 
                     'Garching bei München', 'Gardelegen', 'Garding', 'Gartz (Oder)', 'Garz/Rügen', 'Gau-Algesheim', 'Gebesee', 
                   'Gedern', 'Geesthacht', 'Geestland', 'Gefell', 'Gefrees', 'Gehrden', 'Gehren', 'Geilenkirchen', 'Geisa', 
                   'Geiselhöring', 'Geisenfeld', 'Geisenheim', 'Geisingen', 'Geislingen', 'Geislingen an der Steige', 'Geithain', 
                   'Geldern', 'Gelnhausen', 'Gelsenkirchen', 'Gemünden am Main', 'Gemünden (Wohra)', 'Gengenbach', 'Genthin', 
                   'Georgsmarienhütte', 'Gera', 'Gerabronn', 'Gerbstedt', 'Geretsried', 'Geringswalde', 'Gerlingen', 'Germering',
                   'Germersheim', 'Gernsbach', 'Gernsheim', 'Gerolstein', 'Gerolzhofen', 'Gersfeld (Rhön)', 'Gersthofen', 'Gescher',
                   'Geseke', 'Gevelsberg', 'Geyer', 'Giengen an der Brenz', 'Gießen', 'Gifhorn', 'Ginsheim-Gustavsburg', 'Gladbeck',
                   'Gladenbach', 'Glashütte', 'Glauchau', 'Glinde', 'Glücksburg (Ostsee)', 'Glückstadt', 'Gnoien', 'Goch', 'Goldberg',
                   'Goldkronach', 'Golßen', 'Gommern', 'Göppingen', 'Görlitz', 'Goslar', 'Gößnitz', 'Gotha', 'Göttingen', 'Grabow',
                   'Grafenau', 'Gräfenberg', 'Gräfenhainichen', 'Gräfenthal', 'Grafenwöhr', 'Grafing bei München', 'Gransee', 'Grebenau',
                   'Grebenstein', 'Greding', 'Greifswald', 'Greiz', 'Greußen', 'Greven', 'Grevenbroich', 'Grevesmühlen', 'Griesheim', 
                   'Grimma', 'Grimmen', 'Gröditz', 'Groitzsch', 'Gronau (Leine)', 'Gronau (Westf.)', 'Gröningen', 'Großalmerode', 
                   'Groß-Bieberau', 'Großbottwar', 'Großbreitenbach', 'Großenehrich', 'Großenhain', 'Groß-Gerau', 'Großräschen', 
                   'Großröhrsdorf', 'Großschirma', 'Groß-Umstadt', 'Grünberg', 'Grünhain-Beierfeld', 'Grünsfeld', 'Grünstadt',
                   'Guben', 'Gudensberg', 'Güglingen', 'Gummersbach', 'Gundelfingen an der Donau', 'Gundelsheim', 'Günzburg', 
                   'Gunzenhausen', 'Güsten', 'Güstrow', 'Gütersloh', 'Gützkow', 'Haan', 'Hachenburg', 'Hadamar', 'Hagen', 'Hagenbach',
                   'Hagenow', 'Haiger', 'Haigerloch', 'Hainichen', 'Haiterbach', 'Halberstadt', 'Haldensleben', 'Halle (Saale)', 
                   'Halle (Westf.)', 'Hallenberg', 'Hallstadt', 'Haltern am See', 'Halver', 'Hamburg', 'Hameln', 'Hamm', 'Hammelburg',
                   'Hamminkeln', 'Hanau', 'Hannover', 'Hann. Münden', 'Harburg (Schwaben)', 'Hardegsen', 'Haren (Ems)', 
                   'Harsewinkel', 'Hartenstein', 'Hartha', 'Harzgerode', 'Haselünne', 'Haslach im Kinzigtal', 'Haßfurt', 
                   'Hattersheim am Main', 'Hattingen', 'Hatzfeld (Eder)', 'Hausach', 'Hauzenberg', 'Havelberg', 'Havelsee',
                   'Hayingen', 'Hechingen', 'Hecklingen', 'Heide', 'Heideck', 'Heidelberg', 'Heidenau', 'Heidenheim an der Brenz', 
                   'Heilbad Heiligenstadt', 'Heilbronn', 'Heiligenhafen', 'Heiligenhaus', 'Heilsbronn', 'Heimbach', 'Heimsheim',
                   'Heinsberg', 'Heitersheim', 'Heldrungen', 'Helmbrechts', 'Helmstedt', 'Hemau', 'Hemer', 'Hemmingen', 'Hemmoor',
                   'Hemsbach', 'Hennef (Sieg)', 'Hennigsdorf', 'Heppenheim (Bergstraße)', 'Herbolzheim', 'Herborn', 'Herbrechtingen', 
                   'Herbstein', 'Herdecke', 'Herdorf', 'Herford', 'Heringen/Helme', 'Heringen (Werra)', 'Hermeskeil', 
                   'Hermsdorf', 'Herne', 'Herrenberg', 'Herrieden', 'Herrnhut', 'Hersbruck', 'Herten', 'Herzberg am Harz',
                   'Herzberg (Elster)', 'Herzogenaurach', 'Herzogenrath', 'Hessisch Lichtenau', 'Hessisch Oldendorf', 
                   'Hettingen', 'Hettstedt', 'Heubach', 'Heusenstamm', 'Hilchenbach', 'Hildburghausen', 'Hilden', 
                   'Hildesheim', 'Hillesheim', 'Hilpoltstein', 'Hirschau', 'Hirschberg', 'Hirschhorn (Neckar)', 'Hitzacker (Elbe)', 
                   'Hochheim am Main', 'Höchstadt an der Aisch', 'Höchstädt an der Donau', 'Hockenheim', 'Hof', 'Hofgeismar', 
                   'Hofheim am Taunus', 'Hofheim in Unterfranken', 'Hohenberg an der Eger', 'Hohenleuben', 'Hohenmölsen', 
                   'Hohen Neuendorf', 'Hohenstein-Ernstthal', 'Hohnstein', 'Höhr-Grenzhausen', 'Hollfeld', 'Holzgerlingen', 
                   'Holzminden', 'Homberg (Efze)', 'Homberg (Ohm)', 'Homburg', 'Horb am Neckar', 'Hornbach', 'Horn-Bad Meinberg', 
                   'Hornberg', 'Hörstel', 'Horstmar', 'Höxter', 'Hoya', 'Hoyerswerda', 'Hückelhoven', 'Hückeswagen', 'Hüfingen',
                   'Hünfeld', 'Hungen', 'Hürth', 'Husum', 'Ibbenbüren', 'Ichenhausen', 'Idar-Oberstein', 'Idstein', 'Illertissen',
                   'Ilmenau', 'Ilsenburg (Harz)', 'Ilshofen', 'Immenhausen', 'Immenstadt im Allgäu', 'Ingelfingen', 'Ingelheim am Rhein',
                   'Ingolstadt', 'Iphofen', 'Iserlohn', 'Isny im Allgäu', 'Isselburg', 'Itzehoe', 'Jarmen', 'Jena', 'Jerichow', 
                   'Jessen (Elster)', 'Jever', 'Joachimsthal', 'Johanngeorgenstadt', 'Jöhstadt', 'Jülich', 'Jüterbog', 'Kaarst',
                   'Kahla', 'Kaisersesch', 'Kaiserslautern', 'Kalbe (Milde)', 'Kalkar', 'Kaltenkirchen', 'Kaltennordheim', 
                   'Kamen', 'Kamenz', 'Kamp-Lintfort', 'Kandel', 'Kandern', 'Kappeln', 'Karben', 'Karlsruhe', 'Karlstadt', 
                   'Kassel', 'Kastellaun', 'Katzenelnbogen', 'Kaub', 'Kaufbeuren', 'Kehl', 'Kelbra (Kyffhäuser)', 
                   'Kelheim', 'Kelkheim (Taunus)', 'Kellinghusen', 'Kelsterbach', 'Kemberg', 'Kemnath', 'Kempen', 
                   'Kempten (Allgäu)', 'Kenzingen', 'Kerpen', 'Ketzin/Havel', 'Kevelaer', 'Kiel', 'Kierspe', 'Kindelbrück',
                   'Kirchberg', 'Kirchberg an der Jagst', 'Kirchberg (Hunsrück)', 'Kirchen (Sieg)', 'Kirchenlamitz', 
                   'Kirchhain', 'Kirchheimbolanden', 'Kirchheim unter Teck', 'Kirn', 'Kirtorf', 'Kitzingen', 'Kitzscher',
                   'Kleve', 'Klingenberg am Main', 'Klingenthal', 'Klötze', 'Klütz', 'Knittlingen', 'Koblenz', 'Kohren-Sahlis',
                   'Kolbermoor', 'Kölleda', 'Köln', 'Königsberg in Bayern', 'Königsbrück', 'Königsbrunn', 'Königsee-Rottenbach',
                   'Königslutter am Elm', 'Königstein im Taunus', 'Königstein (Sächsische Schweiz)', 'Königswinter', 
                   'Königs Wusterhausen', 'Könnern', 'Konstanz', 'Konz', 'Korbach', 'Korntal-Münchingen', 'Kornwestheim',
                   'Korschenbroich', 'Köthen (Anhalt)', 'Kraichtal', 'Krakow am See', 'Kranichfeld', 'Krautheim', 
                   'Krefeld', 'Kremmen', 'Krempe', 'Kreuztal', 'Kronach', 'Kronberg im Taunus', 'Kröpelin', 'Kroppenstedt',
                   'Krumbach (Schwaben)', 'Kühlungsborn', 'Kulmbach', 'Külsheim', 'Künzelsau', 'Kupferberg', 'Kuppenheim',
                   'Kusel', 'Kyllburg', 'Kyritz', 'Laage', 'Laatzen', 'Ladenburg', 'Lage', 'Lahnstein', 'Lahr/Schwarzwald',
                   'Laichingen', 'Lambrecht (Pfalz)', 'Lampertheim', 'Landau an der Isar', 'Landau in der Pfalz', 
                   'Landsberg am Lech', 'Landsberg', 'Landshut', 'Landstuhl', 'Langelsheim', 'Langen', 'Langenau', 
                   'Langenburg', 'Langenfeld (Rheinland)', 'Langenhagen', 'Langenselbold', 'Langenzenn', 'Langewiesen', 'Lassan', 
                   'Laubach', 'Laucha an der Unstrut', 'Lauchhammer', 'Lauchheim', 'Lauda-Königshofen', 'Lauenburg/Elbe', 
                   'Lauf an der Pegnitz', 'Laufen', 'Laufenburg (Baden)', 'Lauffen am Neckar', 'Lauingen (Donau)', 'Laupheim', 
                   'Lauscha', 'Lauta', 'Lauter-Bernsbach', 'Lauterbach', 'Lauterecken', 'Lauterstein', 'Lebach', 'Lebus', 
                   'Leer (Ostfriesland)', 'Lehesten', 'Lehrte', 'Leichlingen (Rheinland)', 'Leimen', 'Leinefelde-Worbis', 
                   'Leinfelden-Echterdingen', 'Leipheim', 'Leipzig', 'Leisnig', 'Lemgo', 'Lengenfeld', 'Lengerich', 'Lennestadt',
                   'Lenzen', 'Leonberg', 'Leun', 'Leuna', 'Leutenberg', 'Leutershausen', 'Leutkirch im Allgäu', 'Leverkusen', 
                   'Lich', 'Lichtenau', 'Lichtenau', 'Lichtenberg', 'Lichtenfels', 'Lichtenfels', 'Lichtenstein/Sa.', 'Liebenau', 
                   'Liebenwalde', 'Lieberose', 'Liebstadt', 'Limbach-Oberfrohna', 'Limburg an der Lahn', 'Lindau (Bodensee)', 
                   'Linden', 'Lindenberg im Allgäu', 'Lindenfels', 'Lindow (Mark)', 'Lingen (Ems)', 'Linnich', 'Linz am Rhein',
                   'Lippstadt', 'Löbau', 'Löffingen', 'Lohmar', 'Lohne (Oldenburg)', 'Löhne', 'Lohr am Main', 'Loitz', 'Lollar', 
                   'Lommatzsch', 'Löningen', 'Lorch', 'Lorch', 'Lörrach', 'Lorsch', 'Lößnitz', 'Löwenstein', 'Lotte', 'Lübbecke', 
                   'Lübben (Spreewald)', 'Lübbenau/Spreewald', 'Lübeck', 'Lübtheen', 'Lübz', 'Lüchow (Wendland)', 'Lucka', 
                   'Luckau', 'Luckenwalde', 'Lüdenscheid', 'Lüdinghausen', 'Ludwigsburg', 'Ludwigsfelde', 'Ludwigshafen am Rhein', 
                   'Ludwigslust', 'Ludwigsstadt', 'Lugau', 'Lügde', 'Lüneburg', 'Lünen', 'Lunzenau', 'Lütjenburg', 'Lützen', 
                   'Lychen', 'Magdala', 'Magdeburg', 'Mahlberg', 'Mainbernheim', 'Mainburg', 'Maintal', 'Mainz', 'Malchin', 
                   'Malchow', 'Mannheim', 'Manderscheid', 'Mansfeld', 'Marbach am Neckar', 'Marburg', 'Marienberg', 'Marienmünster',
                   'Markdorf', 'Markgröningen', 'Märkisch Buchholz', 'Markkleeberg', 'Markneukirchen', 'Markranstädt', 
                   'Marktbreit', 'Marktheidenfeld', 'Marktleuthen', 'Marktoberdorf', 'Marktredwitz', 'Marktsteft', 'Marl', 
                   'Marlow', 'Marne', 'Marsberg', 'Maulbronn', 'Maxhütte-Haidhof', 'Mayen', 'Mechernich', 'Meckenheim', 'Medebach', 
                   'Meerane', 'Meerbusch', 'Meersburg', 'Meinerzhagen', 'Meiningen', 'Meisenheim', 'Meißen', 'Meldorf', 'Melle',
                   'Mellrichstadt', 'Melsungen', 'Memmingen', 'Menden (Sauerland)', 'Mendig', 'Mengen', 'Meppen', 'Merkendorf',
                   'Merseburg', 'Merzig', 'Meschede', 'Meßkirch', 'Meßstetten', 'Mettmann', 'Metzingen', 'Meuselwitz', 'Meyenburg', 
                   'Michelstadt', 'Miesbach', 'Miltenberg', 'Mindelheim', 'Minden', 'Mirow', 'Mittenwalde', 'Mitterteich', 'Mittweida',
                   'Möckern', 'Möckmühl', 'Moers', 'Mölln', 'Mönchengladbach', 'Monheim am Rhein', 'Monheim', 'Monschau', 'Montabaur', 
                   'Moosburg an der Isar', 'Mörfelden-Walldorf', 'Moringen', 'Mosbach', 'Mössingen', 'Mücheln (Geiseltal)', 'Mügeln',
                   'Mühlacker', 'Mühlberg/Elbe', 'Mühldorf am Inn', 'Mühlhausen/Thüringen', 'Mühlheim am Main', 'Mühlheim an der Donau',
                   'Mülheim an der Ruhr', 'Mülheim-Kärlich', 'Müllheim', 'Müllrose', 'Münchberg', 'Müncheberg', 'München', 
                   'Münchenbernsdorf', 'Munderkingen', 'Münnerstadt', 'Münsingen', 'Munster', 'Münster', 'Münstermaifeld', 
                   'Münzenberg', 'Murrhardt', 'Nabburg', 'Nagold', 'Naila', 'Nassau', 'Nastätten', 'Nauen', 'Naumburg', 
                   'Naumburg (Saale)', 'Naunhof', 'Nebra (Unstrut)', 'Neckarbischofsheim', 'Neckargemünd', 'Neckarsteinach', 
                   'Neckarsulm', 'Neresheim', 'Netphen', 'Nettetal', 'Netzschkau', 'Neu-Anspach', 'Neubrandenburg', 'Neubukow', 
                   'Neubulach', 'Neuburg an der Donau', 'Neudenau', 'Neuenbürg', 'Neuenburg am Rhein', 'Neuenhaus', 'Neuenrade',
                   'Neuenstadt am Kocher', 'Neuenstein', 'Neuerburg', 'Neuffen', 'Neuhaus am Rennweg', 'Neu-Isenburg', 'Neukalen',
                   'Neukirchen', 'Neukirchen-Vluyn', 'Neukloster', 'Neumark', 'Neumarkt in der Oberpfalz', 'Neumarkt-Sankt Veit', 
                   'Neumünster', 'Neunburg vorm Wald', 'Neunkirchen', 'Neuötting', 'Neuruppin', 'Neusalza-Spremberg', 'Neusäß', 
                   'Neuss', 'Neustadt an der Aisch', 'Neustadt an der Donau', 'Neustadt an der Waldnaab', 'Neustadt am Kulm', 
                   'Neustadt am Rübenberge', 'Neustadt an der Orla', 'Neustadt an der Weinstraße', 'Neustadt bei Coburg', 
                   'Neustadt (Dosse)', 'Neustadt-Glewe', 'Neustadt', 'Neustadt in Holstein', 'Neustadt in Sachsen', 'Neustrelitz',
                   'Neutraubling', 'Neu-Ulm', 'Neuwied', 'Nidda', 'Niddatal', 'Nidderau', 'Nideggen', 'Niebüll', 'Niedenstein', 
                   'Niederkassel', 'Niedernhall', 'Nieder-Olm', 'Niederstetten', 'Niederstotzingen', 'Nieheim', 'Niemegk', 
                   'Nienburg (Saale)', 'Nienburg/Weser', 'Nierstein', 'Niesky', 'Nittenau', 'Norden', 'Nordenham', 'Norderney',
                   'Norderstedt', 'Nordhausen', 'Nordhorn', 'Nördlingen', 'Northeim', 'Nortorf', 'Nossen', 'Nürnberg', 
                   'Nürtingen', 'Oberasbach', 'Oberharz am Brocken', 'Oberhausen', 'Oberhof', 'Oberkirch', 'Oberkochen', 
                   'Oberlungwitz', 'Obermoschel', 'Obernburg am Main', 'Oberndorf am Neckar', 'Obernkirchen', 'Ober-Ramstadt',
                   'Oberriexingen', 'Obertshausen', 'Oberursel (Taunus)', 'Oberviechtach', 'Oberweißbach/Thür. Wald', 'Oberwesel',
                   'Oberwiesenthal', 'Ochsenfurt', 'Ochsenhausen', 'Ochtrup', 'Oderberg', 'Oebisfelde-Weferlingen', 'Oederan',
                   'Oelde', 'Oelsnitz/Erzgeb.', 'Oelsnitz/Vogtl.', 'Oer-Erkenschwick', 'Oerlinghausen', 'Oestrich-Winkel', 
                   'Oettingen in Bayern', 'Offenbach am Main', 'Offenburg', 'Ohrdruf', 'Öhringen', 'Olbernhau', 'Olching', 
                   'Oldenburg (Oldb)', 'Oldenburg in Holstein', 'Olfen', 'Olpe', 'Olsberg', 'Oppenau', 'Oppenheim', 
                   'Oranienbaum-Wörlitz', 'Oranienburg', 'Orlamünde', 'Ornbau', 'Ortenberg', 'Ortrand', 'Oschatz', 
                   'Oschersleben (Bode)', 'Osnabrück', 'Osterburg (Altmark)', 'Osterburken', 'Osterfeld', 'Osterhofen', 
                   'Osterholz-Scharmbeck', 'Osterode am Harz', 'Osterwieck', 'Ostfildern', 'Ostheim vor der Rhön', 'Osthofen', 
                   'Östringen', 'Ostritz', 'Otterberg', 'Otterndorf', 'Ottweiler', 'Overath', 'Owen', 'Paderborn', 'Papenburg',
                   'Pappenheim', 'Parchim', 'Parsberg', 'Pasewalk', 'Passau', 'Pattensen', 'Pausa-Mühltroff', 'Pegau', 'Pegnitz', 
                   'Peine', 'Peitz', 'Penig', 'Penkun', 'Penzberg', 'Penzlin', 'Perleberg', 'Petershagen', 'Pfaffenhofen an der Ilm',
                   'Pfarrkirchen', 'Pforzheim', 'Pfreimd', 'Pfullendorf', 'Pfullingen', 'Pfungstadt', 'Philippsburg', 'Pinneberg',
                   'Pirmasens', 'Pirna', 'Plattling', 'Plau am See', 'Plaue', 'Plauen', 'Plettenberg', 'Pleystein', 'Plochingen',
                   'Plön', 'Pockau-Lengefeld', 'Pocking', 'Pohlheim', 'Polch', 'Porta Westfalica', 'Pößneck', 'Potsdam',
                   'Pottenstein', 'Preetz', 'Premnitz', 'Prenzlau', 'Pressath', 'Preußisch Oldendorf', 'Prichsenstadt', 
                   'Pritzwalk', 'Prüm', 'Puchheim', 'Pulheim', 'Pulsnitz', 'Putbus', 'Putlitz', 'Püttlingen', 'Quakenbrück', 
                   'Quedlinburg', 'Querfurt', 'Quickborn', 'Rabenau', 'Radeberg', 'Radebeul', 'Radeburg', 'Radevormwald', 
                   'Radolfzell am Bodensee', 'Raguhn-Jeßnitz', 'Rahden', 'Rain', 'Ramstein-Miesenbach', 'Ranis', 
                   'Ransbach-Baumbach', 'Rastatt', 'Rastenberg', 'Rathenow', 'Ratingen', 'Ratzeburg', 'Rauenberg', 
                   'Raunheim', 'Rauschenberg', 'Ravensburg', 'Ravenstein', 'Recklinghausen', 'Rees', 'Regen', 
                   'Regensburg', 'Regis-Breitingen', 'Rehau', 'Rehburg-Loccum', 'Rehna', 'Reichelsheim (Wetterau)', 
                   'Reichenbach im Vogtland', 'Reichenbach/O.L.', 'Reinbek', 'Reinfeld (Holstein)', 'Reinheim', 'Remagen',
                   'Remda-Teichel', 'Remscheid', 'Remseck am Neckar', 'Renchen', 'Rendsburg', 'Rennerod', 'Renningen',
                   'Rerik', 'Rethem (Aller)', 'Reutlingen', 'Rheda-Wiedenbrück', 'Rhede', 'Rheinau', 'Rheinbach', 
                   'Rheinberg', 'Rheinböllen', 'Rheine', 'Rheinfelden (Baden)', 'Rheinsberg', 'Rheinstetten', 'Rhens',
                   'Rhinow', 'Ribnitz-Damgarten', 'Richtenberg', 'Riedenburg', 'Riedlingen', 'Riedstadt', 'Rieneck', 
                   'Riesa', 'Rietberg', 'Rinteln', 'Röbel/Müritz', 'Rochlitz', 'Rockenhausen', 'Rodalben', 'Rodenberg',
                   'Rödental', 'Rödermark', 'Rodewisch', 'Rodgau', 'Roding', 'Römhild', 'Romrod', 'Ronneburg', 
                   'Ronnenberg', 'Rosbach vor der Höhe', 'Rosenfeld', 'Rosenheim', 'Rosenthal', 'Rösrath', 'Roßleben',
                   'Roßwein', 'Rostock', 'Rotenburg an der Fulda', 'Rotenburg (Wümme)', 'Roth', 'Rötha',
                   'Röthenbach an der Pegnitz', 'Rothenburg/O.L.', 'Rothenburg ob der Tauber', 'Rothenfels',
                   'Rottenburg am Neckar', 'Rottenburg a.d.Laaber', 'Röttingen', 'Rottweil', 'Rötz', 'Rüdesheim am Rhein',
                   'Rudolstadt', 'Ruhla', 'Ruhland', 'Runkel', 'Rüsselsheim am Main', 'Rutesheim', 'Rüthen', 
                   'Saalburg-Ebersdorf', 'Saalfeld/Saale', 'Saarbrücken', 'Saarburg', 'Saarlouis', 'Sachsenhagen',
                   'Sachsenheim', 'Salzgitter', 'Salzkotten', 'Salzwedel', 'Sandau (Elbe)', 'Sandersdorf-Brehna', 
                   'Sangerhausen', 'Sankt Augustin', 'Sankt Goar', 'Sankt Goarshausen', 'Sarstedt', 'Sassenberg',
                   'Sassnitz', 'Sayda', 'Schalkau', 'Schauenstein', 'Scheer', 'Scheibenberg', 'Scheinfeld', 
                   'Schelklingen', 'Schenefeld', 'Scheßlitz', 'Schieder-Schwalenberg', 'Schifferstadt', 'Schillingsfürst',
                   'Schiltach', 'Schirgiswalde-Kirschau', 'Schkeuditz', 'Schkölen', 'Schleiden', 'Schleiz', 
                   'Schleswig', 'Schlettau', 'Schleusingen', 'Schlieben', 'Schlitz', 'Schloß Holte-Stukenbrock', 
                   'Schlotheim', 'Schlüchtern', 'Schlüsselfeld', 'Schmalkalden', 'Schmallenberg', 'Schmölln', 
                   'Schnackenburg', 'Schnaittenbach', 'Schneeberg', 'Schneverdingen', 'Schömberg', 'Schönau', 
                   'Schönau im Schwarzwald', 'Schönberg', 'Schönebeck (Elbe)', 'Schöneck/Vogtl.', 'Schönewalde', 
                   'Schongau', 'Schöningen', 'Schönsee', 'Schönwald', 'Schopfheim', 'Schöppenstedt', 'Schorndorf', 
                   'Schortens', 'Schotten', 'Schramberg', 'Schraplau', 'Schriesheim', 'Schrobenhausen', 'Schrozberg', 
                   'Schüttorf', 'Schwaan', 'Schwabach', 'Schwäbisch Gmünd', 'Schwäbisch Hall', 'Schwabmünchen', 
                   'Schwaigern', 'Schwalbach am Taunus', 'Schwalmstadt', 'Schwandorf', 'Schwanebeck', 'Schwarzenbach am Wald',
                   'Schwarzenbach an der Saale', 'Schwarzenbek', 'Schwarzenberg/Erzgeb.', 'Schwarzenborn', 
                   'Schwarzheide', 'Schwedt/Oder', 'Schweich', 'Schweinfurt', 'Schwelm', 'Schwentinental', 'Schwerin', 
                   'Schwerte', 'Schwetzingen', 'Sebnitz', 'Seehausen (Altmark)', 'Seeland', 'Seelow', 'Seelze', 'Seesen',
                   'Sehnde', 'Seifhennersdorf', 'Selb', 'Selbitz', 'Seligenstadt', 'Selm', 'Selters (Westerwald)', 'Senden',
                   'Sendenhorst', 'Senftenberg', 'Seßlach', 'Siegburg', 'Siegen', 'Sigmaringen', 'Simbach am Inn',
                   'Simmern/Hunsrück', 'Sindelfingen', 'Singen (Hohentwiel)', 'Sinsheim', 'Sinzig', 'Soest', 'Solingen', 
                   'Solms', 'Soltau', 'Sömmerda', 'Sondershausen', 'Sonneberg', 'Sonnewalde', 'Sonthofen', 'Sontra', 
                   'Spaichingen', 'Spalt', 'Spangenberg', 'Speicher (Eifel)', 'Spenge', 'Speyer', 'Spremberg', 'Springe',
                   'Sprockhövel', 'Stade', 'Stadtallendorf', 'Stadtbergen', 'Stadthagen', 'Stadtilm', 'Stadtlengsfeld',
                   'Stadtlohn', 'Stadtoldendorf', 'Stadtprozelten', 'Stadtroda', 'Stadtsteinach', 'Stadt Wehlen', 
                   'Starnberg', 'Staßfurt', 'Staufen im Breisgau', 'Staufenberg', 'Stavenhagen', 'St. Blasien', 'Stein', 
                   'Steinach', 'Steinau an der Straße', 'Steinbach-Hallenberg', 'Steinbach (Taunus)', 'Steinfurt', 
                   'Steinheim an der Murr', 'Steinheim', 'Stendal', 'Sternberg', 'St. Ingbert', 'St. Georgen im Schwarzwald',
                   'Stockach', 'Stolberg (Rheinland)', 'Stollberg/Erzgeb.', 'Stolpen', 'Storkow (Mark)', 'Stößen',
                   'Straelen', 'Stralsund', 'Strasburg (Uckermark)', 'Straubing', 'Strausberg', 'Strehla', 'Stromberg',
                   'Stühlingen', 'Stutensee', 'Stuttgart', 'St. Wendel', 'Suhl', 'Sulingen', 'Sulz am Neckar', 'Sulzbach/Saar',
                   'Sulzbach-Rosenberg', 'Sulzburg', 'Sundern (Sauerland)', 'Südliches Anhalt', 'Süßen', 'Syke', 
                   'Tambach-Dietharz', 'Tangerhütte', 'Tangermünde', 'Tann (Rhön)', 'Tanna', 'Tauberbischofsheim', 'Taucha',
                   'Taunusstein', 'Tecklenburg', 'Tegernsee', 'Telgte', 'Teltow', 'Templin', 'Tengen', 'Tessin', 
                   'Teterow', 'Tettnang', 'Teublitz', 'Teuchern', 'Teupitz', 'Teuschnitz', 'Thale', 'Thalheim/Erzgeb.', 
                   'Thannhausen', 'Tharandt', 'Themar', 'Thum', 'Tirschenreuth', 'Titisee-Neustadt', 'Tittmoning', 
                   'Todtnau', 'Töging am Inn', 'Tönisvorst', 'Tönning', 'Torgau', 'Torgelow', 'Tornesch', 'Traben-Trarbach',
                   'Traunreut', 'Traunstein', 'Trebbin', 'Trebsen/Mulde', 'Treffurt', 'Trendelburg', 'Treuchtlingen', 
                   'Treuen', 'Treuenbrietzen', 'Triberg im Schwarzwald', 'Tribsees', 'Trier', 'Triptis', 'Trochtelfingen',
                   'Troisdorf', 'Trossingen', 'Trostberg', 'Tübingen', 'Tuttlingen', 'Twistringen', 'Übach-Palenberg',
                   'Überlingen', 'Uebigau-Wahrenbrück', 'Ueckermünde', 'Uelzen', 'Uetersen', 'Uffenheim', 'Uhingen', 
                   'Ulm', 'Ulmen', 'Ulrichstein', 'Ummerstadt', 'Unkel', 'Unna', 'Unterschleißheim', 'Usedom', 
                   'Usingen', 'Uslar', 'Vacha', 'Vaihingen an der Enz', 'Vallendar', 'Varel', 'Vechta', 'Velbert', 
                   'Velburg', 'Velden', 'Vellberg', 'Velen', 'Vellmar', 'Velten', 'Verden (Aller)', 'Veringenstadt',
                   'Verl', 'Versmold', 'Vetschau/Spreewald', 'Viechtach', 'Viernheim', 'Viersen', 'Villingen-Schwenningen',
                   'Vilsbiburg', 'Vilseck', 'Vilshofen an der Donau', 'Visselhövede', 'Vlotho', 
                   'Voerde (Niederrhein)', 'Vogtsburg im Kaiserstuhl', 'Vohburg an der Donau', 'Vohenstrauß',
                   'Vöhrenbach', 'Vöhringen', 'Volkach', 'Völklingen', 'Volkmarsen', 'Vreden',
                   'Wachenheim an der Weinstraße', 'Wächtersbach', 'Wadern', 'Waghäusel', 'Wahlstedt', 'Waiblingen',
                   'Waibstadt', 'Waischenfeld', 'Waldbröl', 'Waldeck', 'Waldenbuch', 'Waldenburg', 'Waldenburg', 
                   'Waldershof', 'Waldheim', 'Waldkappel', 'Waldkirch', 'Waldkirchen', 'Waldkraiburg', 'Waldmünchen', 
                   'Waldsassen', 'Waldshut-Tiengen', 'Walldorf', 'Walldürn', 'Wallenfels', 'Walsrode', 
                   'Waltershausen', 'Waltrop', 'Wanfried', 'Wangen im Allgäu', 'Wanzleben-Börde', 'Warburg',
                   'Waren (Müritz)', 'Warendorf', 'Warin', 'Warstein', 'Wassenberg', 'Wasserburg am Inn', 
                   'Wassertrüdingen', 'Wasungen', 'Wedel', 'Weener', 'Wegberg', 'Wegeleben', 'Wehr', 'Weida', 
                   'Weiden in der Oberpfalz', 'Weikersheim', 'Weil am Rhein', 'Weilburg', 'Weil der Stadt', 
                   'Weilheim an der Teck', 'Weilheim in Oberbayern', 'Weimar', 'Weingarten', 'Weinheim', 'Weinsberg', 
                   'Weinstadt', 'Weismain', 'Weißenberg', 'Weißenburg in Bayern', 'Weißenfels', 'Weißenhorn', 
                   'Weißensee', 'Weißenstadt', 'Weißenthurm', 'Weißwasser/O.L.', 'Weiterstadt', 'Welzheim', 'Welzow',
                   'Wemding', 'Wendlingen am Neckar', 'Werben (Elbe)', 'Werdau', 'Werder (Havel)', 'Werdohl', 
                   'Werl', 'Werlte', 'Wermelskirchen', 'Wernau (Neckar)', 'Werne', 'Werneuchen', 'Wernigerode',
                   'Wertheim', 'Werther (Westf.)', 'Wertingen', 'Wesel', 'Wesenberg', 'Wesselburen', 'Wesseling', 
                   'Westerburg', 'Westerstede', 'Wetter (Ruhr)', 'Wetter', 'Wettin-Löbejün', 'Wetzlar', 'Widdern', 
                   'Wiehe', 'Wiehl', 'Wiesbaden', 'Wiesmoor', 'Wiesensteig', 'Wiesloch', 'Wildau', 'Wildberg',
                   'Wildenfels', 'Wildeshausen', 'Wilhelmshaven', 'Wilkau-Haßlau', 'Willebadessen', 'Willich', 
                   'Wilsdruff', 'Wilster', 'Wilthen', 'Windischeschenbach', 'Windsbach', 'Winnenden', 
                   'Winsen (Luhe)', 'Winterberg', 'Wipperfürth', 'Wirges', 'Wismar', 'Wissen', 'Witten', 
                   'Wittenberg', 'Wittenberge', 'Wittenburg', 'Wittichenau', 'Wittlich', 'Wittingen', 
                   'Wittmund', 'Wittstock/Dosse', 'Witzenhausen', 'Woldegk', 'Wolfach', 'Wolfenbüttel', 
                   'Wolfhagen', 'Wolframs-Eschenbach', 'Wolfratshausen', 'Wolfsburg', 'Wolfstein', 'Wolgast', 
                   'Wolkenstein', 'Wolmirstedt', 'Worms', 'Wörrstadt', 'Wörth am Rhein', 'Wörth an der Donau', 
                   'Wörth am Main', 'Wriezen', 'Wülfrath', 'Wunsiedel', 'Wunstorf', 'Wuppertal', 'Würselen', 
                   'Wurzbach', 'Würzburg', 'Wurzen', 'Wustrow (Wendland)', 'Wyk auf Föhr', 'Xanten', 
                   'Zahna-Elster', 'Zarrentin am Schaalsee', 'Zehdenick', 'Zeil am Main', 'Zeitz', 
                   'Zell am Harmersbach', 'Zell im Wiesental', 'Zell (Mosel)', 'Zella-Mehlis', 'Zerbst/Anhalt', 
                   'Zeulenroda-Triebes', 'Zeven', 'Ziegenrück', 'Zierenberg', 'Ziesar', 'Zirndorf', 'Zittau', 
                   'Zörbig', 'Zossen', 'Zschopau', 'Zülpich', 'Zweibrücken', 'Zwenkau', 'Zwickau', 'Zwiesel', 
                   'Zwingenberg', 'Zwönitz', 
                   "Schleswig-Holstein", "Mecklenburg-Vorpommern", "Hamburg", "Bremen", "Niedersachsen", "NRW", 
                   "Nordrhein-Westfalen", "Hessen", "Rheinland-Pfalz", "Saarland", "Baden-Würrtemberg", "Bayern",
                   "Sachsen", "Brandenburg", "Mitteldeutschland", "Neuenburg", "Freiburg", "Deutschland"]

In [88]:
# Handlungsort einrichten


def check_city(df):
    df["Ort"] = ""
    df["Handlungsorte"] = df["Handlungsorte"].fillna("x")
    
    for i in df.index:
        sentence = df.at[i, "Handlungsorte"]
       
        if any(ext in sentence for ext in orte_weserkurier):
            df.at[i, "Ort"] = "Region"
        elif any(ext in sentence for ext in orte_deutschland):
            df.at[i, "Ort"] = "Deutschland"
        elif sentence == "x": 
            df.at[i, "Ort"] = "keine Angaben"
        elif sentence == "": 
            df.at[i, "Ort"] = "keine Angaben"
        else: 
            df.at[i, "Ort"] = "International"
    return df

df_ort = check_city(df)

In [91]:
df_ort[["SplitId", "Ort"]].to_csv("./data/handlungsorte.csv", index=False)

## Ortsmarken herausfiltern und mit Listen abgleichen

In [50]:

# Deutschlandliste weiter oben, orte_deutschland
#orte_deutschland

In [69]:
#SQL-Abfrage
import pyodbc
import pymssql
server = 'tcp:lw-azuresql-1.database.windows.net'
database = 'LesewertMandantenDB_WKM' # Süddeutsche Zeitung
username = 'Mehrwertmacher'
password = 'fW18ObHgb8qVhlQUsNpx'
driver= '{ODBC Driver 17 for SQL Server}'

cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

query = '''
SELECT ti.TeilartikelId, 
CASE CHARINDEX(' ', ti.Artikeltext, 1)
     WHEN 0 THEN ti.Artikeltext
     ELSE SUBSTRING(ti.Artikeltext, 1, CHARINDEX(' ', ti.Artikeltext, 1)-1)
END as Ortsmarke, 
ti.Ueberschrifttext, 
kum.Handlungsorte
FROM 
Teilartikelinhalte ti
JOIN vw_Artikellesewerte_Kumuliert_AktiveWellen_Vorberechnet kum 
ON ti.TeilartikelId = kum.SplitId

'''

query_gesamter_Artikeltext = '''
SELECT ti.TeilartikelId, 
ti.Artikeltext as Artikeltext, 
ti.Ueberschrifttext, 
kum.Handlungsorte
FROM 
Teilartikelinhalte ti
RIGHT JOIN vw_Artikellesewerte_Kumuliert_AktiveWellen_Vorberechnet kum 
ON ti.TeilartikelId = kum.SplitId


'''
df_query = pd.read_sql(query_gesamter_Artikeltext, cnxn)
cursor.close()
df_query = df_query.drop_duplicates(subset=["TeilartikelId"])
df_query.to_pickle("./data/artikeltext_raw.pkl")


In [8]:
dfom = pd.read_pickle("./data/artikeltext_raw.pkl")

In [9]:
dfom

,TeilartikelId,Artikeltext,Ueberschrifttext,Handlungsorte
0,14389,hat ein junger Fußballfan den argentinischen F...,Übrigens …,None
3,14390,Berlin. Angesichts der Kostensprünge beim Heiz...,Lösung für Haushalte gesucht,"Deutschland, Berlin"
6,14391,Berlin. Im Zusammenhang mit der Bundestagswahl...,Plakate von AfD und Grünen am häufigsten zerstört,Berlin
9,14392,Neuenhagen. Der Brandenburger Strom- und Erdg...,Strom- und Gasversorger Otima insolvent,None
12,14393,"Manchmal wirkt es so, als seien sie moderne Do...",Akupunktur für die Kirche,"Deutschland, Corona"
...,...,...,...,...
25256,26175,Im Zuge der 2G-Regel müssen voraussichtlich ab...,\n 2G bald auch für Jugen...,None
25259,26176,Hanover. Der Preisdruck ist in vielen Bereiche...,Große Tierbestände wegen niedriger Preise,None
25262,26177,Braunlage. Ungewöhnlich früh können Winterspor...,Der Harz startet in die Skisaison,None
25265,26178,Brake. Nach dem Fund einer getöteten 26-Jährig...,26-Jährige tot in Wohnung gefunden,None


In [26]:
# nur die Artikel mit SplitId aus nichtkum-Datei (um NB-Artikel herauszufiltern)
df_merge_ort = pd.merge(dfom, dfnk[["SplitId", "ArtikelId", "LW", "BW", "DW"]], how="left", left_on="TeilartikelId", right_on="SplitId")
df_om = df_merge_ort[~df_merge_ort["SplitId"].isnull()]
df_om.drop_duplicates(subset=["SplitId"], inplace=True)

c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [27]:
df_om

,TeilartikelId,Artikeltext,Ueberschrifttext,Handlungsorte,SplitId,ArtikelId,LW,BW,DW
496,14993,Bremen/Verden. Rund 660-mal haben Diebe im Jah...,Jeden Tag verschwinden Kfz-Kennzeichen,Bremen,14993.0,33626.0,57.140411,68.518519,83.394114
499,14994,Ein Nachmittag Anfang Oktober: Mit der einen H...,Vier Wochen zum Einleben,"Hamburg, Berlin, Cuxhaven",14994.0,33627.0,20.180339,33.333333,60.541016
502,14995,Bremen. 16 Jahre mussten sich die Fans gedulde...,Zwischen zwei Welten,Bremen,14995.0,33628.0,6.075234,16.666667,36.451404
505,14996,Peggy Schierenbeck sitzt mit Beginn der neuen ...,Peggy Schierenbeck (SPD),None,14996.0,33629.0,22.411373,29.629630,75.638383
508,14997,"Einige haben politische Erfahrung, andere – wi...",Neulinge im Bundestag,None,14997.0,33630.0,4.063287,5.555556,73.139158
...,...,...,...,...,...,...,...,...,...
23437,26175,Im Zuge der 2G-Regel müssen voraussichtlich ab...,\n 2G bald auch für Jugen...,None,26175.0,59181.0,10.949033,13.207547,82.899818
23440,26176,Hanover. Der Preisdruck ist in vielen Bereiche...,Große Tierbestände wegen niedriger Preise,None,26176.0,59182.0,14.598807,16.981132,85.970755
23443,26177,Braunlage. Ungewöhnlich früh können Winterspor...,Der Harz startet in die Skisaison,None,26177.0,59180.0,26.023596,35.849057,72.592135
23446,26178,Brake. Nach dem Fund einer getöteten 26-Jährig...,26-Jährige tot in Wohnung gefunden,None,26178.0,59183.0,41.589286,43.396226,95.836181


In [28]:
def identify_ortsmarke(df): 
    print(df.shape)
    
    for i in df.index:
        
        text = df.at[i, "Artikeltext"]
        text_split = text.split(".")[0]
       
        # split up Teil vor dem Punkt
        # wenn der kleiner/gleich 2 ist, ist es vermutlich eine Ortsmarke
        text_split = text_split.split(" ")
        
        
        # jetzt müssen ein paar Sonderformen der Ortsmarken herausgefiltert werden
        
        if (len(text_split) ==4) & (text_split[0]=="Landkreise"):
            ortsm = text_split[0] + " " + text_split[1] + " " + text_split[2] + " " + text_split[3]
        elif (len(text_split)==3):
            if ((text_split[1]=="am") | (text_split[0]=="Rio") \
                                  | (text_split[2]=="York") | (text_split[0]=="Palma") |\
                                  (text_split[0]=="Los") | (text_split[0]=="Santiago")): 
                ortsm = text_split[0] + " " + text_split[1] + " " + text_split[2]
        elif (len(text_split)==2):
            # wenn zweiter eintrag eine Zahl ist, wurde z.B. nach "Zum 60." getrennt -> keine Ortsmarke
            if text_split[1].isdecimal(): 
                print("decimal gefunden")
                ortsm = ""
            else: 
                ortsm = text_split[0] + " " + text_split[1]
        elif(len(text_split)==1):
            
            ortsm = text_split[0]
            
        else:
           
            ortsm = ""
        
        df.at[i, "Ortsmarke"] = ortsm

    return df
        
        
        
        
        
df_ortsm = identify_ortsmarke(df_om)

(9895, 9)
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden
decimal gefunden


c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [29]:
df_ortsm

,TeilartikelId,Artikeltext,Ueberschrifttext,Handlungsorte,SplitId,ArtikelId,LW,BW,DW,Ortsmarke
496,14993,Bremen/Verden. Rund 660-mal haben Diebe im Jah...,Jeden Tag verschwinden Kfz-Kennzeichen,Bremen,14993.0,33626.0,57.140411,68.518519,83.394114,Bremen/Verden
499,14994,Ein Nachmittag Anfang Oktober: Mit der einen H...,Vier Wochen zum Einleben,"Hamburg, Berlin, Cuxhaven",14994.0,33627.0,20.180339,33.333333,60.541016,
502,14995,Bremen. 16 Jahre mussten sich die Fans gedulde...,Zwischen zwei Welten,Bremen,14995.0,33628.0,6.075234,16.666667,36.451404,Bremen
505,14996,Peggy Schierenbeck sitzt mit Beginn der neuen ...,Peggy Schierenbeck (SPD),None,14996.0,33629.0,22.411373,29.629630,75.638383,
508,14997,"Einige haben politische Erfahrung, andere – wi...",Neulinge im Bundestag,None,14997.0,33630.0,4.063287,5.555556,73.139158,
...,...,...,...,...,...,...,...,...,...,...
23437,26175,Im Zuge der 2G-Regel müssen voraussichtlich ab...,\n 2G bald auch für Jugen...,None,26175.0,59181.0,10.949033,13.207547,82.899818,
23440,26176,Hanover. Der Preisdruck ist in vielen Bereiche...,Große Tierbestände wegen niedriger Preise,None,26176.0,59182.0,14.598807,16.981132,85.970755,Hanover
23443,26177,Braunlage. Ungewöhnlich früh können Winterspor...,Der Harz startet in die Skisaison,None,26177.0,59180.0,26.023596,35.849057,72.592135,Braunlage
23446,26178,Brake. Nach dem Fund einer getöteten 26-Jährig...,26-Jährige tot in Wohnung gefunden,None,26178.0,59183.0,41.589286,43.396226,95.836181,Brake


In [36]:
tt = df_ortsm[df_ortsm["Ortsmarke"]=="Brüssel"]
tt.LW.mean()

17.85331413548974

#### 

In [130]:
t = df_ortsm[df_ortsm["Ortsmarke"].str.contains("Landkreis")]
t

,TeilartikelId,Artikeltext,Ueberschrifttext,Handlungsorte,SplitId,ArtikelId,Ortsmarke
887,15168,Landkreis Osterholz. Das Thema Wasserstoff sol...,Kommunen werben für Wasserstoff,Lüneburg,15168.0,33926.0,Landkreis Osterholz
888,15169,Landkreis Osterholz. Allradfahrzeuge sind auf ...,Vormarsch der Allradfahrzeuge,None,15169.0,33928.0,Landkreis Osterholz
908,15191,Landkreis Verden. Seit einer Woche ist nun kos...,Erste Eindrücke,None,15191.0,34075.0,Landkreis Verden
915,15198,Landkreis Verden. Der Kreisfrauenrat hat mit d...,Kreisfrauenrat: Mehr Geld für Frauenhäuser,Liège,15198.0,34081.0,Landkreis Verden
917,15200,Landkreis Verden. Seit einer Woche ist nun kos...,Im Kreis sinkt die Nachfrage,None,15200.0,34092.0,Landkreis Verden
...,...,...,...,...,...,...,...
23008,25975,Landkreis Osterholz. Es ist noch nicht so lang...,Glasfaseranschluss rettet Breitbandgipfel,"Deutschland, Bremen, Berlin",25975.0,58750.0,Landkreis Osterholz
23034,26002,Landkreis Verden. In den nächsten Tagen wird d...,Einheitlicher Wochenabfuhrtag ab 2022,None,26002.0,58922.0,Landkreis Verden
23037,26005,Landkreis Rotenburg. Um die Klimaschutzziele z...,Solar-Investoren klopfen an,None,26005.0,58924.0,Landkreis Rotenburg
23040,26008,Landkreis Verden. Ein 90-jähriger Mann aus dem...,Corona: 82. Todesfall im Landkreis Verden,None,26008.0,58927.0,Landkreis Verden


In [66]:
def filter_ortsmarken(df): 
    ''' 
    Hier verpassen wir dem Dataframe ein 
    '''
    df["OM_vorhanden"] = bool
    df["OM_simple"] = ""
    
    #df["Ortsmarke"] = df["Ortsmarke"].str.replace(".", "")
    
    for i in df.index: 
        ortsm = df.at[i, "Ortsmarke"]
        
        if "Landkreis" in ortsm: 
            print("Achtung, Landkreis")
            print(ortsm)
            print(df.at[i, "Handlungsorte"])
        if "." in ortsm:
            new_ort_split = ortsm.split(".")
            new_ort = new_ort_split[0].replace(".", "")
#             if new_ort == "Landkreis": 
#                 new_ort = new_ort + " " + new_ort_split[1].strip()
            df.at[i, "OM-vorhanden"] = True
            df.at[i, "OM"] = new_ort
            
            
        else: 
            new_ort = ortsm
       
        
    return df
            
dfortsmarken = filter_ortsmarken(df_ortsm)
dfortsmarken

Achtung, Landkreis
Landkreis
Lüneburg
Achtung, Landkreis
Landkreis
None
Achtung, Landkreis
Landkreis
None
Achtung, Landkreis
Landkreis
Liège
Achtung, Landkreis
Sottrum/Landkreis
None
Achtung, Landkreis
Landkreis
None
Achtung, Landkreis
Landkreise
Corona
Achtung, Landkreis
Landkreis
None
Achtung, Landkreis
Landkreis
None
Achtung, Landkreis
Landkreis
None
Achtung, Landkreis
Landkreis
None
Achtung, Landkreis
Landkreis
None
Achtung, Landkreis
Landkreis
None
Achtung, Landkreis
Landkreis
Deutschland, Bremen
Achtung, Landkreis
Landkreis
None
Achtung, Landkreis
Landkreis
Celle
Achtung, Landkreis
Landkreis
None
Achtung, Landkreis
Landkreise
Corona
Achtung, Landkreis
Landkreis
Bremen, Hannover
Achtung, Landkreis
Landkreis
None
Achtung, Landkreis
Landkreis
Bremen, Mannheim
Achtung, Landkreis
Landkreis
Hannover
Achtung, Landkreis
Landkreis
Hamburg
Achtung, Landkreis
Landkreis
None
Achtung, Landkreis
Landkreis
Berlin, Corona
Achtung, Landkreis
Landkreis
None
Achtung, Landkreis
Landkreise
Corona
Ach

<ipython-input-66-614afc0235b6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["OM_vorhanden"] = bool
<ipython-input-66-614afc0235b6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["OM"] = ""


In [64]:
t = df_ortsmarken[df_ortsmarken["OM"].str.contains("Landkreis")]
t

,TeilartikelId,Ortsmarke,Ueberschrifttext,Handlungsorte,SplitId,ArtikelId,OM_vorhanden,OM,OM-vorhanden
5633,17407,Bremen-Nord/Landkreise. Die,Arbeitsmarkt entspannt sich,"Bremen, Liège",17407.0,39110.0,<class 'bool'>,Bremen-Nord/Landkreise,True


In [60]:

df_titel = df[df["Ressort"]=="Titelseite"]
df_test = 

(415, 23)

In [63]:
df_query_ = df_query.drop_duplicates(subset=["TeilartikelId"])

In [70]:
test = pd.merge(df_titel, df_query_, left_on="SplitId", right_on="TeilartikelId", how="left")

In [72]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 415 entries, 0 to 414
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   SplitId             415 non-null    int64  
 1   ArtikelId           415 non-null    int64  
 2   Artikel             415 non-null    object 
 3   ErscheinungsId      415 non-null    int64  
 4   AusgabeId           415 non-null    int64  
 5   Datum               415 non-null    object 
 6   Seite               415 non-null    int64  
 7   BW                  415 non-null    float64
 8   DW                  415 non-null    float64
 9   LW                  415 non-null    float64
 10  Seitentitel         415 non-null    object 
 11  Platzierungen       415 non-null    object 
 12  Darstellungsformen  415 non-null    object 
 13  Beschreibung        56 non-null     object 
 14  Handlungsorte_x     285 non-null    object 
 15  Akteure             255 non-null    object 
 16  Themen  

# Wirtschaft Unternehmen und Vereine & 

In [3]:
liste_unternehmen = ['Accor S.A.', 'Mövenpick', 'RWE', 'BMU', 'Vattenfall','Rewe',
       "Deutsche Bank", "Katag",
       'AOK Nordwest', 
       'Hugo Boss',  'Apple', 'Dr. Wolff-Gruppe',
       'EDEKA', 'Nestlé','Westag & Getalit', 
       'VW', 'Volkswagen', 'ALDI', 'Diebold Nixdorf', 'Wincor Nixdorf', 'Starbucks', 'H&M',
       'Vonovia', 'Seidensticker', 'Phoenix Contact','Facebook', 'Apple',
       'GALERIA Kaufhof', 'Amazon', 'BMW', 'Twitter', 'Allianz SE', 
       'Deutsche Bank', 'Columbus Circle', 'Vodafone', 'Liberty Global',
       'Google Inc.', 'Waymo', 'Deutsche Post AG','Deutsche Bahn', 'Schüco, International',
       'Air Berlin', 'Siemens AG', 'INA Holding Schaeffler', 'ADAC',
       'Bertelsmann','Deutsche Post','KFC', 'Audi', 'Siemens',
       'EDEKA, Mark Rosenkranz', 'Paragon',
       'Vereinte Dienstleistungsgewerkschaft',
       'Claas KGaA', 'Symrise',
       'Adidas', 'Delivery Hero', 'TUI',
       'Wal-Mart Stores Inc.', 'Unitymedia', 'Vodafone',
       'Deutsche Telekom', 'Unitymedia, Vodafone', 'Liberty Global', 'Kabel Deutschland', 'Meyer Werft GmbH', 
       'Robert Bosch', 'Ritex', 'Brussels Airlines', 'Verivox', 
       'BMW', 'Oldenburgische Landesbank', 'Google Inc.',
       'Boston Dynamics', 'Innogy','Adam Opel', 'PSA',
       'Abertis Infraestructuras SA', 'Hochtief', 'Apetito','Easyjet', 'Commerzbank', 'ThyssenKrupp', 'Thyssen', 'Tata Steel',
       'Airbus S. A. S.', 'Boeing', 'Alstom', 
       'Credit Suisse', 'Value Expert Vermögensverwaltung','Total S.A.',
       'Siemens AG', 'ThyssenKrupp', 'RTL','Dr. Oetker', 'Rudolf-August Oetker',
       'ALNO', 'Canon', 
       'Diebold Nixdorf', 'Diebold','Hörmann', 'SAS',
       
       'ALDI','Porsche','Boeing',
       'Ryanair', 'Twitter', 'August Storck', 
       'Melitta Gruppe',
       'Mercedes-Benz', 'Continental', 'Arianespace', 'Unilever', 'Toyota', 'Commerzbank', 'Seat', 
        'Fresenius', "Wirecard", "Rheinmetall", "DZ Bank", "Karstadt", "Kaufhof", 
      'Telefónica', "Vodafone", "British Airways", "Mittal", "Thyssengas", "KPMG", 
       'Deutsche Lufthansa', 'Airbus S. A. S.', ' Airbus', 
       'Netflix','EMI'
                    
                    
                    
        # aus Akteursliste Weserkurier
        
'AXA Group',
 
 'Adidas',
 'Aditya Mittal',
 'Airbus',
 
 'Aldi',

 'Amazon.com',"Amazon", 
 'Amprion',
 
 'Apple',
 'Arcelor',

 'Arianespace',
 
 'Arriva',
 'Audi',
 'Axel Mattern',
 'BASF',
 'BLG Group',
 'BLG Logistics Group',
 'BMW',
 'BP',
 
 'Bertelsmann',

 'British Airways',

 'Comdirect',
 'Commerzbank',
 
 'DB AG',
 'DHL',
 'DZ Bank',
 'Dacia',

 'Diebels',
 
 'Easyjet',
 'Edeka',

 'Emirates',
 'Enercon',
 
 'Eurowings',
 
 'Express',
 
 'Fiat',

 'Ford',

 'Fresenius Kabi',
 
 'GC-Gruppe',

 'Gazprom',

 'Gucci',

 'HSBC',

 'Hapag-Lloyd',

 'Harley-Davidson',

 'Hohner',
 
 'Intel',
 'Interhyp',
 
 'KPMG',

 'Karmann',
 'Karstadt',

 'Kaufhof',

 'KfW',

 'Lidl',

 'Lufthansa',

 'Löwenbräu',
 
 'Merck',
 
 'Microsoft',

 'NASA',
 'NPD Group',
 
 'Nestlé',
 'Netflix',
 'Nextbike',
 
 'OMV',

 'Opel',
 'Operations GmbH',

 'Pfennings GmbH',
 'Pfizer',
 'Phil Spencer',
 
 'Porsche',
 'Postbank',
 'Prada',
 'RWE',
 
 'Rana Plaza',
 
 'Renault',
 'René Benko',
 'Rewe',
 'Rheinmetall',

 'Ryanair',

 'Seat',

 'Siemens',

 'Sony',
 

 'SpaceX',
 'Spar',
 'Sparda-Bank',
 'Spotify',
 'Sprecher Haskamp',
 'Steag',
 
 'Stella Artois',
 'Strabag',
 'Blohm',
 
 'Swiss International Air Lines',

 'Tchibo',
 'Telefónica',
 
 'Thyssengas',

 'Toyota',
 'Traditional',
 'Trigema',

 
 'Unilever',
 
 'VP Bank',

 'Vaude',
 
 
 'Verivox',
 
 'Vodafone',
 
 'Volkswagen',
 'Volvo',

 'Wintershall',
 
 'XXXLutz',
 'la Médicale',
 'wolt PLANT',
 'Öko-Institut'
                     
                     
# aus den Wirtschaftssonderseiten
'Accenture',
 'Adidas',
 'Amazon',
 
 'Apple',
 'BASF',
 'BLG',
 'BLG',
 'BLG',

 'Carnival Corporation',

 'DZ Bank',
 
 'Gamesa',

 'Hapag-Lloyd',

 'Lepple Ende',
 'Liquified Petroleum',
 'MSCI',

 'Mediterranean Shipping Company',

 'Netflix',

 'Priscilla',
 
 'Schüller',
 'Senvion',
 'Shipyard Group',
 'Siemens',
 
 'Tchibo',
 
 'Unilever',
 
 'Vier Seifen',
 'Vodafone',

 'Wärtsilä'

                    
                    ]

liste_firmen = list(set(liste_unternehmen))

In [4]:
len(liste_firmen)

208

In [5]:
# Liste Unternehmen aus Akteursspalte herausfiltern
dfw = df_kum[(df_kum["Ressort"]=="Wirtschaft Sonderseiten") | (df_kum["Ressort"]=="Wirtschaft")]
all_akteure = dfw.Akteure.unique()
all_akteure_clean = [x for x in all_akteure if str(x) != 'nan']
all_akteure_list = [ak.split(", ") for ak in all_akteure_clean]
def flatten(t):
    return [item for sublist in t for item in sublist]
flatten_list = flatten(all_akteure_list)
flatten_set = set(flatten_list)

Aus diesem Set alle Firmennamen herausfiltern und in bestehende Firmen-Liste (siehe oben) übernehmen. 
Dann: 


In [6]:
def find_enterprises(df, liste): 
    # zunächst die Nullwerte in der Akteurscolumn ersetzen, sonst Fehlermeldugn
    df.Akteure.fillna("", inplace=True)
    cols = ["Unternehmen", "LW", "Anzahl_Nennungen"]
    new_df = pd.DataFrame(columns=cols)
    for elem in liste:
        elem = elem.strip()
        df_ = df[df["Akteure"].str.contains(elem)]
        lw = df_.LW.mean()
        anzahl = df_.shape[0]
        new_ser = pd.Series([elem, lw, anzahl], index=cols)
        new_df = new_df.append(new_ser,ignore_index=True )
    return new_df
enterprises_df = find_enterprises(dfw, liste_firmen)        

c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [7]:
t

NameError: name 't' is not defined

In [8]:
enterprises_df.sort_values(by="Anzahl_Nennungen", ascending=False).to_csv("./data/unternehmen.csv", decimal=",")

In [9]:
enterprises_df

,Unternehmen,LW,Anzahl_Nennungen
0,Apetito,NaN,0
1,Adam Opel,NaN,0
2,Porsche,16.082325,1
3,Fresenius Kabi,9.680723,1
4,Gucci,17.852320,1
...,...,...,...
203,Unitymedia,NaN,0
204,Stella Artois,23.897925,1
205,BMW,16.096556,5
206,Shipyard Group,10.600878,1


## LW Seite mit und ohne Börse

Di bis Samstag ist jeweils die zweite Seite die Seite mit den Börsendaten

In [15]:
df_wirt = df_kum[df_kum["Ressort"].isin(["Wirtschaft", "Wirtschaft Sonderseiten"])]

In [18]:
df_w = df_kum[df_kum["Ressort"]=="Wirtschaft"]

In [29]:
def find_stock_data(df):
    df["Börse"] = "nein"
    df_ = df[~df["Wochentag"].isin(["Montag", "Sonntag"])]
    all_dates = df_.Datum.unique()
    for date in all_dates: 
        df_date = df_[df_["Datum"]==date]
        erste_wiseite = df_date.Seite.min()
        zweite_wiseite = erste_wiseite + 1
        df.loc[(df["Datum"]==date) & (df["Seite"]==zweite_wiseite), "Börse"] = "ja"
        if df_date[df_date["Seite"]==zweite_wiseite].shape[0] >= 3: 
            print("Achtung!")
            print(f"Am {date} auf Seite {zweite_wiseite} stehen {df_date[df_date['Seite']==zweite_wiseite].shape[0]} Artikel.")
            print(df_date[df_date["Seite"]==zweite_wiseite][["Artikel"]])
    
    merge = df.groupby("Börse", as_index=False).agg({"LW":"mean", "BW":"mean", "DW":"mean"})
    return merge
    
df_wirtschaft = find_stock_data(df_w)    

Achtung!
Am 20.10.2021 auf Seite 16 stehen 3 Artikel.
                                 Artikel
1032                     Wenig verändert
2240      Empfehlung für Arbeitsreformen
7049  Planungen für vierte Stromautobahn
Achtung!
Am 26.10.2021 auf Seite 16 stehen 3 Artikel.
                                   Artikel
1040  Volle Auftragsbücher, wenig Material
2705               Handel bleibt verhalten
7060        Erholung der Wirtschaft stockt
Achtung!
Am 28.10.2021 auf Seite 16 stehen 3 Artikel.
                                         Artikel
1045              Konsolidierung zur Wochenmitte
3262           Lange Übergangsfristen für Banken
7064  Deutsche Bank steigert Gewinn trotz Umbaus
Achtung!
Am 30.10.2021 auf Seite 16 stehen 3 Artikel.
                                        Artikel
1048             Größeres Wachstum als erwartet
7070  Wirtschaft erholt sich durch Privatkonsum
7579                  Kein guter Wochenausklang
Achtung!
Am 04.12.2021 auf Seite 18 stehen 3 Artikel.
      

<ipython-input-29-e2e2eae169cf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Börse"] = "nein"
c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [34]:
df_wirtschaft.to_csv("./data/börse.csv", decimal=",", index=False)

In [33]:
df_w.LW.mean()

15.522276649358105

In [ ]:
df_fin = pd.DataFrame({})

# Sport / Kultur - Artikel nach Sportart, Vereinen, Kultur nach Sparten

## Sportarten zuordnen

In [38]:
df_kum.Ressort.unique()

array(['Lokales', 'Lokalsport', 'Wirtschaft', 'Niedersachsen', 'Bremen',
       'Kinderzeitung', 'Titelseite', 'Vermischtes', 'Sport', 'Medizin',
       'Wissen', 'Leserforum', 'Immobilien & Wohnen', 'Weser-Strand',
       'Kultur', 'Tipps & Termine', 'Regionalsport', 'Rätsel & Roman',
       'Wirtschaft Sonderseiten', 'Reise', 'Politik', 'Beruf & Karriere',
       'Dossier'], dtype=object)

In [48]:
df_sport = df_kum[df_kum["Ressort"]=="Sport"]
df_sport_lokales = df_kum[df_kum["Ressort"]=="Lokalsport"]
df_sport_bremen = df_kum[(df_kum["Ressort"]=="Lokalsport") & (df_kum["Ausgabe"]=="WK")]
df_regsport = df_kum[df_kum["Ressort"]=="Regionalsport"]

In [165]:
df_sport_bremen.Datum.unique()

array(['06.12.2021', '04.11.2021', '08.11.2021', '18.11.2021',
       '09.12.2021', '01.11.2021', '28.10.2021', '11.11.2021',
       '25.11.2021', '21.10.2021', '15.11.2021', '18.10.2021',
       '25.10.2021', '29.11.2021', '02.12.2021', '22.11.2021'],
      dtype=object)

In [5]:
# Liste alle Themen (dort sind die Sportarten zu finden)
# Liste Unternehmen aus Akteursspalte herausfiltern
def find_all_sports(df): 
    
    all_clubs = df.Themen.unique()
    all_clubs_clean = [x for x in all_clubs if str(x) != 'nan']
    all_clubs_list = [ak.split(", ") for ak in all_clubs_clean]
    def flatten(t):
        return [item for sublist in t for item in sublist]
    flatten_list = flatten(all_clubs_list)
    flatten_set = set(flatten_list)
    return list(flatten_set)
flatten = find_all_sports(df_sport_lokales)

In [49]:
liste_sportarten = [
    
    'Leichtathletik',
 'Radsport',
 'Boxen',
 
 'Wrestling',
 
 'Fünfkampf',
 'Bowling und Kegeln',
 
 'Darts',
 
 'Volleyball',
 

 'Biografien',
 'Angelsport',
 'Golfen',
 'Fernsehen',


 'Rudersport',

 'Schwimmen',
 'Motorsport',

 'Tennis',
 'Jazz',
 'Kampfkunstsport',
 
 'Gymnastik und Turnen',
 
 'Schwimmsport und Turmspringen',
 'Pferderennen',
 'Beach Volleyball',
 
 'Skifahren und -springen',
 'Schach',
 
 'Formel 1',
 
 'Frauenfußball',
 'Bodybuilding',
 
 'Reifen',
 'Softball',
 'Eiskunstlauf',
 
 'Handball',

 'Badminton',

 'Halbmarathon',
 
 'weitere Wassersportarten',
 'Rennsport',

 'Inline Skating',
 'Gewichtheben',

 'Snowboard',
 'American Football',
 'Hockey und Feldhockey',

 'Skeleton',

 'weitere Wintersportarten',
 
 'Jagd- und Schießsport',

 'Reitsport',
 'Skibob and Sledging',
 
 'Squash',

 'Tanzen und Tanzarten',
 
 'Eisschnelllaufen',

 'Hockey',

 'Rodelsport',

 'Fußball',

 'Skisport',
 'Behindertensport',
 'Tischtennis',

 'Eishockey',
 
 'Lacrosse',
 
 'Motorradsport',
 
 'Basketball',

 'Fechten',
 'Autorennsport',

 'Langlauf',
 'Snooker und Billard'
    
    
    
]

In [50]:
def find_sports(df, liste): 
    # zunächst die Nullwerte in der Akteurscolumn ersetzen, sonst Fehlermeldugn
    df.Themen.fillna("", inplace=True)
    cols = ["Sportart", "LW", "Anzahl_Nennungen"]
    new_df = pd.DataFrame(columns=cols)
    for elem in liste:
        elem = elem.strip()
        df_ = df[df["Themen"].str.contains(elem)]
        lw = df_.LW.mean()
        anzahl = df_.shape[0]
        new_ser = pd.Series([elem, lw, anzahl], index=cols)
        new_df = new_df.append(new_ser,ignore_index=True )
    return new_df
loksport_ach = df_sport_lokales[df_sport_lokales["Ausgabe"]=="ACH"]
loksport_nor = df_sport_lokales[df_sport_lokales["Ausgabe"]=="NOR"]
sportarten_ach = find_sports(loksport_ach, liste_sportarten)  
sportarten_nor = find_sports(loksport_nor, liste_sportarten)    
sportarten_bre = find_sports(df_sport_bremen, liste_sportarten) 
sport_reg = find_sports(df_regsport, liste_sportarten)

c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [51]:
sport_reg.sort_values(by="Anzahl_Nennungen", ascending=False).to_csv("./data/regionalsportarten.csv", decimal=",", index=False)
#sportarten_nor.sort_values(by="Anzahl_Nennungen", ascending=False).to_csv("./data/loksportarten_nor.csv", decimal=",", index=False)

In [163]:
sportarten_bre.sort_values(by="Anzahl_Nennungen", ascending=False).to_csv("./data/loksportarten_bre.csv", decimal=",", index=False)

## Sportvereine zuordnen


In [127]:
def find_all_clubs(df): 
    
    all_clubs = df.Akteure.unique()
    all_clubs_clean = [x for x in all_clubs if str(x) != 'nan']
    all_clubs_list = [ak.split(", ") for ak in all_clubs_clean]
   
    def flatten(t):
        return [item for sublist in t for item in sublist]
    flatten_list = flatten(all_clubs_list)
    
    flatten_set = set(flatten_list)
    return list(flatten_set)
flatten = find_all_clubs(df_sport_lokales)

In [130]:
flatten

['Luca Nguyen',
 'Tobias Freese',
 'Nina Müller',
 'Bernardo Alexandre',
 'Heiko Bergmann',
 'Sina Stange',
 'Ahmed Mahmoud',
 'Elvedin Bibic',
 'Ernest Nwiee Deede',
 'Pass von Nina',
 'Olaf Häsler',
 'Vechta II.',
 'Oliver Nieß',
 'Peter Beck',
 'Pascal Sperling',
 'Bastian Deke',
 'Burak Tayboga',
 'John Westermeier',
 'Ariel Panzer',
 'Sebastian Drees',
 'Fabian Kauf',
 'Kai-Oliver Bates',
 'Sarro Siemers',
 'Sean Al-Kaledi',
 'Lars Behrens',
 'John Ahrens',
 'Marcel Klein',
 'Torge Wiedenroth',
 'Manuel Harms',
 'Mario Sonntag',
 'Lukas Fischer',
 'Joel Schmidt',
 'Noah Schweichler',
 'Noah Klaas Kruse',
 'Collin Malliaros',
 'Alina Voß',
 'Steven France',
 'Christopher Hupe',
 'Jonas Rohländer',
 'Michael Appleton',
 'Netflix',
 'Jahn Hollenstedt',
 'Ulrich (Unternehmen)',
 'Sebastian Loscha',
 'Georg Bringmann',
 'Mathew Manriquez',
 'Thomas Panitz',
 'Matthias Haase',
 'Susanne Lorenz',
 'Gareth Davies',
 'Saskia Gelhaus',
 'Jessica Szaukellis',
 'David Airich',
 'Kim Sulinski'

In [18]:
sort_out_list_1 = ['Karla Borger',
 'Sam Bennett',
 'Fabian Schmutzler',
 'Felix Loch',
 'Jürgen Born',
 'Emma Hinze',
 'Asger Sörensen',
 'Bowen Wang',
 'Alfons Hörmann',
 'Zverev Knapp',
 'Silvio Heinevetter',
 'Förderer Joachim Löw',
 'Felix Auger-Aliassime',
 'Peter Wright',
 'Günter Neubecker',
 'Amos Pieper',
 'Joey Kelly',
 'Netflix',
 'Mate Pavic',
 'Ulrich (Unternehmen)',
 'Wenqiang Geng',
 'Joshua Kimmich',
 'Jan Aage Fjörtoft',
 'Vladimir Darida',
 'Thorsten Margis',
 'Jamie Leweling',
 'Werner Themann',
 'Raymond van Barneveld',
 'Olaf Thon',
 'Boris Herrmann',
 'Marco Reus',
 'Toni Polster',
 'Philipp Lienhart',
 'Sturm Graz',
 'Ondrej Duda',
 'Lars Stindl',
 'Flemming Pedersen',
 'Nelson Piquet',
 'Aleksey Shpilevski',
 'Rötger Feldmann',
 'Cedric Makiadi',
 'Weil Rodgers',
 'Otto Nerz',
 'Alexandra Sonnenberg',
 'Richard Nixon',
 'Benedikt Pichler',
 'Gianni Infantino',
 'Florent Lambiet',
 'Manfred Loppe',
 'Marco Haase',
 'Kevin Strangmeyer',
 'Rainer Greffrath',
 'Keven Schlotterbeck',
 'Marco Rose',
 'Max Kruse',
 'Nico Hülkenberg',
 'Wang Yang',
 'Andreas Wellinger',
 'Julia Büsselberg',
 'Benedikt Duda',
 'Dieter Nisipeanu',
 'Darwin Nunez',
 'Lim Jong-hoon',
 'Roman Rees',
 'Andrea Belotti',
 'Werner Krutsch',
 'Robin Zentner',
 'Sohn von Klaus Toppmöller',
 'Marcel Schmelzer',
 'Tim Lemperle',
 'Steffen Seibert',
 'Mitch Weiser',
 'Peters',
 'Halvor Egner Granerud',
 'Patrick Wimmer',
 'Sebastian Breuing',
 'Domenico Tedesco',
 'Martin Allegro',
 'Kamil Stoch',
 'Andrei Muresan',
 'Martin Schmidt',
 'Yann Sommer',
 'Maik Machulla',
 'Marco Guida',
 'Guido Burgstaller',
 'Josip Juranovic',
 'Kim Raisner',
 'Christian Lindner',
 'Daniel Andrews',
 'Daniel Kreutzer',
 'Sacha Deshmukh',
 'Robert Klauß',
 'Lewis Hamilton',
 'Evelyn Sparwasser',
 'Julian Draxler',
 'Lina Magull',
 'John Verhoek',
 'Timo Boll',
 'Oliver Glasner',
 'Vanessa Hinz',
 'Jakub Piotrowski',
 'Casper Ruud',
 'Lars Lukas',
 'Alexandra Popp',
 'Helden von Bern',
 'Nicolas Höfler',
 'Justin Kripps',
 'Sven Schwarz',
 'Alexander Zverev',
 'Bernie Ecclestone',
 'Luisa Schulze',
 'Anthony Davis',
 'Konstantin Konga',
 'Stefan Luitz',
 'Claudia Pechstein',
 'Patrick Hager',
 'Stefan Osnabrügge',
 'Stuart Webber',
 'Alberto Riva',
 'Jens Härtel',
 'Marvin Ducksch',
 'Sebastian Mielitz',
 'Malte Meyer',
 'Christoph Schweiger',
 'Andreas Luthe',
 'Sommer von Oliver Glasner',
 'Antonio Adan',
 'Felix Sturm',
 'Jonathan Burkardt',
 'Nanna Skodborg',
 'Aleksandra Zapekina',
 'Julia Taubitz',
 'Mick Schumacher',
 'Masaya Okugawa',
 'Viktorija Andrulyte',
 'McAfee',
 'Ralf Schumacher',
 'Manfred Müller',
 'José Manuel Franco',
 'Sébastien Haller',
 'Svenja Huth',
 'Grigor Dimitrov',
 'Christian Stoll',
 'Lily Zhang',
 'Lea Schüller',
 'Dennis Diekmeier',
 'Dieter Hecking',
 'Robin Braun',
 'José Carlos',
 'Aaron Rodgers',
 'Thomas Arnold',
 'Uwe Gensheimer',
 'Fredrik Rask',
 'Moussa Diaby',
 'Andrea Filser',
 'Zhuang Zedong',
 'Florian Junge',
 'Mark Flekken',
 'Leopoldo Luque',
 'Angelo Ogbonna',
 'Karin Fehres',
 'Lukas Fuhrmann',
 'Santos Haesler',
 'Jonas Boldt',
 'Jürgen Klinsmann',
 'Finn Ole Becker',
 'Dajana Eitberger',
 'Herzen von Kiel',
 'Anthony Losilla',
 'Lauterer Block',
 'RTL Television',
 'Lionel Messi',
 'Emina Herenda',
 'Kevin Kampl',
 'Alexander Schlobohm',
 'Rudi Assauer',
 'Bastian Dankert',
 'Simon Terodde',
 'Grischa Prömel',
 'Viktor Szilagyi',
 'Julia Maidhof',
 'Mats Hummels',
 'Lukas Podolski',
 'Clay Regazzoni von Ferrari',
 'Christlich Demokratische Union Deutschlands',
 'Zinedine Zidane',
 'Lukas Mai',
 'Sonntag Zweite',
 'Fritz Keller',
 'Serena Williams',
 'Leon Draisaitl',
 'Paulina Krumbiegel',
 'Jan Thielmann',
 'Florian Scholz',
 'Niklas Süle',
 'Martin Minchev',
 'Michael Wiesinger',
 'Frank Baumann',
 'Andy Murray',
 'Josh Young',
 'Pascal Testroet',
 'Kyle Walker',
 'Anna Weidel',
 'Mads Pedersen',
 'Henning Matriciani',
 'Mina Sato',
 'Sebastian Hoeneß',
 'Martin Kaymer',
 'John Gimson',
 'Ryan Charles',
 'Valentino Rossi',
 'Kristina Bolbat',
 'Roberto Massimo',
 'Dominik Kohr',
 'Mihai Bobocica',
 'Michael Scharf',
 'Marketa Vondrousova',
 'Sebastian Rudy',
 'Bobby Burns',
 'Agata Barwinska',
 'Thomas Weikert',
 'Marius Funk',
 'Derrick Allen',
 'Franziska Preuß',
 'Harry Giles',
 'Patrik Kühnen',
 'Marius Wolf',
 'Thomas Müller',
 'Stefan Leyhe',
 'Jarelle Reischel',
 'Joel Birlehm',
 'Mohamed Salah',
 'Stürmer Lukas Hinterseer',
 'Carsten Sander',
 'Uwe Schubert',
 'Karsten Tadda',
 'Robert Harting',
 'Stefan Kießling',
 'Daria Brehm',
 'Christian Kukuk',
 'Ralf Fährmann',
 'Tobias Rieder',
 'Christopher Grotheer',
 'Thorsten Bolder',
 'Lukas Mertens',
 'Roubaix Gold',
 'Thomas Päch',
 'Ole Werner',
 'Blanka Biro',
 'Hannes Wolf',
 'Elena Krawzow',
 'Alexander Lukaschenko',
 'Roberto Mancini',
 'Felix Czerny',
 'Audi',
 'Chris Förster',
 'Christian Frodl',
 'Lukas Kübler',
 'Dieter Burdenski',
 'Michael Schumacher',
 'Mattias Falck',
 'Victor Palsson',
 'Axel Plaat',
 'David Rademacher',
 'Malick Thiaw',
 'Kevin Krawietz',
 'Nigel Mansell',
 'ZDF',
 'Marcel Mehlem',
 'Admira Wacker',
 'Achim Beierlorzer',
 'Alexander Schwolow',
 'Stadion Maske',
 'Severin Freund',
 'Felix Klaus',
 'Alessandro Hämmerle',
 'Arrow McLaren',
 'Sargis Adamjan',
 'Lucas Cueto',
 'Adan Mohamed',
 'Burak Aksin',
 'Wassily Krastanas',
 'Lucas Hernández',
 'Verschiedene',
 'Fabijan Buntic',
 'Marc Hensel',
 'Leroy Sané',
 'Royal Antwerpen',
 'Tommy Doyle',
 'Michal Skoras',
 'Benedikt Doll',
 'Stefan Ortega',
 'Nicolai Rapp',
 'Aslan Karazew',
 'Andreas Geisel',
 'Georginio Rutter',
 'Christoph Kramer',
 'Martin Nörl',
 'Carina Vogt',
 'Amazon.com',
 'Christopher Antwi-Adjei',
 'Jacob Laursen',
 'Tim Wohlgemuth',
 'Markus Eisenbichler',
 'David Howell',
 'Rudi Völler',
 'Martin Ponsiluoma',
 'Qatar Airways',
 'Rainer Schüttler',
 'Dean Smith',
 'Sam Schreck',
 'Daniil Medwedew',
 'Alexis Pinturault',
 'Jens Hofer',
 'Ulrich Mäurer',
 'Carolin Langenhorst',
 'Mahid Matar',
 'Peter Knäbel',
 'Sergio Cordova',
 'Mike Gabel',
 'Zach Walton',
 'Taylor Fritz',
 'Pierre de Coubertin',
 'Ryan Gravenberch',
 'Kevin Boakye-Schumann',
 'Glenn Cowan',
 'Joachim Löw',
 'Agustina Cosachov',
 'Paul Berg',
 'Hofmann Hannover',
 'Manuel Schäffler',
 'Benedikt Höwedes',
 'Tina Hermann',
 'Mark van Bommel',
 'Harry Maguire',
 'Jannes Hundt',
 'Martin Schimke',
 'Jessica Winter',
 'Max Eberl',
 'Sjoeke Nüsken',
 'Michael Kohlmann',
 'Fabiano Caruana',
 'Reiner Calmund',
 'Johannes Kühn',
 'Stefan Mayer',
 'Klaus-Dieter Fischer',
 'Spotify',
 'Stefan Schafheitlin',
 'Yang Wang',
 'Florian Hempel',
 'Deniz Aytekin',
 'Sarah Köhler',
 'Konstantinos Mavropanos',
 'Daley Blind',
 'Jordan Beyer',
 'Eugen Uschpol',
 'Sascha Mölders',
 'Dominic Thiem',
 'Tom Cichon',
 'Volker Fahlbusch',
 'Antje Lauenroth',
 'Carole da Silva',
 'Marina Hegering',
 'Ivan Klasnic',
 'Christoph Harting',
 'Toni Söderholm',
 'Elana Meyers',
 'Stefan Holz',
 'John Millman',
 'Daniel Gordon',
 'Pedro Porro',
 'Marco Sturm',
 'Mirela Rahneva',
 'Magnus Gullerud',
 'Felix Neureuther',
 'Andreas Bovenschulte',
 'Burak Yilmaz',
 'Henk Groener',
 'Stadion De Kuip',
 'Claus Gröbner',
 'Volker Zerbe',
 'Arne Friedrich',
 'Khaled Narey',
 'Linton Maina',
 'Juliane Seyfarth',
 'Tobias Mohr',
 'Ruta del Sol',
 'Faride Alidou',
 'Felix Zwayer',
 'Toni Eggert',
 'Selina Freitag',
 'Maximilian Arnold',
 'Karl Lauterbach',
 'Strahinja Pavlovic',
 'Sergi Barjuan',
 'Mathieu Valbuena',
 'Péter Gulácsi',
 'Pellegrino Matarazzo',
 'Hannelore',
 'Yasin Chaaban',
 'Veronika Rücker',
 'Jacqueline Lölling',
 'Michael Mai',
 'Martina Voss-Tecklenburg',
 'Rafa Mir',
 'Julia Görges',
 'Marco Koch',
 'Jos Luhukay',
 'Paul Pogba',
 'Arsène Wenger',
 'Lukas Middendorf',
 'Aleksander Aamodt Kilde',
 'Mohammed bin Salman',
 'Gabriele Gravina',
 'Thomas Dähne',
 'Sieger Messi',
 'Alexander Sather',
 'Claus Knupfer',
 'Rolf Rüssmann',
 'Niklas Schmidt',
 
 'Sören Gonther',
 'Lisa Ashton',
 'Roger Schmidt',
 'Tamara Boros',
 'Gabriel Clemens',
 'Sergio Perez',
 'Marco Kurth',
 'Hilde Gerg',
 'Milton Keynes',
 'Robert Gumny',
 'Joan Jordán',
 'Schultz Dieter Hecking',
 'Carrington Love',
 'Lucas Hernandez',
 'Lando Norris',
 'Frank Büter',
 'Stefan Mitrovic',
 'Daniel del Sol',
 'Thomas Hitzlsperger',
 'Mario Götze',
 'Denise Herrmann',
 'Wenzel Michalski',
 'Carsten Eich',
 'Brose Bamberg',
 'Montafon',
 'Luca Zander',
 'Gregor MacLeod',
 'Toni Turek',
 'Andreas Herzog',
 'Silje Opseth',
 'Thomas Meunier',
 'Marco Richter',
 'Robin Dutt',
 'Moritz Heyer',
 'Tim Borowski',
 'Matthias Ginter',
 'Aki Watzke',
 'Ramona Hofmeister',
 'Rafal Gikiewicz',
 'Felix Magath',
 'Bryan Linssen',
 'Peter Peters',
 'Micky van de Ven',
 'Urs Fischer',
 'Arnd Peiffer',
 'Natalie Geisenberger',
 'Philip Kostic',
 'Alexander Gassner',
 'Jeffrey Hoogland',
 'Jessica Neterebski',
 'Thomas Schaaf',
 'Ralf Rangnick',
 'Martin Grothkopp',
 'Maria Höfl-Riesch',
 'Stefan Bötticher',
 'Nico Gießelmann',
 'Sharon Beck',
 'Jürgen Klopp',
 'Sprecher Fuhrmann',
 'Antonio Rüdiger',
 'Jamal Musiala',
 'Kerem Demirbay',
 'Michael Ströll',
 'Spanien zu Rayo',
 'Ottmar Walter',
 'Sofiane Diop',
 'John Degenkolb',
 'Constantin Schmid',
 'Danny Schwarz',
 'Angelique Kerber',
 'Zidan Sertdemir',
 'Jörg Wontorra',
 'Tobias Strobl',
 'Bernd Berkhahn',
 'Jonathan Burkhardt',
 'Thomas Koch',
 'Tobias Eder',
 'Nico Rosberg',
 'Recep Tayyip Erdogan',
 'Dietrich Mateschitz',
 'Mensur Suljovic',
 'Nico Schlotterbeck',
 'Christoph Frankowski',
 'Katharina Filter',
 'Mark Bult',
 'Marin Pongracic',
 'Maciel Mota',
 'Meike Schmelzer',
 'Sophie Friedrich',
 'Martin Schindler',
 'Dimitrios Grammozis',
 'Arnd Zeigler',
 'Kim Kalicki',
 'Roberto Albanese',
 'Bastian Schweinsteiger',
 'Jesper Lindström',
 'Andrea Berg',
 'Uwe Schwenker',
 'Peng Shuai',
 'Arne Maier',
 'Sam Snead',
 'Alexander Ehl',
 'Axel Kromer',
 'Alexander Nouri',
 'Ridle Baku',
 'Alexander Zickler',
 'Gerhard Struber',
 'Sebastian Vettel',
 'Ingo Froböse',
 'Michael Carrick',
 'Rickey Paulding',
 'Gernot Tripcke',
 'Stadion Camp',
 'Antonio di Salvo',
 'Steffen Tigges',
 'Lena Dürr',
 'Oscar Schönfelder',
 'Laura Freigang',
 'Adriano Celentano',
 'Philipp Wehler',
 'Philipp Hofmann',
 'Florian Wirtz',
 'Erik Lesser',
 'Melanie Leupolz',
 'André Silva',
 'Miroslav Klose',
 'Kevin Behrens',
 'Gordon Mathers',
 'Chris Evert',
 'Franziska Brauße',
 'Lucien Favre',
 'Emanuel Aiwu',
 'Khoren Bayramyan',
 'Eric Maxim Choupo-Moting',
 'Rainer Koch',
 'Wolfgang Wolf',
 'Karim Bounou',
 'Tai Odiase',
 'Raphael Nadal',
 'Jupp Derwall',
 'Tabea Waßmuth',
 'Christoph Dabrowski',
 'Oliver Bierhoff',
 'Emily Bölk',
 'Etihad Airways',
 'Niklas Dorsch',
 'Nick Woltemade',
 'Viktoria Berlin',
 'Rui Pedro Soares',
 'Manuel Feller',
 'Katharina Braun',
 'Daryl Morey',
 'Caja Mágica',
 'Benjamin Pavard',
 'Lukas Zerbe',
 'Annika Schleu',
 'Pat McAfee',
 'Alexander Schüller',
 'Simon Makienok',
 'Torsten Lieberknecht',
 'Fredi Bobic',
 'Markus Merk',
 'Markus Krösche',
 'Kevin Volland',
 'Valtteri Bottas',
 'Alexander Fritzsche',
 'Sydney Lohmann',
 'Alexander Koke',
 'Tayfun Korkut',
 'Lukas Muszong',
 'Marlene Schmotz',
 'Herbert Hainer',
 'Max Hopp',
 'Simon Geschke',
 'Thomas Reis',
                   
 'Marco Hiller',
 'David Schmidt',
 'Emma Aicher',
 'Franz Bokop',
 'Ramón Sánchez-Pizjuán',
 'Bashkim Adjini',
 'Johannes Lochner',
 'Niels Rossow',
 'Denis Zakaria',
 'Diego Armando Maradona',
 'Aitor Ruibal',
 'Philipp Oswald',
 'Theo Gebre Selassie',
 'Remko Pasveer',
 'Florian Kainzinger',
 'Markus Weinzierl',
 'Jana Tyschtschenko',
 'Uwe Stöver',
 'André Hahn',
 'Mikaela Shiffrin',
 'Helge Leonhardt',
 'Robert Skov',
 'Johannes Golla',
 'Tomaszow Mazowiecki',
 'Patrick Wiencek',
 'Klaus Allofs',
 'Abdullah Basdas',
 'Mason Mount',
 'Sebastian Klaas',
 'Lob von Clemens',
 'Jannik Sinner',
 'Kevin Mbabu',
 'Deutscher Fußball-Bund',
 'Daniel Kaufmann',
 'Stadion von Union Berlin',
 'Alain Prost',
 'Davie Selke',
 'Verwaltungs-Berufsgenossenschaft',
 'Huang Chun',
 'Möller Daehli',
 'del Rey',
 'Tyler Adams',
 'Irene Schouten',
 'Maximilian Levy',
 'Hansi Flick',
 'Michael Esser',
 'Martin Hamann',
 'Alphonso Davies',
 'Amine Adli',
 'Philipp Förster',
 'Katharina Liensberger',
 'David Moyes',
 'Sebastien Haller',
 'Olaf Scholz',
 'Pal Dardai',
 'Timo Werner',
 'Fabio Quartararo',
 'Andreas Dotzauer',
 'Neuling Lukas Nmecha',
 'Gerard Piqué',
 'Tim Pütz',
 'Daniel Cottäus',
 'Lisa Theresa Hauser',
 'Mitchell Weiser',
 'Julian Rauchfuß',
 'Tim Torn Teutenberg',
 'Michael Damgaard',
 'Claudia Bokel',
 'Daniel Evans',
 'Stefan Klett',
 'Mourad Aliev',
 'Omar Marmoush',
 'Dietmar Hopp',
 'Justus Strelow',
 'Patrick Beckert',
 'Maximilian Göppel',
 'Patrik Schick',
 'Moussa Niakhaté',
 'Tiger Woods',
 'Ferrari',
 'Clemens Fritz',
 'Jerome Roussillon',
 'Holger Stanislawski',
 'Bernardo Silva',
 'Ingo Weiss',
 'Sturla Holm',
 'Ruben Vargas',
 'Patrick Reimer',
 'Manu Koné',
 'Roman Birjukov',
 'Justin Njinmah',
 'Detroit Red',
 'Ioannis Topalidis',
 'Maximilian Kieffer',
 'Bundesgerichtshof',
 'Pass von Marco',
 'Michael Geiger',
 'Kevin Jochems',
 'Ishak Belfodil',
 'Lukas Klostermann',
 'Louis Poznanski',
 'Marie Tajdel',
 'Tobias Welz',
 'Leo Pföderl',
 'Pedro Goncalves',
 'Lang Lang',
 'Yannik Keitel',
 'André Schubert',
 'Erich Ribbeck',
 'Fraport',
                   
 'Naomi Osaka',
 'Ray McCallum',
 'Martin Harnik',
 'Julia Putinzewa',
 'Sebastian Polter',
 'Hendrik Wüst',
 'Omid Nouripour',
 'Pascal Ackermann',
 'Markus Frömming',
 'Lisa Brennauer',
 'Christopher Trimmel',
 'Günter Ebert',
 'Jannes Wieckhoff',
 'Norbert Loch',
 'Cedric Brunner',
 'Quentin Fillon Maillet',
 'Rouven Schröder',
 'Hugo Novoa',
 'van der Werff',
 'Marcus Ehning',
 'Dominik Becker',
 'Julian Ryerson',
 'Felicitas Rauch',
 'Marita Kramer',
 'Peter Gojowczyk',
 'Turki Alali',
 'Johannes Herber',
 'Max Hartung',
 'Jonas David',
 'Michael Schulz',
 'Chris Richards',
 'Julian Weigl',
 'Maximilian Mechler',
 'Franco di Santo',
 'Jadon Sancho',
 'Larissa Drygala',
 'Jörg Schmadtke',
 'Vincent Heilmann',
 'David Navara',
 'Albert Mengue Ayissi',
 
 'Philip Tietz',
 'Gregorio Paltrinieri',
 'Johan Cruyff',
 'Reece Oxford',
 'Christlich-Soziale Union in Bayern',
 'Sönke Wortmann',
 'Johanna Stockschläder',
 'Uli Hoeneß',
 'FIFA',
 'Ulrike Hiller',
 'Sander Sagosen',
 'Jannik Huth',
 'Max Verstappen',
 'Michael Heinig',
 'Roxana Maracineanu',
 'Frank Williams',
 'Rodrigo Zalazar',
 'Tommy Haas',
 'Vincent Keymer',
 'Lukas Reichel',
 'Sven Michel',
 'Merle Frohms',
 'Gerrit Nieberg',
 'Alexander Okafor',
 'Jonathan David',
 'Bob Hanning',
 'Dagmar Freitag',
 'DFL Deutsche Fußball Liga',
 'Michael Jacobsen',
 'Mitchel Bakker',
 'Gene Haas',
 'Bretagne Schütz',
 'Antonio Conte',
 'Jan Didjurgeit',
 'Angelo Stiller',
 'Alina Grijseels',
 'Jan van Eijden',
 'Helmut Rahn',
 'Nico Schulz',
 'André Korff',
 'Kevin Möhwald',
 'Emma Plasschaert',
 'Marvin Friedrich',
 'Marcel Halstenberg',
 'Helmut Schön',
 'Manuel Neuer',
 'Tyrone Nash',
 'Luca Toni',
 'Klaus Filbry',
 'Guilherme Ramos',
 'Maximilian Dittgen',
 'Ronald Koeman',
 'Gael Monfils',
 'Gunda Niemann-Stirnemann',
 'Manuel Riemann',
 'Franz Steinle',
 'Enrico Valentini',
 'Jorik Wulff',
 'Marcel Sabitzer',
 'Rodgers Ende',
 'Janik Haberer',
 'Werner Werder',
 'Konrad Laimer',
 'Kevin Yebo',
 'Raphael Czichos',
 'Timo Baumgartl',
 'Roger Wittmann',
 'Bernd Neuendorf',
 'Marco Fuchs',
 'Kliment Kolesnikow',
 'Nikola Mektic',
 'Belinda Bencic',
 'Hasan Salihamidzic',
 'Marta Kostjuk',
 'Stefan Peters',
 'Mieke Kröger',
 'Klaus Schormann',
 'Daniel Farke',
 'Enes Kanter',
 'Benjamin Cortus',
 'Robert Habeck',
 'Katrin Göring-Eckardt',
 'Markus Kauczinski',
 'Paul II.',
 'Karen Chatschanow',
 'Christian Schwaiger',
 'Julian Börner',
 'Xenia Smits',
 'Andreas Bergmann',
 'Dr. Burkard Plenge',
 'Novak Djokovic',
 'Michael Frontzeck',
 'Villa Fiorito',
 'Giovanni Trapattoni',
 'Danny Blum',
 'Mona Barthel',
 'Jelena Rybakina',
 'Il Mondo',
 'Thomas Strobl',
 'Hendrik Dombek',
 'Amy Wang',
 'Fernando Alonso',
 'Stefan Kuntz',
 'Florian Kainz',
 'Michael Müller',
 'Oskar Kosche',
 'Domagoj Duvnjak',
 'Victoria Stirnemann',
 'Jürgen Kohler',
 'Luca Pfeiffer',
 'Streich von Lewandowski',
 'Carsten Watzl',
 'Dennis Schröder',
 'Jörg Roßkopf',
 'André Trulsen',
 'Claudia Rücker',
 'Benjamin Henrichs',
 'Zhang Gaoli',
 'Florian Badstübner',
 'Christian Brand',
 'Noah Okafor',
 'Edmond Tapsoba',
 'Bo Svensson',
 'Óscar García García',
 'Yussuf Poulsen',
 'Mauricio Pochettino',
 'Topstar',
 'Xavi Hernández',
 'Fritz Walter',
 'Leonie Fiebig',
 'Simon Schempp',
 'Wolfgang Hesl',
 'Tobias Kröger',
 'Andreas Bornemann',
 'Lukas Hradecky',
 'Nastasja Schunk',
 'Winter Spieler',
 'Alexander Mühling',
 'Philipp Petzschner',
 'Andi Herzog',
 'Ole Gunnar Solskjaer',
 'Santiago Garcia',
 'Eric Bailly',
 'Ducati',
 'Andreas Mies',
 'Schweizer Franken',
 'Cristiano Ronaldo',
 'Borja Iglesias',
 'Fabian Klos',
 'Justin Kuchinke',
 'Johan Cruijff',
 'David Alaba',
 'Pius Paschke',
 'Detlef Uibel',
 'Karl Geiger',
 'Sebastian Coe',
 'Tim Walter',
 'Sven Ulreich',
 'Alireza Jahanbakhsh',
 'Oliver Marach',
 'Barbara',
 'Frank Neubarth',
 'Stefan Kraft',
 'Lukas Nmecha',
 'Steffen Baumgart',
 'Hermann Schüller',
 'Jessica von Bredow-Werndl',
 'Pierre Casiraghi',
 'Sebastian Heymann',
 'Henry Kissinger',
 'Josko',
 'Roberto Firmino',
 'Philipp Horn',
 'Andreas Müller',
 'Torsten Frings',
 'Greta Thunberg',
 'Karim Benzema',
 'Phillip Semechin',
 'Paulo Otavio',
 'Jo Kimmich',

 'Carlos Alcaraz',
 'Christian Horner',
 'Leonardo Bonucci',
 'Werner Liebrich',
 'Marco Odermatt',
 'Werners Werder',
 'Bruno Labbadia',
 'Emil Forsberg',
 'Krzysztof Ratajski',
 'Thomas Bliwier',
 'Adi Hütter',
 'Koen Casteels',
 'Sebastian Samuelsson',
 'Stefan Niemeyer',
 'Chen Xingtong',
 'Max Aarons',
 'Kristian Karlsson',
 'Madoka Kodama',
 'Dimitri Payet',
 'Hopps Buch',
 'Donata Hopfen',
 'Julia Sude',
 'Julian Brandt',
 'Marina Cicuit',
 'Christian Wulff',
 'Eddie Howe',
 'Traditional',
 'Raphael Borré',
 'Andreas von Thien',
 'Rolf Aldag',
 'Paolo Maldini',
 'Oliver Baumann',
 'Marius Winkelmann',
 'Stefan Baumeister',
 'Amelie Berger',
 'Oliver Reck',
 'Andreas Wank',
 'van Bommel',
 'Auto Technisches Spezialzubehör',
 'Hans-Günter Klemm',
 'Christian Streich',
 'Anthony Modeste',
 'Robert Andrich',
 'Timo Schultz',
 'Ilka Seeler',
 'Martin Pakula',
 'Ralph Denk',
 'Finn Porath',
 'Sabine Winter',
 'Robert Hoyzer',
 'Reinhold Messner',
 'Angel Gomes',
 'Andrej Rubljow',
 'Ruwen Filus',
 'Palma de Mallorca',
 'Makoto Hasebe',
 'Adrian Lewis',
 'Holly Cova',
 'Marcus Mann',
 'Otto Rehhagel',
 'Karl-Heinz Rummenigge',
 'Nürnberger Stadion',
 'Christina',
 'Pablo Fornals',
 'Hamilton Ende',
 'Richard Prause',
 'Christopher Nkunku',
 'Philipp Tietz',
 'Diogo Goncalves',
 'Aaron McCarey',
 'Klara Bühl',
 'Sepp Herberger']

In [22]:
flatten_ = [x for x in flatten if x not in sort_out_list_1]

In [25]:
flatten_.count("Werder Bremen")

0

In [14]:
def find_clubs(df, liste): 
    # zunächst die Nullwerte in der Akteurscolumn ersetzen, sonst Fehlermeldugn
    df.Akteure.fillna("", inplace=True)
    cols = ["Vereine", "LW", "Anzahl_Nennungen"]
    new_df = pd.DataFrame(columns=cols)
    for elem in liste:
        elem = elem.strip()
        df_ = df[df["Akteure"].str.contains(elem)]
        lw = df_.LW.mean()
        anzahl = df_.shape[0]
        new_ser = pd.Series([elem, lw, anzahl], index=cols)
        new_df = new_df.append(new_ser,ignore_index=True )
    return new_df
clubs_df = find_clubs(df_sport, flatten)     

c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
<ipython-input-14-a901428b469f>:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  df_ = df[df["Akteure"].str.contains(elem)]


## Kultur nach Sparten

In [32]:
liste_sparten = ["Literatur", "Kino", "Theater", "Musik", "Fernsehen", "Bildende Künste", "Promi"]

In [33]:
df_kultur = df_kum[df_kum["Ressort"]=="Kultur"]

In [34]:
def find_culture_sections(df, liste): 
    # zunächst die Nullwerte in der Akteurscolumn ersetzen, sonst Fehlermeldugn
    df.Themen.fillna("", inplace=True)
    cols = ["Kultursparten", "LW", "Anzahl_Nennungen"]
    new_df = pd.DataFrame(columns=cols)
    for elem in liste:
        elem = elem.strip()
        df_ = df[df["Themen"].str.contains(elem)]
        lw = df_.LW.mean()
        anzahl = df_.shape[0]
        new_ser = pd.Series([elem, lw, anzahl], index=cols)
        new_df = new_df.append(new_ser,ignore_index=True )
    return new_df
culture_df = find_culture_sections(df_kultur, liste_sparten)   

c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [36]:
culture_df.rename(columns={"Promi":"Promis"}, inplace=True)

In [37]:
culture_df.sort_values(by="Anzahl_Nennungen", ascending=False).to_csv("./data/kultur.csv", decimal=",")

In [81]:
df.Ausgabe.unique()

array(['ACH', 'NOR', 'WK'], dtype=object)

## Sportvereine aus Volltext herauslesen

In [77]:
# wichtige Sportvereine Bremen Sport
liste_sportvereine = ["FC Oberneuland", "Werder Bremen", "Grün-Gold-Club", "Fischtown Pinguins", \
                     "Eisbären Bremerhaven", "Bremer SV", "Bremer HC", "HC Bremen"]
liste_sportvereine2 = ["Oberneuland", "Werder", "Grün-Gold", "Pinguins", \
                     "Eisbären", "Bremer SV", "HC Bremen", "SV Bremen", "Bremen 1860"]

In [96]:
dfs = df_volltext_sport.copy()

In [99]:
dfs = dfs.drop_duplicates(subset="TeilartikelId")

In [100]:
dfs

,TeilartikelId,Dachzeilentext,Ueberschrifttext,Untertiteltext,Artikeltext
0,19466,Fußball,Werder-Frauen enttäuschen,,Meppen. Das war nichts: Mit 1:4 verloren die ...
1,19692,Handball,"Achte Niederlage für Werders ""Juniorteam""",,Bremen. Die Handballerinnen des SV Werder Brem...
2,22953,,Ständig neue Probleme,Nach Inkrafttreten der Corona-Warnstufe 2 ist ...,Bremen. Vor dem Hintergrund weiter steigender ...
3,25147,Hockey,Club zur Vahr vergibt Siebenmeter und verliert,,Bremen. Das 1:6 des Clubs zur Vahr in der Hock...
4,25150,Tanz-Weltmeisterschaft im Contemporary,Als Nachrücker zur Weltmeisterschaft,Die Bremer Gruppe „The Next Generation“ hat be...,Bremen. Am kommenden Montag startet für Traine...
...,...,...,...,...,...
909,25832,,Weniger Fans bei den Pinguins,Änderungen am Spielplan,Bremerhaven. Die kritische Corona-Lage hat wie...
943,18234,Volleyball,"1860 verliert 1:3, der Trainer sieht Rot",,Bremen. Am Ende wurde es den Schiedsrichtern z...
950,18402,,Pinguins siegen mit 6:4 gegen München,,Bremerhaven. Ein Wechselbad der Gefühle durchl...
1023,21897,Cindy König im Interview,"""Ich bin hier eine kleine Attraktion""",Die frühere Werder-Spielerin Cindy König erzäh...,"Frau König, warum Japan?\n\nCindy König: Ich h..."


In [101]:
cols = ["Verein", "Liste_Artikel", "Anzahl_Artikel"]
new_df = pd.DataFrame(columns=cols)

for ver in tqdm(liste_sportvereine): 
    liste_vereine = []
    df_ = dfs[(dfs["Dachzeilentext"].str.contains(ver)) | (dfs["Ueberschrifttext"].str.contains(ver)) | \
            (dfs["Untertiteltext"].str.contains(ver)) | (dfs["Artikeltext"].str.contains(ver))]
    if df_.shape[0]>0: 
        liste_vereine.extend(df_.TeilartikelId.unique())
        
        new_ser = pd.Series([ver, liste_vereine, df_.shape[0]], index=cols)
        new_df = new_df.append(new_ser, ignore_index=True)
    


100%|██████████| 8/8 [00:00<00:00, 225.30it/s]


In [102]:
new_df

,Verein,Liste_Artikel,Anzahl_Artikel
0,FC Oberneuland,"[19690, 20499, 21523, 16881, 16829, 18650, 203...",28
1,Werder Bremen,"[19692, 20499, 15068, 15319, 20645, 21523, 244...",29
2,Grün-Gold-Club,"[22953, 25323, 25688, 17322, 19871, 20634, 215...",15
3,Fischtown Pinguins,"[15688, 15094, 19257, 21103, 21523, 22541, 161...",28
4,Eisbären Bremerhaven,"[24482, 15099, 21523, 24878, 15971, 15975, 159...",13
5,Bremer SV,"[15563, 15097, 21523, 22066, 16881, 18650, 203...",24
6,Bremer HC,"[25147, 15558, 21523, 22092, 22952, 15326, 159...",16
7,HC Bremen,"[16696, 18042, 21523, 25143, 16445, 22295, 229...",13


In [113]:
t = df[df["SplitId"]==22952]
t

,SplitId,ArtikelId,Artikel,ErscheinungsId,AusgabeId,Datum,Seite,BW,DW,LW,...,Beschreibung,Handlungsorte,Akteure,Themen,Ausgabe,Layerlink,Ausgabeteil,Wochentag,offen/codiert,Ressort


In [117]:
cols = ["Verein", "LW", "BW", "DW", "Anzahl Artikel"]
sport_df = pd.DataFrame(columns=cols)
for row in new_df.itertuples():
    verein = row.Verein
    artikel_liste = row.Liste_Artikel
    
    df_ = df[df["SplitId"].isin(artikel_liste)]
    
    lw = df_.LW.mean()
    bw = df_.BW.mean()
    dw = df_.DW.mean()
    anzahl = df_.SplitId.nunique()
    ser_data = [verein, lw, bw, dw, anzahl]
    new_ser = pd.Series(ser_data, index=cols)
    sport_df = sport_df.append(new_ser, ignore_index=True)
    

In [120]:
sport_df.to_csv("./data/regionalsport_vereine.csv", index=False, decimal=",")

# Lokales

In [27]:
df_bremen = dfnk[dfnk["Ressort"]=="Bremen"]

In [28]:
df_bremen.Seitentitel.unique()

array(['Gemeinsam In Bremen', 'Bremen', 'Fotoreportage'], dtype=object)

## Seitenzahl Bremen einfügen

In [29]:
def create_pagenumber(df): 
    
    
    # Fehlercheck
#     if (len(ressorts)>1) | (ressorts[0]!="Lokales"): 
#         raise ValueError("Achtung, multiple Ressorts entdeckt.")
   
    df["Seite_im_Lokalteil"] = 0
    df["Sortierung"] = 0
    all_dates = df.Datum.unique()
    for date in all_dates: 
        df_ = df[df["Datum"]==date]
        
        for aus in df.Ausgabe.unique(): 
            df_aus = df_[df_["Ausgabe"]==aus]
            seite_min = df_aus.Seite.min()
            seite_max = df_aus.Seite.max()
            print(seite_min)
            print(seite_max)
            print(aus, date)
            for c, i in enumerate(range(seite_min, seite_max+1)):
                df.loc[(df["Datum"]==date) & (df["Seite"]==i), "Seite_im_Lokalteil"] = "Seite " + str(c+1)
                df.loc[(df["Datum"]==date) & (df["Seite"]==i), "Sortierung"] = c+1
    
    return df
                
            
df_lokseiten = create_pagenumber(df_bremen)     

<ipython-input-29-afc096ead0b7>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Seite_im_Lokalteil"] = 0
<ipython-input-29-afc096ead0b7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sortierung"] = 0
c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

9
13
WK Dienstag, 9. November 2021
9
13
NOR Dienstag, 9. November 2021
9
13
ACH Dienstag, 9. November 2021
7
11
WK Freitag, 12. November 2021
7
11
NOR Freitag, 12. November 2021
7
11
ACH Freitag, 12. November 2021
9
11
WK Dienstag, 16. November 2021
9
11
NOR Dienstag, 16. November 2021
9
11
ACH Dienstag, 16. November 2021
9
13
WK Dienstag, 23. November 2021
9
13
NOR Dienstag, 23. November 2021
9
13
ACH Dienstag, 23. November 2021
9
13
WK Samstag, 27. November 2021
9
13
NOR Samstag, 27. November 2021
9
13
ACH Samstag, 27. November 2021
9
13
WK Dienstag, 7. Dezember 2021
9
13
NOR Dienstag, 7. Dezember 2021
9
13
ACH Dienstag, 7. Dezember 2021
9
11
WK Donnerstag, 4. November 2021
9
11
NOR Donnerstag, 4. November 2021
9
11
ACH Donnerstag, 4. November 2021
9
11
WK Dienstag, 19. Oktober 2021
9
11
NOR Dienstag, 19. Oktober 2021
9
11
ACH Dienstag, 19. Oktober 2021
9
12
WK Mittwoch, 27. Oktober 2021
9
12
NOR Mittwoch, 27. Oktober 2021
9
12
ACH Mittwoch, 27. Oktober 2021
7
13
WK Samstag, 6. Novem

In [37]:
df_b = df_lokseiten.groupby(["Ausgabe", "Seite_im_Lokalteil"], as_index=False)\
.agg({"LW":"mean", "Sortierung":"min", "SplitId":"count"})

In [44]:
df_hb.to_csv("./data/seiten_bremen_wide.csv", index=False, decimal=",")

In [ ]:
df_b_w = df_lokseiten.

In [42]:
def create_pagenumber_wide(df): 
    
    cols = ["Seite", "LW WK", "LW ACH", "LW NOR", "Sortierung"]
    new_df = pd.DataFrame(columns=cols)
    # Fehlercheck
#     if (len(ressorts)>1) | (ressorts[0]!="Lokales"): 
#         raise ValueError("Achtung, multiple Ressorts entdeckt.")
   
    
    alle_seiten = df.Seite_im_Lokalteil.unique()
    print(alle_seiten)
    
   
    
    for se in alle_seiten:
        
        df_ = df[df["Seite_im_Lokalteil"]==se]
        sortierung = df_.iloc[0]["Sortierung"]
        
        lw_wk = df_[df_["Ausgabe"]=="WK"].LW.mean()
        lw_ach = df_[df_["Ausgabe"]=="ACH"].LW.mean()
        lw_nor = df_[df_["Ausgabe"]=="NOR"].LW.mean()
        
        new_ser = pd.Series([se,lw_wk, lw_ach, lw_nor, sortierung], index=cols)
        new_df = new_df.append(new_ser, ignore_index=True)
        
    return new_df
                
            
df_hb = create_pagenumber_wide(df_lokseiten)     

['Seite 2' 'Seite 5' 'Seite 3' 'Seite 4' 'Seite 7' 'Seite 1' 'Seite 6']


In [43]:
df_hb

,Seite,LW WK,LW ACH,LW NOR,Sortierung
0,Seite 2,21.303268,11.829055,16.568090,2
1,Seite 5,22.276535,12.181537,17.565566,5
2,Seite 3,26.456870,15.097940,21.582824,3
3,Seite 4,20.340798,11.125249,15.745342,4
4,Seite 7,15.840680,8.379007,10.632048,7
5,Seite 1,39.850954,25.943915,33.690758,1
6,Seite 6,16.310243,9.953114,11.996758,6


In [48]:
df_lokbre = df_kum[(df_kum["Ressort"]=="Lokales") & (df_kum["Ausgabe"]=="ACH")]

In [97]:
df.Seitentitel.unique()

array(['Lokales', 'Erste Lokalseite', 'Stadtteil-Kurier',
       'Landkreis Osterholz', 'Lemwerder / Berne / Wesermarsch',
       'Lokales / Lokalsport',
       'Lemwerder / Berne / Wesermarsch / Lokalsport', 'Lokalsport',
       'Wirtschaft', 'Niedersachsen', 'Bremen', 'Thema', 'Kinderzeitung',
       'Titelseite', 'Vermischtes', 'Sport', 'Gesundheit',
       'Bildung & Wissen', 'Briefe An Die Redaktion',
       'Immobilien & Wohnen', 'Hin & Weg', 'Dick & Dünn', 'Kultur',
       'Tipps & Termine', 'Politik', 'Regionalsport',
       'Gemeinsam In Bremen', 'Rätsel & Roman', 'Verbraucher (Finanzen)',
       'Stadtteil-Sport', 'Reise', 'Weser Strand', 'Familie & Bande',
       'Digitales', 'Maritime Wirtschaft & Logistik', 'Standpunkte',
       'Verbraucher', 'Beruf & Karriere', 'Fotoreportage Lokales',
       'Übung & Meister', 'Dossier', 'Fotoreportage', 'Weser-Talk'],
      dtype=object)

In [26]:
#df_lok = df_kum[(df_kum["Ressort"]=="Lokales") & (df_kum["Ausgabe"]!="WK") & ]
df_bremen = dfnk[dfnk["Ressort"]=="Bremen"]

## Seitenzahl anfügen

In [115]:
def create_pagenumber(df): 
    ressorts = df.Ressort.unique()
    
    # Fehlercheck
#     if (len(ressorts)>1) | (ressorts[0]!="Lokales"): 
#         raise ValueError("Achtung, multiple Ressorts entdeckt.")
    cols=["Lokalteil", "Seite", "LW", "Anzahl_Artikel"]
    new_df = pd.DataFrame(columns=cols)
    df["Seite_im_Lokalteil"] = 0
    df["Sortierung"] = 0
    all_dates = df.Datum.unique()
    for date in all_dates: 
        df_ = df[df["Datum"]==date]
        
        for aus in df.Ausgabe.unique(): 
            df_aus = df_[df_["Ausgabe"]==aus]
            seite_min = df_aus.Seite.min()
            seite_max = df_aus.Seite.max()
            print(seite_min)
            print(seite_max)
            print(aus, date)
            for c, i in enumerate(range(seite_min, seite_max+1)):
                df.loc[(df["Datum"]==date) & (df["Seite"]==i), "Seite_im_Lokalteil"] = "Seite " + str(c+1)
                df.loc[(df["Datum"]==date) & (df["Seite"]==i), "Sortierung"] = c+1
    
    return df
                
            
df_lokseiten = create_pagenumber(df_lok)     

<ipython-input-115-9b22ba1965c2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Seite_im_Lokalteil"] = 0
<ipython-input-115-9b22ba1965c2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Sortierung"] = 0
c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

25
29
ACH 18.10.2021
25
29
NOR 18.10.2021
29
32
ACH 19.10.2021
29
33
NOR 19.10.2021
29
33
ACH 20.10.2021
29
33
NOR 20.10.2021
25
28
ACH 21.10.2021
25
29
NOR 21.10.2021
29
33
ACH 22.10.2021
29
33
NOR 22.10.2021
47
52
ACH 23.10.2021
47
52
NOR 23.10.2021
29
33
ACH 25.10.2021
29
33
NOR 25.10.2021
29
33
ACH 26.10.2021
29
33
NOR 26.10.2021
29
33
ACH 27.10.2021
29
33
NOR 27.10.2021
29
33
ACH 28.10.2021
29
33
NOR 28.10.2021
29
33
ACH 29.10.2021
29
33
NOR 29.10.2021
51
57
ACH 30.10.2021
51
60
NOR 30.10.2021
33
36
ACH 01.11.2021
33
37
NOR 01.11.2021
33
38
ACH 02.11.2021
33
37
NOR 02.11.2021
29
32
ACH 03.11.2021
29
34
NOR 03.11.2021
29
33
ACH 04.11.2021
29
33
NOR 04.11.2021
29
32
ACH 05.11.2021
29
33
NOR 05.11.2021
51
61
ACH 06.11.2021
51
57
NOR 06.11.2021
25
28
ACH 08.11.2021
25
29
NOR 08.11.2021
29
32
ACH 09.11.2021
29
34
NOR 09.11.2021
29
35
ACH 10.11.2021
29
34
NOR 10.11.2021
33
36
ACH 11.11.2021
33
37
NOR 11.11.2021
25
29
ACH 12.11.2021
25
29
NOR 12.11.2021
53
58
ACH 13.11.2021
53
58
NOR 13.

In [108]:
df_lokseiten[(df_lokseiten["Ausgabe"]=="ACH") & (df_lokseiten["Seite_im_Lokalteil"]=="Seite 2")].LW.mean()

8.563231604730042

In [116]:
df_groupby_seite = df_lokseiten.groupby(["Ausgabe", "Seite_im_Lokalteil"], as_index=False)\
.agg({"SplitId":"count", "LW":"mean", "BW":"mean", "DW":"mean", "Sortierung":"mean"})
df_groupby_seite.loc[df_groupby_seite["Seite_im_Lokalteil"]=="Seite 1", "Seite_im_Lokalteil"] = "Lokale 1"
df_groupby_seite.to_csv("./data/lokales_seiten.csv", decimal=",", index=False)

In [118]:
df_groupby_seite

,Ausgabe,Seite_im_Lokalteil,SplitId,LW,BW,DW,Sortierung
0,ACH,Lokale 1,343,17.655582,23.537837,70.218604,1.0
1,ACH,Seite 11,2,4.699543,10.377358,41.981627,11.0
2,ACH,Seite 2,264,8.563232,12.542075,63.681712,2.0
3,ACH,Seite 3,322,11.983636,16.870889,65.062939,3.0
4,ACH,Seite 4,228,5.264597,9.900819,48.257103,4.0
5,ACH,Seite 5,101,6.934758,12.286809,51.732642,5.0
6,ACH,Seite 6,33,6.341979,11.260632,50.765497,6.0
7,ACH,Seite 7,7,11.206513,16.219488,72.228531,7.0
8,ACH,Seite 8,2,6.055578,13.207547,46.942389,8.0
9,ACH,Seite 9,2,1.525315,6.521739,25.933088,9.0


In [89]:
x = df_lokseiten[df_lokseiten["Seite_im_Lokalteil"]==0][["Ausgabe", "Artikel", "Seite", "Datum"]]

In [90]:
x

,Ausgabe,Artikel,Seite,Datum
0,ACH,Farbspur auf der Fahrbahn,28,18.10.2021
1,ACH,Von Erntemaschine erfasst,28,18.10.2021
2,ACH,Gegenverkehr übersehen,32,19.10.2021
3,ACH,Stromstörung in Bierden,30,19.10.2021
4,ACH,56-Jährige fährt über Verkehrsinsel,32,20.10.2021
...,...,...,...,...
9749,NOR,Kreative Ideen sind gefragt,33,24.11.2021
9754,ACH,Wahrlich kein Eyecatcher,30,03.12.2021
9755,ACH,Erschlagendes Gebäude,30,03.12.2021
9853,NOR,Vegesacker Balkon am falschen Ort,30,17.11.2021


In [64]:
for c, i in enumerate(range(5, 8)): 
    print(c, i)

0 5
1 6
2 7


In [53]:


for day in df_lokach.Datum.unique(): 
    df_ = df_lokach[df_lokach["Datum"]==day]
    seiten = df_.Seite.nunique()
    wochentag = df_.Wochentag.iloc[0]
    print(f"Seitenzahl am {day}: {seiten} ,{wochentag}.")

Seitenzahl am 18.10.2021: 5 ,Montag.
Seitenzahl am 19.10.2021: 4 ,Dienstag.
Seitenzahl am 20.10.2021: 5 ,Mittwoch.
Seitenzahl am 21.10.2021: 4 ,Donnerstag.
Seitenzahl am 22.10.2021: 5 ,Freitag.
Seitenzahl am 23.10.2021: 5 ,Samstag.
Seitenzahl am 25.10.2021: 5 ,Montag.
Seitenzahl am 26.10.2021: 5 ,Dienstag.
Seitenzahl am 27.10.2021: 5 ,Mittwoch.
Seitenzahl am 28.10.2021: 5 ,Donnerstag.
Seitenzahl am 29.10.2021: 5 ,Freitag.
Seitenzahl am 30.10.2021: 7 ,Samstag.
Seitenzahl am 01.11.2021: 4 ,Montag.
Seitenzahl am 02.11.2021: 6 ,Dienstag.
Seitenzahl am 03.11.2021: 4 ,Mittwoch.
Seitenzahl am 04.11.2021: 5 ,Donnerstag.
Seitenzahl am 05.11.2021: 4 ,Freitag.
Seitenzahl am 06.11.2021: 7 ,Samstag.
Seitenzahl am 08.11.2021: 4 ,Montag.
Seitenzahl am 09.11.2021: 4 ,Dienstag.
Seitenzahl am 10.11.2021: 6 ,Mittwoch.
Seitenzahl am 11.11.2021: 4 ,Donnerstag.
Seitenzahl am 12.11.2021: 5 ,Freitag.
Seitenzahl am 13.11.2021: 6 ,Samstag.
Seitenzahl am 15.11.2021: 5 ,Montag.
Seitenzahl am 16.11.2021: 6 ,Dienst

In [ ]:
for day in df

# Artikel nach Position identifizieren

In [10]:
# SQL-Query
#SQL-Abfrage
import pyodbc
import pymssql
server = 'tcp:lw-azuresql-1.database.windows.net'
database = 'LesewertMandantenDB_WKM' # Weserkurier
username = 'Mehrwertmacher'
password = 'fW18ObHgb8qVhlQUsNpx'
driver= '{ODBC Driver 17 for SQL Server}'

cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

query_id = '''
SELECT TeilartikelVeroeffentlichungId as ArtikelId, nk.SplitId,
XPosition, YPosition, Breite, Höhe
FROM TeilartikelZonen tz
JOIN vw_Artikellesewerte_NichtKumuliert_AktiveWellen_Vorberechnet nk
ON nk.ArtikelId = tz.TeilartikelVeroeffentlichungId


'''

df_query_id = pd.read_sql(query_id, cnxn)
cursor.close()
df_query_id = df_query_id.drop_duplicates(subset=["SplitId"])
all_split_ids = df.SplitId.unique()
df_query_id = df_query_id[df_query_id["SplitId"].isin(all_split_ids)]
df_query_id.to_pickle("./data/positionen.pkl")


In [11]:
df_pos = df_query_id.merge()

9895

In [9]:
df_query_id

,ArtikelId,SplitId,XPosition,YPosition,Breite,Höhe
0,32136,14390,801.377008,156.143465,161.007874,713.999055
1,32137,14389,802.527874,1083.849291,159.857008,184.467402
2,32138,14391,46.800000,125.546299,172.913386,306.000000
3,32139,14392,789.471496,564.142520,172.913386,306.000000
4,32140,14393,232.463622,575.316693,544.251969,815.040000
...,...,...,...,...,...,...
24225,59181,26175,788.235111,668.320410,177.000000,356.368272
24226,59182,26176,223.469291,1033.617165,544.251969,346.739528
24227,59183,26178,37.797165,921.342520,172.913386,265.200945
24228,59184,26179,37.800000,115.546299,172.913386,785.400945


# Titelseite

## Vergleich Titelseite / Inhaltsverzeichnis / restliche Artikel

In [4]:
# Nur Inhaltsverzeichis (Sonntag kein IV)
t = df_kum[(df_kum["Ressort"]=="Titelseite") & \
           (df_kum["Platzierungen"]=="TS2") & \
          (df_kum["DF"]!="Hintergrundkasten") & \
          (df_kum["Wochentag"]!="Sonntag")]
#t.sort_values(by="Datum")
t.LW.mean()

8.095090605207377

In [92]:
# Alle anderen Artikel Titelseite Mo-Sa
df_titel_ohne_sonntag = df_kum[(df_kum["Ressort"]=="Titelseite") & (df_kum["Wochentag"]!="Sonntag")]
ohne_t = df_titel_ohne_sonntag[(df_titel_ohne_sonntag["Platzierungen"]!="TS2") | \
                  ((df_titel_ohne_sonntag["Platzierungen"]=="TS2") &(df_titel_ohne_sonntag["DF"]=="Hintergrundkasten"))]

In [93]:
ohne_t.LW.mean()

36.83759902415115

In [ ]:
df

In [86]:
df_titel_nur_sonntag =df_kum[(df_kum["Ressort"]=="Titelseite") & (df_kum["Wochentag"]=="Sonntag")]

In [87]:
df_titel_nur_sonntag.mean()

<ipython-input-87-14b2b7e6788a>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_titel_nur_sonntag.mean()


SplitId                  21555.411765
ArtikelId                48689.941176
ErscheinungsId             301.500000
AusgabeId                    1.000000
Seite                        1.000000
BW                          36.533302
DW                          86.913413
LW                          32.181722
Sortierung                  43.235294
Sortierung_Wochentage        7.000000
dtype: float64

In [94]:
df_titel_ohne_sonntag.mean()

<ipython-input-94-61a3bcc10905>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_titel_ohne_sonntag.mean()


SplitId                  20308.719160
ArtikelId                45848.419948
ErscheinungsId             283.921260
AusgabeId                    1.000000
Seite                        1.000000
BW                          34.022679
DW                          80.723357
LW                          28.991875
Sortierung                  44.304462
Sortierung_Wochentage        3.435696
dtype: float64

In [95]:
df_titel_all = df_kum[df_kum["Ressort"]=="Titelseite"]

In [96]:
df_titel_all.DF.value_counts()

Nachricht            281
Bildnachricht         63
Bericht/Feature       53
Komm./Kolumne          8
Hintergrundkasten      5
Struktur. Artikel      4
Interview              1
Name: DF, dtype: int64

## Korrelation AA 1 + Thema d. T. 

In [22]:
df_titel = df[df["Ressort"] == "Titelseite"]

In [25]:
df_titel[df_titel["Datum"] == "05.06.2021"]

,SplitId,ArtikelId,Artikel,ErscheinungsId,AusgabeId,Datum,Seite,Ausgabeleser,Artikelleser,BW,...,Kumulierung,Wellennummer,Dachzeile,Wochentag,Worte,Seitenreihung,Optik,Ressort_neu,Ressort_neu2,DF
275,19074,60289,DAS WETTER,296,1,05.06.2021,1,181,2,"1,1",...,ja,4,NaN,Samstag,39,1,nein,Politik,Titelseite,Nachricht
280,19354,60877,DAS WETTER,298,3,05.06.2021,1,148,4,"2,7",...,ja,6,NaN,Samstag,35,1,nein,Politik,Titelseite,Nachricht
853,19068,60249,Mission Apollofalter,296,1,05.06.2021,1,127,43,"33,86",...,ja,4,NaN,Samstag,373,1,ja,Politik,Titelseite,Komm./Kolumne
857,19271,60646,Das Streiflicht,297,2,05.06.2021,1,54,32,"59,26",...,ja,5,NaN,Samstag,373,1,nein,Politik,Titelseite,Komm./Kolumne
858,19363,60939,Mission Apollofalter,298,3,05.06.2021,1,148,39,"26,35",...,ja,6,NaN,Samstag,373,1,ja,Politik,Titelseite,Bildnachricht
2117,19361,60927,Sachsen-Anhalt: CDU klar vor AfD,298,3,05.06.2021,1,148,52,"35,14",...,ja,6,NaN,Samstag,107,1,nein,Politik,Titelseite,Nachricht
3004,19062,60242,Merkel hält sich im Wahlkampf zurück,296,1,05.06.2021,1,329,123,"37,39",...,ja,4,NaN,Samstag,44,1,nein,Politik,Titelseite,Nachricht
3005,19120,60431,Sachsen-Anhalt: CDU klar vor AfD,296,1,05.06.2021,1,181,73,"40,33",...,ja,4,NaN,Samstag,132,1,nein,Politik,Titelseite,Nachricht
5114,19335,60733,Am toten Punkt,298,3,05.06.2021,1,148,75,"50,68",...,ja,6,NaN,Samstag,646,1,ja,Politik,Titelseite,Bericht
5432,19069,60252,Am toten Punkt,296,1,05.06.2021,1,181,89,"49,17",...,ja,4,NaN,Samstag,655,1,ja,Politik,Titelseite,Bericht


Dafür bastele ich eine Tabelle, die ich ausspiele und den Coaches mitliefere. 


In [4]:
df_a

,SplitId,ArtikelId,Ueberschrifttext,ErscheinungsId,Erscheinungsdatum,Seitennummer,Ressortname,UnterRessortname,Ausgabeleser (Erscheinung),Artikelleser (Erscheinung),...,Platzierungen,Darstellungsformen,Beschreibung,Handlungsorte,Akteure,Themen,Ausgabename,Wellennummer,offen/codiert,NB
0,1502,4566,Jeder stirbt für sich allein,26,2021-03-27,3,R2389,R2389,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,HF3,1,offen,NaN
1,1502,4566,Jeder stirbt für sich allein,26,2021-03-27,3,R2389,R2389,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,HF3,3,offen,NaN
2,1502,4566,Jeder stirbt für sich allein,26,2021-03-27,3,R2389,R2389,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,HF3,4,offen,NaN
3,1499,4567,"Medien, TV-Programm",26,2021-03-27,1,R2441,R2441,1,1,...,NaN,NaN,NaN,"Bayern, München, Schach 36 Traueranzeigen 20",NaN,TV-Programm,HF3,1,offen,NaN
4,1499,4567,"Medien, TV-Programm",26,2021-03-27,1,R2441,R2441,1,1,...,NaN,NaN,NaN,"Bayern, München, Schach 36 Traueranzeigen 20",NaN,TV-Programm,HF3,3,offen,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62843,19307,61520,Oper im Weltall,300,2021-06-05,82,R2350,R2350,202,9,...,SK,RK,NaN,"Australien, München, Wien","Miroslav Srnka, Nicolas Brieger, Raimund Baue...","Bildende Künste, Gesangskunst, Theater/Oper",1MG_Starnberg,7,codiert,NaN
62844,19309,61521,Im Hoagartn der Hoffnung,300,2021-06-05,82,R2350,R2350,202,6,...,TS1,KM,NaN,München,"Alexander Strauch, Andreas Skouras, Caio de A...","Klassische Musik, Musik, Musik und Audio, Sou...",1MG_Starnberg,7,codiert,NaN
62845,19276,61522,Auf der Suche nach dem Spirit,300,2021-06-05,84,R2350,R2350,202,27,...,AA,RK,NaN,"Berlin, Mannheim, München, Split","Aki Takase, Beate Sampson, Christian Lillinge...","Country- und Western-Musik, Hip-Hop- und Rap-...",1MG_Starnberg,7,codiert,NaN
62846,19298,61523,Kunst auf Rädern,300,2021-06-05,84,R2350,R2350,202,15,...,SK,RK,NaN,"Chía, Italien, Le Mans, München","Alexander Calder, Andy Warhol, BMW, Christian...","Bildende Künste, Digitalkunst, Kunst/Kunsthan...",1MG_Starnberg,7,codiert,NaN


In [5]:
df_auswahl = df_[(df_["Seite"] <= 2) & (df_["Platzierungen"] == "AA")]

In [6]:
cols = ["Datum", "Titelaufmacher", "Themenaufmacher",
        "LW_AA_1", "LW_AA_2", "Ø LW_Seite_2"]
new_df = pd.DataFrame(columns=cols)
for date in df_auswahl.Datum.unique():
    df_date = df_auswahl[df_auswahl.Datum == date]
    aa_1 = df_date[(df_date["Seite"] == 1) & (
        df_date["Platzierungen"] == "AA")]
    print(aa_1.Artikel)

    #data = [date, ]

6259    Rückhalt für Laschet schwindet
6260    Rückhalt für Laschet schwindet
6261    Rückhalt für Laschet schwindet
Name: Artikel, dtype: object
8173    Der Schatten-Kandidat
Name: Artikel, dtype: object
6276    Künftig weniger Steuern für Rentner
6277    Künftig weniger Steuern für Rentner
Name: Artikel, dtype: object
5045    Laschet und Söder im offenen Duell
5046    Laschet und Söder im offenen Duell
5317    Laschet und Söder im offenen Duell
Name: Artikel, dtype: object
5047     Kabinett beschließt Notbremse
5048    Unterstützung für Söder wächst
5318    Unterstützung für Söder wächst
Name: Artikel, dtype: object
5051    Angst in der Union vor Spaltung wächst
5324    Angst in der Union vor Spaltung wächst
Name: Artikel, dtype: object
5053    Impfpriorisierung endet im Juni
5054    Impfpriorisierung endet im Juni
5330    Impfpriorisierung endet im Juni
Name: Artikel, dtype: object
5069    Angst vor Bürgerkrieg in Israel
5351    Angst vor Bürgerkrieg in Israel
Name: Artikel, dtype: 

In [18]:
df_auswahl.columns

Index(['SplitId', 'ArtikelId', 'Artikel', 'ErscheinungsId', 'AusgabeId',
       'Datum', 'Seite', 'Ausgabeleser', 'Artikelleser', 'BW', 'DW', 'LW',
       'Seitentitel', 'Platzierungen', 'Darstellungsformen', 'Beschreibung',
       'Handlungsorte', 'Akteure', 'Themen', 'Ausgabe', 'Layerlink', 'Ressort',
       'Ausgabeteil', 'Kumulierung', 'Wellennummer', 'Dachzeile', 'Wochentag',
       'Worte', 'Seitenreihung', 'Optik', 'Ressort_neu', 'Ressort_neu2', 'DF'],
      dtype='object')

In [16]:
t = df_auswahl[df_auswahl["Platzierungen"] == "SK"]
t

,SplitId,ArtikelId,Artikel,ErscheinungsId,AusgabeId,Datum,Seite,Ausgabeleser,Artikelleser,BW,...,Kumulierung,Wellennummer,Dachzeile,Wochentag,Worte,Seitenreihung,Optik,Ressort_neu,Ressort_neu2,DF
1863,8427,26205,Die Vertrauensfrage,133,3,23.04.2021,2,186,59,"31,72",...,ja,6,NaN,Freitag,768,1,ja,Themen des Tages,Themen des Tages,Struktur. Artikel
2134,7246,22908,Schwesternliebe,116,1,20.04.2021,2,389,87,"22,37",...,ja,4,NaN,Dienstag,816,1,ja,Themen des Tages,Themen des Tages,Struktur. Artikel
2142,8255,25897,Die Vertrauensfrage,131,1,23.04.2021,2,246,81,"32,93",...,ja,4,NaN,Freitag,767,1,ja,Themen des Tages,Themen des Tages,Struktur. Artikel
2172,11549,36383,Nicht ohne mein Kind,181,1,06.05.2021,2,386,92,"23,83",...,ja,4,NaN,Donnerstag,773,1,ja,Themen des Tages,Themen des Tages,Struktur. Artikel
2207,16829,53140,Tröpfchen für Tröpfchen,261,1,27.05.2021,2,337,113,"33,53",...,ja,4,NaN,Donnerstag,771,1,ja,Themen des Tages,Themen des Tages,Struktur. Artikel
2222,18465,58314,"Träumchen, wechsle dich",286,1,02.06.2021,2,312,82,"26,28",...,ja,4,NaN,Mittwoch,937,1,nein,Themen des Tages,Themen des Tages,Struktur. Artikel
3036,9282,29282,"Geimpft, genesen, getestet – und dann?",146,1,27.04.2021,2,400,172,"43,0",...,ja,4,NaN,Dienstag,741,1,ja,Themen des Tages,Themen des Tages,Struktur. Artikel
5948,16366,51455,"Wen man bestraft, und wen man trifft",253,3,25.05.2021,2,171,64,"37,43",...,ja,6,NaN,Dienstag,709,1,ja,Themen des Tages,Themen des Tages,Analyse
5960,5340,16930,Störmeldungen aus München,86,1,13.04.2021,2,389,162,"41,65",...,ja,4,NaN,Dienstag,756,1,ja,Themen des Tages,Themen des Tages,Analyse
5965,5633,17856,Finale zwischen den Fronten,91,1,14.04.2021,2,384,174,"45,31",...,ja,4,NaN,Mittwoch,695,1,ja,Themen des Tages,Themen des Tages,Analyse


In [24]:
tt = df_auswahl[df_auswahl["Datum"] == "20.04.2021"]
tt

,SplitId,ArtikelId,Artikel,ErscheinungsId,AusgabeId,Datum,Seite,Ausgabeleser,Artikelleser,BW,...,Kumulierung,Wellennummer,Dachzeile,Wochentag,Worte,Seitenreihung,Optik,Ressort_neu,Ressort_neu2,DF
5326,7252,22914,Baerbock will Kanzlerin werden,116,1,20.04.2021,1,389,199,"51,16",...,ja,4,NaN,Dienstag,413,1,ja,Politik,Titelseite,Bericht
8580,7250,22912,Ganz ohne Groll,116,1,20.04.2021,2,138,53,"38,41",...,ja,4,Die Union und die K-Frage So geschlossen sich...,Dienstag,1741,1,ja,Themen des Tages,Themen des Tages,Analyse
9784,7369,23035,Ganz ohne Groll,117,2,20.04.2021,2,63,29,"46,03",...,ja,5,Die Union und die K-Frage So geschlossen sich...,Dienstag,1484,1,ja,Themen des Tages,Themen des Tages,Essay
9785,7434,23218,Ganz ohne Groll,118,3,20.04.2021,2,188,62,"32,98",...,ja,6,Die Union und die K-Frage So geschlossen sich...,Dienstag,1728,1,ja,Themen des Tages,Themen des Tages,Essay


## Rubriken Wetter, Streiflicht, Kasten, Meldungen

In [18]:
df_titel = df_merge[df_merge["Seitennummer"] == 1]
df_titel.shape

(3833, 30)

In [20]:
df_titel.to_clipboard(index=False, decimal=",")

## Anreißer auf der Titel 1

HEUTE-Kästen gibt es nur wochentags, am Samstag und vor Feiertagen fällt er weg.

Kästen kumulieren häufig nicht. 

In [7]:
#df_artikel_nb.to_clipboard(index=False, decimal=",")
#df_content.to_clipboard(index=False, decimal=",")

### Infoboxen HEUTE herausfiltern

In [47]:
# Infoboxen filtern

# Teaserboxen tragen Überschrift: HEUTE (gibt es nur an Wochentagen)
df_teaserbox = df_artikel_nb[(df_artikel_nb["Ueberschrifttext"] == "HEUTE") & (
    df_artikel_nb["Seitennummer"] == 1)]
# doppelte Split-IDs filtern
df_teaserbox = df_teaserbox.drop_duplicates(subset="SplitId")
# Artikel kumulieren schlecht, nur einen HEUTE-Kasten pro Tag behalten
df_teaserbox = df_teaserbox.drop_duplicates(subset="Erscheinungsdatum")

### Themen, Seitenzahlen, Ressort pro HEUTE herausfiltern - dazu Asugabe, eventuell sind es ja andere Seitenzahlen

In [147]:
# Artikel-IDs Liste erstellen
liste_artikel_ids = df_teaserbox.SplitId.unique()
# Artikel heraussuchen und Volltext ausspielen
df_HEUTE_volltext = df_content[df_content["TeilartikelId"].isin(
    liste_artikel_ids)]
df_HEUTE = df_HEUTE_volltext[["TeilartikelId", "Artikeltext"]]

Jetzt muss ich den kompletten String bei ("\r\n") splitten - das teilt den String jeweils beim Zeilenumbruch. 

Danach suche ich jedes Mal nach einem Ressort. 

Wenn ich eins finde, suche ich den nächsten Eintrag, der nur aus einer Zahl (Seitenzahl besteht). 

Die Strings dazwischen muss ich wieder verketten, in Ihnen steht Überschrift bzw. Teasertext. 

In [246]:
df_content

,Id,TeilartikelId,Dachzeilentext,Ueberschrifttext,Untertiteltext,Artikeltext,ArtikeltextHtml,Vorspanntext,Unterschrifttext,Bildunterschrifttext,Importdatum,VersionId,RowVersion,Wortanzahl
0,1,1,NaN,Gewinnzahlen,NaN,Gewinnzahlen vom Wochenende\r\nLotto (20.03.):...,NaN,NaN,NaN,NaN,2021-03-22 13:59:51.917,NaN,b'\x00\x00\x00\x00\x00\x00\xbf\x12',NaN
1,2,3,NaN,Glühende Bäche,NaN,Ein Hubschrauber überfliegt am Samstag den Fag...,NaN,NaN,NaN,NaN,2021-03-22 14:04:29.337,NaN,b'\x00\x00\x00\x00\x00\x00\xcfA',NaN
2,3,4,NaN,Türkei schwächt den Schutz von Frauen,Präsident Erdoğan tritt aus der Istanbul-Konve...,Istanbul – Mit dem Austritt aus der Istanbul-K...,NaN,NaN,NaN,NaN,2021-03-22 14:04:30.353,NaN,b'\x00\x00\x00\x00\x00\x00\xcfX',NaN
3,4,2,NaN,HEUTE,NaN,Meinung\r\nDie Bundesregierung verpasst\r\nein...,NaN,NaN,NaN,Foto: Hector Vivas/Getty,2021-03-22 14:04:24.517,NaN,b'\x00\x00\x00\x00\x00\x00\xce\xaf',NaN
4,5,5,NaN,SZ App,NaN,Die SZ gibt es als App\r\nfür Tablet und Smart...,NaN,NaN,NaN,NaN,2021-03-22 14:04:25.620,NaN,b'\x00\x00\x00\x00\x00\x00\xce\xd2',NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19653,19683,19668,NaN,ohne_Titel,NaN,Leserbriefe stellen keine redaktionelle Meinun...,NaN,NaN,NaN,NaN,2021-06-07 03:04:08.390,NaN,"b""\x00\x00\x00\x00\x00\xdcW'""",NaN
19654,19684,19669,NaN,Motorradfahrer von Auto erfasst,NaN,Gauting – Ein 29-jähriger Motorradfahrer aus M...,NaN,NaN,NaN,NaN,2021-06-07 03:04:08.423,NaN,b'\x00\x00\x00\x00\x00\xdcW*',NaN
19655,19685,19670,NaN,Mörderisches Puzzle,Die Gautinger Autorin Anna Schneider ist mit d...,Gauting – Alexa Jahn hat ihren Dienst in der K...,NaN,NaN,NaN,"Der zweite Fall ist schon fertig, der dritte i...",2021-06-07 03:04:08.487,NaN,"b'\x00\x00\x00\x00\x00\xdcW,'",NaN
19656,19686,19671,NaN,LESERBRIEFE,NaN,NaN,NaN,NaN,NaN,NaN,2021-06-07 03:04:08.590,NaN,b'\x00\x00\x00\x00\x00\xdcW8',NaN


In [167]:
# Liste mit erwünschten Ressorts erstellen
liste_ressorts = list(df.Ressort.unique())
liste_zusätzliche_ressorts = [
    "Thema des Tages", "Die Seite Drei", "Meinung", "Das Politische Buch", "Medien"]
liste_ressorts.extend(liste_zusätzliche_ressorts)

# diverse Ressorts, die regelmäßig ohne Text angeteasert werden, rausnehmen
liste_ressorts_ohne_berücksichtigung = ["Medien, TV-/Radioprogramm", "Forum & Leserbriefe", "München · Bayern",
                                        "Traueranzeigen", "Rätsel & Schach", "Rätsel"]
liste_ressorts = [
    res for res in liste_ressorts if res not in liste_ressorts_ohne_berücksichtigung]

In [168]:
cols = ["SplitId", "Teaser", "Ressort", "Seite", "Teasertext"]
new_df = pd.DataFrame(columns=cols)
for row in df_HEUTE.itertuples():
    teaser = "HEUTE"
    text = row.Artikeltext
    artikelId = row.TeilartikelId
    # String in Artikeltext aufspalten
    liste_text = text.split("\r\n")
    # print(liste_text)
    # print(len(liste_text))

    for i in range(len(liste_text)):

        split_string = liste_text[i]
        if split_string in liste_ressorts:
            ressort = split_string
            print(split_string)
            j = i+1
            seite = "0"
            # Suche nach der nächsten Zahl
            for k in range(j, len(liste_text)):
                # if re.match("(?<![A-Za-z])[0-9]+", liste_text[k]):
                if liste_text[k].isdigit():
                    print(liste_text[k])
                    seite = liste_text[k]
                    print(k, j)
                    teaser_text = ""
                    for l in range(j, k, 1):
                        teaser_text = teaser_text + " " + liste_text[l]
                    print(teaser_text.strip())
                    break

            new_ser = pd.Series(
                [artikelId, teaser, ressort, seite, teaser_text.strip()], index=cols)
            new_df = new_df.append(new_ser, ignore_index=True)

Meinung
4
4 1
Russlands Aufmarsch an der Grenze zur Ukraine ist mehr als eine Drohgebärde
Politik
7
9 6
Krieg gegen das eigene Volk: Das Militär in Myanmar geht immer brutaler vor
Feuilleton
9
13 11
Hereinspaziert – trotz Pandemie will die Architekturbiennale in Venedig im Mai eröffnen
Das Politische Buch
12
18 15
Alec MacGillis analysiert Macht und Machenschaften von Amazon
Sport
23
23 20
Eskalation im Streit von Manager Salihamidzic und Trainer Flick beim FC Bayern
Schule und Hochschule
13
27 27

Bayern
22
29 29

Meinung
4
3 1
Nach der Wahl ist Bulgarien politisch noch instabiler. Trotzdem zeichnet sich ein positiver Trend ab
Politik
6
7 5
Die Parteitagsbeschlüsse zeigen, wie die AfD in vielen Punkten weiter nach rechts abdriftet
Panorama
10
12 9
Die Grünen fordern ein Verbot der Trophäenjagd. Aus Afrika kommt ein Protestbrief
Feuilleton
11
15 14
Hat der Berliner Künstler Sebastian Bieniek einen Verschwörungskult verbreitet?
Wissen
15
20 17
Lebensretter im Müll: Wie täglich wertvolle

Politik
7
8 6
Ein Jahr nach dem Tod von George Floyd: Ein US-Bürgermeister will seine Polizei auflösen
Feuilleton
11
13 10
Whisky, Cola, Bass und Liebe: Das neue Album des Sängers und Rappers Jan Delay
Das Politische Buch
14
18 15
Robin Alexander schildert das Ende der Ära Merkel als Politthriller
Sport
25
23 20
Nicht mehr zu retten: Werder Bremen steht vor einer ungewissen Zukunft
Meinung
4
4 1
Ein Jahr danach: Was sich in den USA durch den Tod von George Floyd verändert hat
Panorama
8
8 6
In Finnland sucht ein Gericht nach einem Mörder, der 34 Jahre lang unentdeckt bleiben konnte
Feuilleton
9
12 10
Eine Spätgotik-Ausstellung in Berlin zeigt, wie gegenwärtig die Kunst des 15. Jahrhunderts ist
Wissen
14
17 14
In neuem Licht: Eine Gentherapie lässt Blinde wieder sehen
Wirtschaft
16
22 19
Wer bist du? Persönlichkeitstests im Beruf sind beliebt, aber oft auch unzuverlässig
Die Seite Drei
3
3 1
Unser Bester: Der Schützenpanzer „Puma“ ist so gut, dass man ihn kaum brauchen kann
Meinung
4
8 

In [169]:
# Test nach Vollständigkeit
# in der Regel hat jeder Tag fünf beteaserte Ressorts
for splitid in new_df["SplitId"].unique():
    df_ = new_df[new_df["SplitId"] == splitid]
    if len(df_) < 5:
        print(splitid)

In [170]:
new_df[new_df["SplitId"] == 8236]

,SplitId,Teaser,Ressort,Seite,Teasertext
48,8236,HEUTE,Thema des Tages,2,Astra Zeneca für alle: Wird die Impfpriorisier...
49,8236,HEUTE,Meinung,4,Die Städte müssen grüner werden – das hilft Me...
50,8236,HEUTE,Politik,7,Die Corona-Lage in Brasilien ist dramatisch. A...
51,8236,HEUTE,Wirtschaft,18,Aus einem kleinen Laden wurde mit Billigkleidu...
52,8236,HEUTE,Sport,25,Woran die Fußball-Revolution namens Super Leag...


In [73]:
t = df_HEUTE_volltext.iloc[0]

In [77]:
li = t.Artikeltext.split("\r\n")

### Titelbild-Teaser herausfiltern

In [201]:
df_titel = df_artikel_nb[df_artikel_nb["Seitennummer"] == 1]
df_optik = df_titel[(df_titel["Platzierungen"] == "ÜA")]

In [202]:
df_optik.drop_duplicates(subset="SplitId", inplace=True)
df_optik.drop_duplicates(subset="Erscheinungsdatum", inplace=True)

<ipython-input-202-da413590a913>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_optik.drop_duplicates(subset="SplitId", inplace=True)
<ipython-input-202-da413590a913>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_optik.drop_duplicates(subset="Erscheinungsdatum", inplace=True)


In [205]:
liste_ids_titelbild = df_optik.SplitId.unique()

In [232]:
cols = ["SplitId", "Teaser", "Ressort", "Seite", "Teasertext"]
new_df2 = pd.DataFrame(columns=cols)

df_optik_kom = df_content[df_content.TeilartikelId.isin(liste_ids_titelbild)]
for row in df_optik_kom.itertuples():
    teaser = "Titelbild"

    artikeltext = row.Artikeltext
    split_text = artikeltext.split("\r\n►")
    ressort = split_text[-1]
    restlicher_text = split_text[0].split("Foto:")[0].strip()
    print(ressort.strip())
    print(restlicher_text)
    print(len(restlicher_text))
    new_ser = pd.Series([row.TeilartikelId, "Titelbild",
                        ressort, "k.A.", restlicher_text], index=cols)

    new_df2 = new_df2.append(new_ser, ignore_index=True)

Meinung, Panorama
Eine digitale Abschiedsbotschaft für Prinz Philip am Piccadilly Circus in London. Der Ehemann von Königin Elizabeth II. war am Freitag im Alter von 99 Jahren verstorben. Die Glocke von Westminster Abbey läutete 99 Mal, überall im Vereinigten Königreich hallten Salutschüsse. Zum Begräbnis auf dem Gelände von Windsor Castle am kommenden Samstag soll der Sarg in einem eigens gestalteten Land Rover gefahren werden – so, wie Prinz Philip es sich gewünscht hat.
459
Feuilleton
Vor zwei Jahren brannte
die Kathedrale von Notre-Dame. Jetzt fallen Bäume für
den Wiederaufbau – und in Paris herrscht Denkmal-Euphorie
137
SZ Magazin
Die Schauspielerin Nora
Tschirner war depressiv.
Im Interview erzählt sie, was
sie krank gemacht hat –
und warum das auch mit der
Kunstbranche zu tun hat
158
Seiten 4 und 5
Fast 80 000 Menschen sind in Deutschland bislang an oder mit Covid-19 gestorben. Ihnen zum Gedenken hat Bundespräsident Frank-Walter Steinmeier am Sonntag Bundeskanzlerin Angela Merke

In [234]:
all_df = new_df.append(new_df2)

In [240]:
all_df

,SplitId,Teaser,Ressort,Seite,Teasertext
0,4996,HEUTE,Meinung,4,Russlands Aufmarsch an der Grenze zur Ukraine ...
1,4996,HEUTE,Politik,7,Krieg gegen das eigene Volk: Das Militär in My...
2,4996,HEUTE,Feuilleton,9,Hereinspaziert – trotz Pandemie will die Archi...
3,4996,HEUTE,Das Politische Buch,12,Alec MacGillis analysiert Macht und Machenscha...
4,4996,HEUTE,Sport,23,Eskalation im Streit von Manager Salihamidzic ...
...,...,...,...,...,...
17,16845,Titelbild,Feuilleton,k.A.,Über ein Jahr lang waren die Kinos\r\nmehr ges...
18,17200,Titelbild,SZ Magazin,k.A.,Joshua Kimmich ist\r\neiner der besten\r\nFußb...
19,17894,Titelbild,Die Seite Drei,k.A.,Im Ibiza-Ausschuss\r\nkann man die zunehmende\...
20,18204,Titelbild,Wissen,k.A.,Island will das Kohlendioxid\r\nanderer Staate...


### Volltext-Daten mit Artikeldaten mergen

In [267]:
df_m = pd.merge(all_df, df_artikel_nb[[
                "SplitId", "Erscheinungsdatum", "AusgabeId", "ErscheinungsId"]], on="SplitId")

In [268]:
df_m.drop_duplicates(subset="Teasertext", inplace=True)

In [269]:
df_m.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 0 to 440
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   SplitId            192 non-null    object
 1   Teaser             192 non-null    object
 2   Ressort            192 non-null    object
 3   Seite              192 non-null    object
 4   Teasertext         192 non-null    object
 5   Erscheinungsdatum  192 non-null    object
 6   AusgabeId          192 non-null    int64 
 7   ErscheinungsId     192 non-null    int64 
dtypes: int64(2), object(6)
memory usage: 13.5+ KB


In [282]:
df_m["AusgabeId"] = df_m["AusgabeId"].astype(str)
df_m["ErscheinungsId"] = df_m["ErscheinungsId"].astype(str)

In [284]:
df_m["url"] = "https://mein.lesewert.de/1020/Coaching/Page/Detail?&Id=1&Parent.Id=" + \
    df_m['ErscheinungsId']+"&Parent.Parent.Id="+df_m['AusgabeId']

In [286]:
df_m.url

0      https://mein.lesewert.de/1020/Coaching/Page/De...
2      https://mein.lesewert.de/1020/Coaching/Page/De...
4      https://mein.lesewert.de/1020/Coaching/Page/De...
6      https://mein.lesewert.de/1020/Coaching/Page/De...
8      https://mein.lesewert.de/1020/Coaching/Page/De...
                             ...                        
420    https://mein.lesewert.de/1020/Coaching/Page/De...
425    https://mein.lesewert.de/1020/Coaching/Page/De...
430    https://mein.lesewert.de/1020/Coaching/Page/De...
435    https://mein.lesewert.de/1020/Coaching/Page/De...
440    https://mein.lesewert.de/1020/Coaching/Page/De...
Name: url, Length: 192, dtype: object

In [287]:
df_m.to_clipboard()

In [258]:
df_m.to_excel("Übersicht_Teasertexte.xlsx", index=False)

In [ ]:
https: // mein.lesewert.de/1020/Coaching/Page/Detail?& Id = 1 & Parent.Id = 126 & Parent.Parent.Id = 1

## Feuilleton - aktueller Aufmacher

In [11]:
df_f = df[df.Ressort == "Feuilleton"]

In [13]:
df_f[df_f["Datum"] == "16.04.2021"]

,SplitId,ArtikelId,Artikel,ErscheinungsId,AusgabeId,Datum,Seite,Ausgabeleser,Artikelleser,BW,...,Kumulierung,Wellennummer,Dachzeile,Wochentag,Worte,Seitenreihung,Optik,Ressort_neu,Ressort_neu2,DF
556,6372,19837,Antrag für Öffnungen scheitert vor Gericht,101,1,16.04.2021,12,438,27,"6,16",...,ja,4,NaN,Freitag,134,2,nein,Feuilleton,Feuilleton,Nachricht
2453,6391,19833,"Gehen Sie weiter, gibt nichts zu sehen",101,1,16.04.2021,23,438,75,"17,12",...,ja,4,NaN,Freitag,882,4,nein,Medien,Medien,Gastartikel
2484,6360,19788,Vier Wände und ein Todesfall,101,1,16.04.2021,11,438,120,"27,4",...,ja,4,NaN,Freitag,785,1,nein,Feuilleton,Feuilleton,Komm./Kolumne
2558,6376,19808,Damals schuf niemand solche Städte,101,1,16.04.2021,13,438,118,"26,94",...,ja,4,NaN,Freitag,1109,3,ja,Feuilleton,Feuilleton,Rezension/Kritik
2662,6352,19780,Künstler der großen Mythen,101,1,16.04.2021,12,438,64,"14,61",...,ja,4,NaN,Freitag,987,2,ja,Feuilleton,Feuilleton,Rezension/Kritik
2663,6370,19824,Ferien von der Gewalt,101,1,16.04.2021,12,438,49,"11,19",...,ja,4,NaN,Freitag,489,2,ja,Feuilleton,Feuilleton,Rezension/Kritik
2761,6358,19796,Verlage haften nicht für Gesundheitstipps,101,1,16.04.2021,23,438,27,"6,16",...,ja,4,NaN,Freitag,186,4,nein,Medien,Medien,Nachricht
3196,6383,19843,Roboter,101,1,16.04.2021,11,438,48,"10,96",...,ja,4,NaN,Freitag,140,1,nein,Feuilleton,Feuilleton,Zitattext/Umfrage
3197,6398,19848,Doppelt schwer,101,1,16.04.2021,23,438,20,"4,57",...,ja,4,NaN,Freitag,201,4,nein,Medien,Medien,Rezension/Kritik
4188,6366,19809,Schweigende Unterstützer,101,1,16.04.2021,11,438,82,"18,72",...,ja,4,NaN,Freitag,486,1,ja,Feuilleton,Feuilleton,Bericht


In [17]:
all_dates = df_f.Datum.unique()
liste_all_aa = []
for date in all_dates:
    df_ = df_f[df_f["Datum"] == date]
    min_seite = df_.Seite.min()
    df_min = df_[df_["Seite"] == min_seite]
    liste_aa = df_min[df_min["Platzierungen"] == "AA"].SplitId.unique()

    liste_all_aa.extend(liste_aa)
liste_all_aa

[11331,
 14893,
 6360,
 7909,
 10280,
 14613,
 15554,
 9020,
 12624,
 5160,
 5019,
 5745,
 6019,
 7387,
 7327,
 9638,
 10677,
 13900,
 14245,
 17822,
 18810,
 19109,
 10980,
 6777,
 6811,
 7619,
 8354,
 8653,
 9359,
 9896,
 11592,
 11943,
 11953,
 12280,
 15230,
 15869,
 15868,
 7035,
 12982,
 12983,
 13070,
 13256,
 13807,
 16196,
 17203,
 18220,
 18480,
 5374,
 16584,
 16909,
 17574]

In [18]:
df_aa = df[df["SplitId"].isin(liste_all_aa)]
df_aa

,SplitId,ArtikelId,Artikel,ErscheinungsId,AusgabeId,Datum,Seite,Ausgabeleser,Artikelleser,BW,...,Kumulierung,Wellennummer,Dachzeile,Wochentag,Worte,Seitenreihung,Optik,Ressort_neu,Ressort_neu2,DF
1884,5160,16117,Geteiltes Leid,82,2,12.04.2021,9,63,34,"53,97",...,ja,5,NaN,Montag,1237,1,ja,Feuilleton,Feuilleton,Komm./Kolumne
1909,7387,23124,Salatköpfe gegen Betonpfeiler,117,2,20.04.2021,9,63,29,"46,03",...,ja,5,NaN,Dienstag,1214,1,ja,Feuilleton,Feuilleton,Komm./Kolumne
2476,5019,15928,Geteiltes Leid,81,1,12.04.2021,9,318,131,"41,19",...,ja,4,NaN,Montag,1237,1,ja,Feuilleton,Feuilleton,Komm./Kolumne
2480,5745,17970,Zur Kultur der Gesundheit,91,1,14.04.2021,9,384,128,"33,33",...,ja,4,NaN,Mittwoch,1151,1,nein,Feuilleton,Feuilleton,Komm./Kolumne
2481,6019,18785,Deutscher Frühling,96,1,15.04.2021,9,381,88,"23,1",...,ja,4,NaN,Donnerstag,1219,1,nein,Feuilleton,Feuilleton,Komm./Kolumne
2484,6360,19788,Vier Wände und ein Todesfall,101,1,16.04.2021,11,438,120,"27,4",...,ja,4,NaN,Freitag,785,1,nein,Feuilleton,Feuilleton,Komm./Kolumne
2487,7035,22063,"Wer, wenn nicht wir?",111,1,19.04.2021,9,398,87,"21,86",...,ja,4,NaN,Montag,1094,1,ja,Feuilleton,Feuilleton,Komm./Kolumne
2489,7327,22989,Salatköpfe gegen Windmühlen,116,1,20.04.2021,9,326,98,"30,06",...,ja,4,NaN,Dienstag,1214,1,ja,Feuilleton,Feuilleton,Komm./Kolumne
2492,7619,23825,Mitten ins Herz,121,1,21.04.2021,11,403,134,"33,25",...,ja,4,NaN,Mittwoch,1096,1,ja,Feuilleton,Feuilleton,Komm./Kolumne
2495,7909,24796,Weiße Antike,126,1,22.04.2021,9,405,124,"30,62",...,ja,4,NaN,Donnerstag,960,1,nein,Feuilleton,Feuilleton,Komm./Kolumne


In [19]:
df_aa.shape

(51, 33)

In [24]:
df_aa["LW"] = df_aa["LW"].replace(",", ".").astype(float)

ValueError: could not convert string to float: '43,44'

In [25]:
df_aa["LW"] = df_aa["LW"].str.replace(",", ".")

<ipython-input-25-bf0c785b485c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aa["LW"] = df_aa["LW"].str.replace(",", ".")


In [27]:
df_aa["LW"] = df_aa["LW"].astype(float)

<ipython-input-27-d8272858804e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aa["LW"] = df_aa["LW"].astype(float)


In [28]:
df_aa.LW.mean()

22.182352941176465

In [29]:
df_aa.Artikel.nunique()

48

## Wirtschaft Börsenseiten

In [92]:
df_w = df[(df["Ressort"] == "Wirtschaft") & (df["Seitentitel"] != "Report")
          & (df["Seitentitel"] != "Wirtschaft & Technik")]
# Wirtschaft-Report rausnehmen, das ist manchmal die letzte Seite, z.B. an Samstagen
df_w.Seitentitel.value_counts()

Wirtschaft    1055
Name: Seitentitel, dtype: int64

In [93]:
df_w["wt"] = pd.to_datetime(df_w["Datum"]
                            )
df_w["wt"] = df_w["wt"].dt.strftime("%A")

df_w = df_w[df_w["Wochentag"] != "Montag"]

<ipython-input-93-3de1450bfcb0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w["wt"] = pd.to_datetime(df_w["Datum"]
<ipython-input-93-3de1450bfcb0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w["wt"] = df_w["wt"].dt.strftime("%A")


In [94]:
df_w.shape

(954, 34)

In [95]:
cols = ["Datum", "Seiten", "Liste_SplitIds"]
new_df = pd.DataFrame(columns=cols)
all_dates = df_w.Datum.unique()
for date in all_dates:
    df_ = df_w[df_w["Datum"] == date]
    min_seite = df_.Seite.min()
    max_seite = df_.Seite.max()
    abseite = max_seite - 1
    ersatzseite = abseite

    if date == "21.04.2021":
        max_seite = 24
        abseite = 21
    elif date == "28.04.2021":
        max_seite = 20
        abseite = 18
    elif date == "27.04.2021":
        max_seite = 29
        abseite = 34
    elif date == "24.04.2021":
        max_seite = 28
        ersatzseite = 29
        abseite = 34
    elif date == "02.06.2021":
        max_seite = 25
        ersatzseite = 24
        abseite = 23
    elif date == "28.05.2021":
        max_seite = 21
        abseite = 20
    elif date == "08.05.2021":
        max_seite = 27
        ersatzseite = 28
        abseite = 29
    elif date == "04.06.2021":
        max_seite = 20
        abseite = 21
    elif date == "21.04.2021":
        max_seite = 21
        abseite = 24
    elif date == "17.04.2021":
        max_seite = 34
        abseite = 29

    df_ = df_w[df_w["Datum"] == date]
    min_seite = df_.Seite.min()
    max_seite = df_.Seite.max()
    abseite = max_seite - 1
    ersatzseite = abseite
    print(min_seite)
    print(abseite)
    print(max_seite)
    df_letzten_beiden = df_[(df_["Seite"] == abseite) | (df_["Seite"] == max_seite) |
                            (df_["Seite"] == ersatzseite)]
    number_articles = df_letzten_beiden.Artikel.nunique()
    liste_seiten = [abseite, max_seite]
    liste_split = df_letzten_beiden.SplitId.unique()
    new_ser = pd.Series([date, liste_seiten, liste_split], index=cols)
    new_df = new_df.append(new_ser, ignore_index=True)

15
20
21
21
28
29
21
28
29
15
20
21
17
24
25
15
20
21
21
33
34
17
23
24
15
20
21
17
22
23
15
20
21
15
21
22
17
22
23
15
20
21
15
20
21
21
28
29
15
20
21
15
20
21
17
22
23
17
21
22
13
18
19
15
28
29
17
24
25
15
21
22
17
22
23
15
22
23
17
29
30
17
24
25
17
22
23
15
20
21
21
28
29
21
33
34
15
21
22
23
30
31
23
31
32
15
20
21
23
32
33


In [100]:
# alle SplitIds in Liste packen
list_all_ids = []
for row in new_df.itertuples():
    list_all_ids.extend(row.Liste_SplitIds)

In [101]:
list_all_ids

[7310,
 7300,
 7314,
 7431,
 7462,
 15945,
 15925,
 16031,
 16051,
 15922,
 17611,
 17732,
 17599,
 17683,
 17721,
 17723,
 11280,
 11279,
 11285,
 11427,
 11441,
 15536,
 15527,
 15539,
 15667,
 15669,
 5701,
 5669,
 5673,
 5833,
 5866,
 6858,
 6583,
 7600,
 7746,
 10973,
 10961,
 10967,
 11112,
 11124,
 11577,
 11575,
 11769,
 11770,
 11573,
 13653,
 13591,
 13584,
 13588,
 13707,
 13723,
 18827,
 18824,
 18829,
 19032,
 18819,
 18825,
 5545,
 5368,
 5407,
 5364,
 5544,
 6506,
 6333,
 6509,
 6340,
 6334,
 9446,
 9336,
 9345,
 9467,
 12360,
 12469,
 12350,
 14556,
 14561,
 14722,
 14724,
 14552,
 17321,
 17220,
 17227,
 17221,
 17369,
 17371,
 18166,
 18152,
 18160,
 6225,
 5998,
 6215,
 5991,
 5997,
 7877,
 8116,
 8115,
 7878,
 7883,
 11985,
 11981,
 12095,
 12096,
 11984,
 18477,
 18676,
 9614,
 9606,
 9610,
 9611,
 9607,
 9612,
 9605,
 16887,
 17059,
 17062,
 16884,
 16889,
 9849,
 10122,
 10128,
 9851,
 9853,
 8357,
 8359,
 8429,
 8432,
 8356,
 12914,
 13048,
 13050,
 12906,
 1292

In [102]:
df_wi = df[df["Ressort"] == "Wirtschaft"]

In [103]:
df_wi["Börse"] = "nein"

<ipython-input-103-f5052967691d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_wi["Börse"] = "nein"


In [108]:
df_wi.loc[df_wi["SplitId"].isin(list_all_ids), "Börse"] = "ja"

c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [109]:
df_wi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1088 entries, 207 to 11313
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   SplitId             1088 non-null   int64 
 1   ArtikelId           1088 non-null   int64 
 2   Artikel             1088 non-null   object
 3   ErscheinungsId      1088 non-null   int64 
 4   AusgabeId           1088 non-null   int64 
 5   Datum               1088 non-null   object
 6   Seite               1088 non-null   int64 
 7   Ausgabeleser        1088 non-null   int64 
 8   Artikelleser        1088 non-null   int64 
 9   BW                  1088 non-null   object
 10  DW                  1088 non-null   object
 11  LW                  1088 non-null   object
 12  Seitentitel         1088 non-null   object
 13  Platzierungen       1088 non-null   object
 14  Darstellungsformen  1088 non-null   object
 15  Beschreibung        642 non-null    object
 16  Handlungsorte       9

In [110]:
df_wi.to_csv("börse.csv", index=False, decimal=",")

In [111]:
df_wi.to_clipboard(index=False, decimal=",")

In [87]:
# Fehler rauswerfen
for i in new_df.index:
    datum = df_w.at[i, "Datum"]
    if datum == "21.04"

20.04.2021
22.05.2021
29.05.2021
05.05.2021
21.05.2021
14.04.2021
14.04.2021
14.04.2021
17.04.2021
20.04.2021
20.04.2021
21.04.2021
21.04.2021
04.05.2021
06.05.2021
14.05.2021
14.05.2021
04.06.2021
13.04.2021
14.04.2021
14.04.2021
16.04.2021
27.04.2021
04.05.2021
08.05.2021
14.05.2021
18.05.2021
18.05.2021
21.05.2021
28.05.2021
28.05.2021
01.06.2021
04.06.2021
16.04.2021
14.04.2021
15.04.2021
16.04.2021
22.04.2021
07.05.2021
29.05.2021
02.06.2021
28.04.2021
27.05.2021
27.05.2021
29.04.2021
28.05.2021
14.04.2021
23.04.2021
27.04.2021
28.04.2021
29.04.2021
05.05.2021
07.05.2021
07.05.2021
11.05.2021
21.05.2021
25.05.2021
26.05.2021
26.05.2021
02.06.2021
05.06.2021
20.04.2021
24.04.2021
11.05.2021
27.05.2021
15.04.2021
17.04.2021
20.04.2021
20.04.2021
21.04.2021
22.04.2021
22.04.2021
23.04.2021
24.04.2021
27.04.2021
27.04.2021
04.05.2021
04.05.2021
06.05.2021
07.05.2021
08.05.2021
08.05.2021
11.05.2021
12.05.2021
15.05.2021
15.05.2021
20.05.2021
22.05.2021
25.05.2021
27.05.2021
28.05.2021

## Ressorts_neu richtig vergeben


Für die Ressortübersicht brauchen wir eine neue Ressortspalte, um es so abzubilden, wie es sein soll. Problem: Seitentitel wie Service kommen in mehreren Lokalredaktionen vor, wir müssen also genauer unterteilen. 

In [9]:
# Ausgangsdaten und bereits vorhandene Match-Datei laden
df_k = df.copy()
res_match = pd.read_pickle("ressorts_neu.pkl")

In [10]:
res_match.rename(columns={"Ressort_neu": "Ressort_neu3"}, inplace=True)

In [11]:
# beide Dateien matchen
df_ = df_k.merge(res_match, on="Seitentitel")

In [12]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11315 entries, 0 to 11314
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   SplitId             11315 non-null  int64  
 1   ArtikelId           11315 non-null  int64  
 2   Artikel             11312 non-null  object 
 3   ErscheinungsId      11315 non-null  int64  
 4   AusgabeId           11315 non-null  int64  
 5   Datum               11315 non-null  object 
 6   Seite               11315 non-null  int64  
 7   Ausgabeleser        11315 non-null  int64  
 8   Artikelleser        11315 non-null  int64  
 9   BW                  11315 non-null  float64
 10  DW                  11315 non-null  float64
 11  LW                  11315 non-null  float64
 12  Seitentitel         11315 non-null  object 
 13  Platzierungen       11315 non-null  object 
 14  Darstellungsformen  11315 non-null  object 
 15  Beschreibung        4677 non-null   object 
 16  Hand

In [13]:
# für Lokales Ebersberg, Lokales Starnberg neu vergeben
# Hier müssen die Seitentitel Lankdreis Ebersberg, Kultur im Landkreis, Service jeweils
# für die einzelne Lokalausgabe erfasst werden.

df_.loc[df_["Ausgabeteil"] == "Lokales Ebersberg",
        "Ressort_neu3"] = "Landkreis Ebersberg"
df_.loc[df_["Ausgabeteil"] == "Lokales Starnberg",
        "Ressort_neu3"] = "Landkreis Starnberg"
df_.loc[df_["Seite"] == 1, "Ressort_neu3"] = "Titelseite"
df_.loc[df_["Seitentitel"] == "Medien", "Ressort_neu3"] = "Medien"

In [83]:
final = df_[["SplitId", "Ressort_neu3"]]

In [84]:
final.to_csv("ressorts_neu3.csv", index=False, decimal=",")

In [85]:
t = df_[df_["Ressort_neu3"] == "Landkreis Starnberg"]
t.shape

(661, 34)

In [87]:
t["LW"] = t["LW"].str.replace(",", ".")

<ipython-input-87-bae80a57e206>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t["LW"] = t["LW"].str.replace(",", ".")


In [89]:
t["LW"] = t["LW"].astype(float)

<ipython-input-89-5a0bba165dc1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t["LW"] = t["LW"].astype(float)


In [90]:
t["LW"].mean()

15.43629349470499

## Teaser-Element und angefeaturte Texte 

In [92]:
df_nk = pd.read_csv()

In [120]:
data_url_nk = "./data/NK.csv"
df_nk = pd.read_csv(data_url_nk,
                    keep_default_na=False,
                    na_values=na_val)

data_url = "./data/K.csv"
df_k = pd.read_csv(data_url,
                   keep_default_na=False,
                   na_values=na_val)

c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [121]:
df_t = pd.read_excel("./data/210831_Teaser_Titelseite.xlsx")

In [122]:
df_t["ArtikelID des geteasten Artikels"] = df_t["ArtikelID des geteasten Artikels"].astype(
    int)

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [139]:
t

,SplitId,Teaser,Ressort,Seite,Teasertext,Erscheinungsdatum,ArtikelID des geteasten Artikels,Unnamed: 7
0,4996.0,HEUTE,Meinung,4,Russlands Aufmarsch an der Grenze zur Ukraine ...,2021-04-12,16626,NaN
1,4996.0,HEUTE,Politik,7,Krieg gegen das eigene Volk: Das Militär in My...,2021-04-12,16557,NaN
2,4996.0,HEUTE,Feuilleton,9,Hereinspaziert – trotz Pandemie will die Archi...,2021-04-12,16549,NaN
3,4996.0,HEUTE,Das Politische Buch,12,Alec MacGillis analysiert Macht und Machenscha...,2021-04-12,16480,NaN
4,4996.0,HEUTE,Sport,23,Eskalation im Streit von Manager Salihamidzic ...,2021-04-12,16488,NaN
...,...,...,...,...,...,...,...,...
232,NaN,Titelbild,Panorama,8,Noch ganz Sauber?,2021-06-02,58886,NaN
233,NaN,Titelbild,Die Seite Drei,3,Skrupellos,2021-06-02,58873,NaN
234,NaN,Titelbild,Die Seite Drei,3,Mission Appolofalter,2021-06-05,60982,NaN
235,NaN,Titelbild,Feuilleton,15,Abgründe der Liebe,2021-06-05,61051,NaN


In [145]:
t = df_t[~df_t["ArtikelID des geteasten Artikels"].isnull()]
t["ArtikelID des geteasten Artikels"] = t["ArtikelID des geteasten Artikels"].astype(
    int)
t_titel = t[(t["Teaser"] == "Titelseite") | (t["Teaser"] == "Titelbild")]
t_heute = t[t["Teaser"] == "HEUTE"]
all_artikelids = t["ArtikelID des geteasten Artikels"].to_list()
print(all_artikelids)
print(len(all_artikelids))
titel_artikelids = t_titel["ArtikelID des geteasten Artikels"].to_list()
heute_artikelids = t_heute["ArtikelID des geteasten Artikels"].to_list()
print(len(titel_artikelids))
print(len(heute_artikelids))
t.Teaser.value_counts()

[16626, 16557, 16549, 16480, 16488, 16472, 16607, 17494, 17456, 17634, 17525, 17496, 18481, 18420, 18368, 18426, 18415, 19310, 19325, 19359, 19402, 19432, 19383, 20400, 20268, 20329, 20309, 20341, 22502, 22570, 22562, 22569, 22610, 22506, 23433, 23462, 23490, 23491, 23457, 24453, 24416, 24350, 24373, 24471, 25374, 25553, 25453, 25396, 25511, 24781, 24790, 26388, 26432, 26415, 26495, 26418, 28829, 28878, 28892, 28997, 29875, 29895, 29804, 29834, 29354, 29333, 29856, 29848, 30618, 30784, 30627, 30685, 30679, 31646, 31852, 31672, 31686, 31699, 34304, 34109, 34107, 34121, 34145, 34096, 35208, 35097, 35101, 35122, 35147, 34962, 35979, 36028, 36089, 35977, 35985, 35462, 36462, 36964, 37093, 37061, 37025, 36988, 37976, 38096, 37993, 38070, 38003, 40213, 40263, 40293, 40320, 40299, 40276, 40271, 40626, 40659, 41187, 41299, 41230, 41245, 43190, 43203, 43250, 43305, 43204, 45515, 45675, 45575, 45538, 45581, 46597, 46593, 46455, 46466, 46538, 46446, 47356, 47381, 47387, 47485, 47475, 46834, 46901

<ipython-input-145-caca61d49dc2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t["ArtikelID des geteasten Artikels"] = t["ArtikelID des geteasten Artikels"].astype(int)


HEUTE         169
Titelbild      51
Titelseite      7
Name: Teaser, dtype: int64

In [124]:
df_nk

,SplitId,ArtikelId,Artikel,ErscheinungsId,AusgabeId,Datum,Seite,Ausgabeleser,Artikelleser,BW,...,Kumulierung,Wellennummer,Dachzeile,Wochentag,Worte,Seitenreihung,Optik,Ressort_neu,Ressort_neu2,DF
0,5420,17724,Havertz in Götzes Spur,90,5,"Dienstag, 13. April 2021",27,62,7,"11,2903225806452",...,ja,7,NaN,Dienstag,410,1,nein,Sport,Sport,Bericht
1,5420,17474,Havertz in Götzes Spur,89,4,"Dienstag, 13. April 2021",27,50,7,"14,0",...,ja,8,NaN,Dienstag,410,1,nein,Sport,Sport,Bericht
2,5420,17359,Havertz in Götzes Spur,88,3,"Dienstag, 13. April 2021",27,72,6,"8,33333333333333",...,ja,6,NaN,Dienstag,410,1,nein,Sport,Sport,Bericht
3,8327,25968,Champions-League-Playoff in Wolfsburg,131,1,"Freitag, 23. April 2021",26,183,24,"13,1147540983607",...,ja,4,NaN,Freitag,529,2,ja,Sport,Sport,Bericht
4,8327,26197,Champions-League-Playoff in Wolfsburg,133,3,"Freitag, 23. April 2021",26,66,3,"4,54545454545455",...,ja,6,NaN,Freitag,529,2,ja,Sport,Sport,Bericht
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35712,19164,60366,"Sage keiner, diese Rente sei Luxus",296,1,"Samstag, 5. Juni 2021",14,127,10,"7,8740157480315",...,ja,4,TROTZ ANHEBUNG NACH CORONA,Samstag,980,1,ja,Forum & Leserbriefe,Forum & Leserbriefe,Leserbrief
35713,19164,61079,"Sage keiner, diese Rente sei Luxus",299,4,"Samstag, 5. Juni 2021",14,48,11,"22,9166666666667",...,ja,8,TROTZ ANHEBUNG NACH CORONA,Samstag,980,1,ja,Forum & Leserbriefe,Forum & Leserbriefe,Leserbrief
35714,19164,60775,"Sage keiner, diese Rente sei Luxus",298,3,"Samstag, 5. Juni 2021",14,46,8,"17,3913043478261",...,ja,6,TROTZ ANHEBUNG NACH CORONA,Samstag,980,1,ja,Forum & Leserbriefe,Forum & Leserbriefe,Leserbrief
35715,19164,61358,"Sage keiner, diese Rente sei Luxus",300,5,"Samstag, 5. Juni 2021",14,54,5,"9,25925925925926",...,ja,7,TROTZ ANHEBUNG NACH CORONA,Samstag,980,1,ja,Forum & Leserbriefe,Forum & Leserbriefe,Leserbrief


In [148]:
# alle gefeatureten Artikel heraussuchen
print(len(titel_artikelids))
print(len(heute_artikelids))

df_nk_titel = df_nk[df_nk["ArtikelId"].isin(titel_artikelids)]
df_nk_heute = df_nk[df_nk["ArtikelId"].isin(heute_artikelids)]

titel_splits = df_nk_titel.SplitId.unique()
heute_splits = df_nk_heute.SplitId.unique()
# alle SplitIds herausfiltern
# alle Überschriften herausfiltern
titel_üs = df_nk_titel.Artikel.unique()
heute_üs = df_nk_heute.Artikel.unique()


df_nk_üs_titel = df_nk[df_nk["Artikel"].isin(titel_üs)]
df_nk_üs_heute = df_nk[df_nk["Artikel"].isin(heute_üs)]

58
169


In [149]:
all_splits_titel = df_nk_üs_titel.SplitId.unique()
all_splits_heute = df_nk_üs_heute.SplitId.unique()

In [159]:
df_new = df_k[['SplitId', 'ArtikelId', 'Artikel',
               'Datum', 'Seite', 'BW', 'DW', 'LW',
               'Seitentitel', 'Platzierungen', 'Darstellungsformen', 'Beschreibung',
               'Handlungsorte', 'Akteure', 'Themen', 'Ausgabe', 'Layerlink', 'Ressort',
               'Ausgabeteil', 'Wochentag',
               'Worte']]

In [160]:
df_new["Teaser ja/nein"] = "nein"
df_new["Teaserart"] = ""
df_new.loc[df_new["SplitId"].isin(all_splits_titel), "Teaser ja/nein"] = "ja"
df_new.loc[df_new["SplitId"].isin(all_splits_titel), "Teaserart"] = "Titelbild"

df_new.loc[df_new["SplitId"].isin(all_splits_heute), "Teaser ja/nein"] = "ja"
df_new.loc[df_new["SplitId"].isin(all_splits_heute), "Teaserart"] = "Heute"

<ipython-input-160-1ea5d5a26cc2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["Teaser ja/nein"] = "nein"
<ipython-input-160-1ea5d5a26cc2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["Teaserart"] = ""
c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [161]:
df_new["LW"] = df_new["LW"].str.replace(",", ".").astype(float)

<ipython-input-161-605800cf18b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["LW"] = df_new["LW"].str.replace(",", ".").astype(float)


In [162]:
df_new["BW"] = df_new["BW"].str.replace(",", ".").astype(float)
df_new["DW"] = df_new["DW"].str.replace(",", ".").astype(float)

<ipython-input-162-55b482ca4ca4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["BW"] = df_new["BW"].str.replace(",", ".").astype(float)
<ipython-input-162-55b482ca4ca4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new["DW"] = df_new["DW"].str.replace(",", ".").astype(float)


In [165]:
df_new.to_csv("teaser.csv", index=False)

In [164]:
df_new.to_clipboard(index=False, decimal=",")

In [163]:
df_new.shape

(11315, 23)

## Rubriken und Kolumnen Wirtschaft

In [77]:
df_w = df[df["Ressort"] == "Wirtschaft"]

In [78]:
df_w["kol"] = ""

<ipython-input-78-759caa834297>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w["kol"]=""


In [79]:
df_w["Dachzeile"] = df_w["Dachzeile"].fillna("")

<ipython-input-79-bac334958031>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w["Dachzeile"] = df_w["Dachzeile"].fillna("")


In [80]:
# Leitartikel
df_w.loc[df_w.Beschreibung == "Leitartikel", "kol"] = "Leitartikel"

# Report
df_w.loc[df_w.Seitentitel == "Report", "kol"] = "Report"

# Reden wir über Geld
df_w.loc[(df_w.Beschreibung == "Reden wir über Geld") |
         (df_w.Dachzeile.str.contains("REDEN WIR ÜBER GELD")), "kol"] = \
    "Reden wir über Geld"

# Nahaufnahme
df_w.loc[df_w.Dachzeile == "NAHAUFNAHME", "kol"] = "Nahaufnahme"

# Montagsinterview
df_w.loc[(df_w.Beschreibung == "Montagsinterview") |
         (df_w.Dachzeile.str.contains("MONTAGSINTERVIEW")), "kol"] = \
    "Montagsinterview"

# Forum
df_w.loc[(df_w.Beschreibung == "Forum") |
         (df_w.Dachzeile.str.contains("FORUM")), "kol"] = \
    "Forum"

# Nach Corona
df_w.loc[(df_w.Beschreibung == "Nach Corona") |
         (df_w.Dachzeile.str.contains("NACH CORONA")), "kol"] = \
    "Nach Corona"

# Braucht man das?
df_w.loc[df_w.Dachzeile == "BRAUCHT MAN DAS?", "kol"] = "Braucht man das?"


# Personalien
df_w.loc[(df_w.Beschreibung == "Personalien") |
         (df_w.Dachzeile.str.contains("PERSONALIEN")), "kol"] = \
    "Personalien"

# Personalien
df_w.loc[(df_w.Beschreibung == "Pipers Welt") |
         (df_w.Dachzeile.str.contains("PIPERS")), "kol"] = \
    "Pipers Welt"

# Bei uns...
#df_w.loc[df_w["Dachzeile"].str.contains("BEI UNS IN"), "kol"] = "Bei uns in..."

# Bei uns in...
df_w.loc[df_w.Dachzeile.str.contains("BEI UNS IN"), "kol"] = "Bei uns in..."

# Zwischen den Zahlen
df_w.loc[(df_w.Beschreibung == "Zwischen den Zahlen") |
         (df_w.Dachzeile.str.contains("ZWISCHEN DEN ZAHLEN")), "kol"] = \
    "Zwischen den Zahlen"

# Börse vom...
df_w.loc[df_w.Dachzeile.str.contains("BÖRSE VOM"), "kol"] = "Börse vom..."


# Kolumne am Mittwoch
# Silicon Beach, 14.4.
# Das Deutsche Valley 21.4.
# Silicon Future, 7.4.
# Femme Digital, 28.4.
# silicon Future, 5.5.
# Silicon Beach, 12.5.
# das Deutsche Valley, 19.5.
# Femme Digital, 26.5.
# Silicon Future, 2.6.
# Kolumne am Mittwoch
df_w.loc[(df_w.Dachzeile.str.contains("FEMME DIGITAL")) |
         (df_w.Dachzeile.str.contains("SILICON FUTURE")) |
         (df_w.Dachzeile.str.contains("SILICON BEACH")) |
         (df_w.Dachzeile.str.contains("DAS DEUTSCHE VALLEY")), "kol"] = "Kolumne am Mittwoch"

c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [81]:
df_w.kol.value_counts()

                       830
Personalien            103
Leitartikel             42
Nahaufnahme             21
Börse vom...            20
Kolumne am Mittwoch      9
Report                   9
Bei uns in...            8
Montagsinterview         8
Reden wir über Geld      8
Nach Corona              7
Braucht man das?         7
Zwischen den Zahlen      7
Forum                    6
Pipers Welt              3
Name: kol, dtype: int64

In [82]:
cols = ["Beschreibung", "Anzahl", "Lesewert"]
new_df = pd.DataFrame(columns=cols)

In [83]:
df_w["LW"] = df_w["LW"].str.replace(",", ".").astype(float)
df_w["BW"] = df_w["BW"].str.replace(",", ".").astype(float)
df_w["DW"] = df_w["DW"].str.replace(",", ".").astype(float)

<ipython-input-83-d65502409d12>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w["LW"] = df_w["LW"].str.replace(",", ".").astype(float)
<ipython-input-83-d65502409d12>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w["BW"] = df_w["BW"].str.replace(",", ".").astype(float)
<ipython-input-83-d65502409d12>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [84]:
alle_beschreibungen = df_w.kol.unique()
for be in alle_beschreibungen:
    df_ = df_w[df_w["kol"] == be]
    lw = df_.LW.mean()
    anzahl = df_.shape[0]
    besch = be + f" ({anzahl})"
    new_ser = pd.Series([besch, anzahl, round(lw, 1)], index=cols)
    new_df = new_df.append(new_ser, ignore_index=True)

In [85]:
new_df

,Beschreibung,Anzahl,Lesewert
0,(830),830,9.8
1,Report (9),9,23.4
2,Forum (6),6,9.5
3,Leitartikel (42),42,21.1
4,Reden wir über Geld (8),8,19.5
5,Personalien (103),103,6.0
6,Nahaufnahme (21),21,14.5
7,Börse vom... (20),20,2.9
8,Montagsinterview (8),8,17.3
9,Nach Corona (7),7,9.0


In [86]:
new_df.to_csv("Kolumnen_Wirtschaft.csv", index=False, decimal=",")

In [7]:
df_w.Beschreibung.value_counts()

Analyse                          124
Personalien                      100
Bericht                           79
Börse                             65
Feature                           50
Devisen & Rohstoffe               45
Kurz gemeldet                     44
Leitartikel                       42
Porträt                           39
Anleihen, Devisen & Rohstoffe     17
Porträt, Mittwochsporträt          7
Börse, Bericht                     5
Forum                              5
Reportage                          5
Nach Corona                        5
Porträt, Nachruf                   3
Reden wir über Geld                2
Zwischen den Zahlen                2
Kommentar                          1
Montagsinterview                   1
Pipers Welt                        1
Name: Beschreibung, dtype: int64

## Lesewerte Gesellschaft vs Stil

In [124]:
df_ = df[(df["Ressort"] == "Gesellschaft") | (df["Ressort"] == "Stil")]

In [125]:
all_darst = df_.DF.unique()

In [126]:
all_darst

array(['Bildnachricht', 'Komm./Kolumne', 'Rezension/Kritik', 'Interview',
       'Geschichtstext', 'Essay', 'Struktur. Artikel', 'Bericht',
       'Analyse', 'Feature', 'Porträt', 'Reportage', nan], dtype=object)

In [127]:
cols = ["DF", "LW_g", "LW_s", "Anzahl_gesamt",
        "Anzahl_Gesellschaft", "Anzahl_Stil"]
new_df = pd.DataFrame(columns=cols)

In [128]:
for da in all_darst:
    df_d = df_[df_["DF"] == da]
    df_dg = df_d[df_d.Ressort == "Gesellschaft"]
    df_ds = df_d[df_d.Ressort == "Stil"]
    lw_g = df_dg["LW"].mean()
    lw_s = df_ds["LW"].mean()
    anzahl_g = df_dg.shape[0]
    anzahl_s = df_ds.shape[0]
    anzahl_ges = anzahl_g + anzahl_s

    new_ser = pd.Series([da, round(lw_g, 1), round(
        lw_s, 1), anzahl_ges, anzahl_g, anzahl_s], index=cols)
    new_df = new_df.append(new_ser, ignore_index=True)

In [129]:
new_df

,DF,LW_g,LW_s,Anzahl_gesamt,Anzahl_Gesellschaft,Anzahl_Stil
0,Bildnachricht,2.2,NaN,1,1,0
1,Komm./Kolumne,10.8,8.5,48,17,31
2,Rezension/Kritik,NaN,6.2,3,0,3
3,Interview,25.5,NaN,10,10,0
4,Geschichtstext,22.3,NaN,7,7,0
5,Essay,25.0,14.5,14,9,5
6,Struktur. Artikel,NaN,14.8,9,0,9
7,Bericht,11.4,15.4,2,1,1
8,Analyse,5.8,8.4,2,1,1
9,Feature,15.4,7.3,11,2,9


In [135]:
new_df = new_df.fillna(0)

In [141]:
new_s = pd.Series(["Bildbericht", 13.3, 0, 7, 7, 0], index=new_df.columns)

In [143]:
new_df = new_df.append(new_s, ignore_index=True)

In [144]:
new_df

,DF,LW_g,LW_s,Anzahl_gesamt,Anzahl_Gesellschaft,Anzahl_Stil
0,Bildnachricht,2.2,0.0,1.0,1.0,0.0
1,Komm./Kolumne,10.8,8.5,48.0,17.0,31.0
2,Rezension/Kritik,0.0,6.2,3.0,0.0,3.0
3,Interview,25.5,0.0,10.0,10.0,0.0
4,Geschichtstext,22.3,0.0,7.0,7.0,0.0
5,Essay,25.0,14.5,14.0,9.0,5.0
6,Struktur. Artikel,0.0,14.8,9.0,0.0,9.0
7,Bericht,11.4,15.4,2.0,1.0,1.0
8,Analyse,5.8,8.4,2.0,1.0,1.0
9,Feature,15.4,7.3,11.0,2.0,9.0


In [58]:
new_df.to_csv("gesell_stil_darstellungsformen.csv", index=False, decimal=",")

## Lesewert Gesellschaft Stil Thematisch

In [60]:
df_ = df[(df["Ressort"] == "Gesellschaft") | (df["Ressort"] == "Stil")]
suchworte = ["Familie", "Historie", "Mode",
             "Design", "Architektur", "Kulinarik", ]

In [61]:
cols = ["Thema", "Anzahl", "Lesewert"]
new_d = pd.DataFrame(columns=cols)

for w in suchworte:
    if w == "Architektur":
        df_wo = df_[(df_["Themen"].str.contains(w)) |
                    (df_["Themen"].str.contains("Haus")) |
                    (df_["Themen"].str.contains("Wohnen")) |
                    (df_["Themen"].str.contains("Gebäude"))]
    elif w == "Kulinarik":
        df_wo = df_[(df_["Themen"].str.contains(w)) |
                    (df_["Themen"].str.contains("Essen")) |
                    (df_["Themen"].str.contains("kochen")) |
                    (df_["Themen"].str.contains("essen")) |
                    (df_["Themen"].str.contains("Kochen")) |
                    (df_["Themen"].str.contains("Küche")) |
                    (df_["Themen"].str.contains("Trinken")) |
                    (df_["Themen"].str.contains("Essen")) |
                    (df_["Themen"].str.contains("kochen")) |
                    (df_["Themen"].str.contains("trinken"))]

    elif w == "Historie":
        df_wo = df_[(df_["Themen"].str.contains(w)) |
                    (df_["Themen"].str.contains("Geschichte")) |
                    (df_["Themen"].str.contains("Damals")) |
                    (df_["Themen"].str.contains("Vergangenheit")) |
                    (df_["Themen"].str.contains("History"))
                    ]

    else:
        df_wo = df_[df_["Themen"].str.contains(w)]

    lw = df_wo.LW.mean()
    anzahl = df_wo.shape[0]
    new_ser = pd.Series([w, anzahl, lw], index=cols)
    new_d = new_d.append(new_ser, ignore_index=True)

In [62]:
new_d

,Thema,Anzahl,Lesewert
0,Familie,14,14.521429
1,Historie,9,21.670000
2,Mode,14,7.274286
3,Design,7,5.684286
4,Architektur,21,13.535714
5,Kulinarik,17,13.363529


In [63]:
new_d.to_csv("gesell_stil_themen.csv", index=False, decimal=",")

## GEsellschaft Stil Kolumnen und Rubriken 

In [64]:
df_["kol"] = ""

<ipython-input-64-6c7f07d365e5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_["kol"] = ""


In [65]:
df_["Dachzeile"] = df_["Dachzeile"].fillna("")
# La Boum
df_.loc[(df_.Beschreibung == "La Boum") |
        (df_.Dachzeile.str.contains("LA BOUM")), "kol"] = \
    "La Boum"

# Ladies & Gentlemen
df_.loc[(df_.Beschreibung == "Ladies & Gentlemen") |
        (df_.Dachzeile.str.contains("LADIES & GENTLEMEN")) |
        (df_.Dachzeile.str.contains("LADIES & GENTELMEN")), "kol"] = \
    "Ladies & Gentlemen"

# Fotoalbum
df_.loc[df_.Beschreibung == "Fotoalbum", "kol"] = "Fotoalbum"

# Beziehungsweise
df_.loc[df_.Beschreibung == "Beziehungsweise", "kol"] = "Beziehungsweise"

# Eigener Herd
df_.loc[df_.Beschreibung == "Eigener Herd", "kol"] = "Eigener Herd"


# Beziehungsweise
df_.loc[df_.Beschreibung == "Beziehungsweise", "kol"] = "Beziehungsweise"

# Familientrio
df_.loc[df_.Beschreibung == "Familientrio", "kol"] = "Familientrio"

# Kurz gesichtet
df_.loc[df_.Beschreibung == "Kurz gesichtet", "kol"] = "Kurz gesichtet"

# Test
df_.loc[df_.Beschreibung == "test", "kol"] = "Test"


# INterview
df_.loc[df_["Dachzeile"].str.contains("ÜBER"), "kol"] = "Das Interview"

<ipython-input-65-7f8a501cfcdf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_["Dachzeile"] = df_["Dachzeile"].fillna("")
c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [66]:
df_.kol.value_counts()

                      79
Ladies & Gentlemen    16
Kurz gesichtet         8
Das Interview          8
Fotoalbum              7
La Boum                7
Test                   6
Familientrio           5
Eigener Herd           4
Beziehungsweise        1
Name: kol, dtype: int64

In [67]:
cols = ["Name", "LW", "Anzahl"]
new_df = pd.DataFrame(columns=cols)

all_kols = df_.kol.unique()

for ko in all_kols:
    df_k = df_[df_["kol"] == ko]
    lw = round(df_k["LW"].mean(), 1)
    anzahl = df_k.shape[0]

    new_ser = pd.Series([ko, lw, anzahl], index=cols)

    new_df = new_df.append(new_ser, ignore_index=True)

In [68]:
new_df

,Name,LW,Anzahl
0,,15.0,79
1,Familientrio,11.3,5
2,Eigener Herd,14.0,4
3,Test,14.4,6
4,Fotoalbum,13.3,7
5,Beziehungsweise,22.1,1
6,Kurz gesichtet,3.8,8
7,La Boum,7.9,7
8,Ladies & Gentlemen,9.6,16
9,Das Interview,28.1,8


In [69]:
new_df.to_csv("ges_stil_kolumnen.csv", index=False, decimal=",")

## Geschlechter-Daten Panorama

In [163]:
#df_g = pd.read_clipboard(decimal=",")
#df_g.to_csv("Geschlechterdaten.csv", index=False)

na_val = ['-1.#IND', '1.#QNAN', '1.#IND', '-1.#QNAN', '#N/A N/A', '#N/A', 'N/A',
          'n/a', '', '#NA', 'NULL', 'null',  '-NaN', 'nan', '-nan', '']
data_g = "Geschlechterdaten.csv"
df_g = pd.read_csv(data_g,
                   keep_default_na=False,
                   na_values=na_val)

In [164]:
df_final = df_g[['TeilartikelId', 'Platzierung',
                 'LW_w', 'BW_w', 'DW_w', 'LW_m', 'BW_m', 'DW_m']].merge(df[["SplitId", "Artikel", "Datum",
                                                                            'Seitentitel', 'Platzierungen', 'Darstellungsformen', 'Beschreibung',
                                                                            'Handlungsorte', 'Akteure', 'Themen', 'Ausgabe', 'Layerlink', 'Ressort',
                                                                            'Ausgabeteil']], left_on="TeilartikelId", right_on="SplitId")

In [165]:
df_final.columns

Index(['TeilartikelId', 'Platzierung', 'LW_w', 'BW_w', 'DW_w', 'LW_m', 'BW_m',
       'DW_m', 'SplitId', 'Artikel', 'Datum', 'Seitentitel', 'Platzierungen',
       'Darstellungsformen', 'Beschreibung', 'Handlungsorte', 'Akteure',
       'Themen', 'Ausgabe', 'Layerlink', 'Ressort', 'Ausgabeteil'],
      dtype='object')

In [14]:
df_fin = df_final[["SplitId", "Artikel", "Datum", "Ressort",  'LW_w', 'BW_w',
                   'DW_w', 'LW_m', 'BW_m', 'Darstellungsformen', 'Platzierungen', 'Ausgabeteil']]

In [15]:
df_fin.to_clipboard(decimal=",", index=False)

In [22]:
# nur Panorama ausspielen
df_pg = df_g[df_g["Ressort"] == "Panorama"]
df_pg = df_pg[["TeilartikelId", "LW_w", "LW_m", "Ressort"]]

In [25]:
# Differenz zw. M und F - neue Column
df_pg["diff"] = 0
df_pg["mode"] = ""

In [59]:
# Tabelle für Frauenartikel mit großen Unterschied zu Männerartikeln
def differences(df):
    cols = ["SplitId", "LW_w", "LW_m", "difference%", "mode"]
    # jeweils alle ARtikel über 20, bei denen W größer ist als männlich und umgekehrt
    df_w = df[(df["LW_w"] >= 20) & (df["LW_w"] > df["LW_m"])]
    df_m = df[(df["LW_m"] >= 20) & (df["LW_m"] > df["LW_w"])]
    df_w["diff"] = (df_w["LW_m"]-df_w["LW_w"]) / df_w["LW_w"] * 100
    df_m["diff"] = (df_m["LW_w"]-df_m["LW_m"]) / df_m["LW_m"] * 100

    # Daten sortieren, Rang vergeben, beide Datensätze verbinden
    df_w.sort_values(by="diff", ascending=False, inplace=True)
    df_m.sort_values(by="diff", ascending=False, inplace=True)

    df_w["Rang"] = df_w["diff"].rank()
    df_m["Rang"] = df_m["diff"].rank()

    # mode vergeben
    df_w["mode"] = "w"
    df_m["mode"] = "m"

    print(df_m)

    df_all = df_w.append(df_m)
    return df_all


df_all = differences(df_pg)

      TeilartikelId       LW_w       LW_m   Ressort       diff mode  Rang
6422          12611  21.651903  21.945996  Panorama  -1.340076    m  24.0
6489          18233  20.516841  20.873157  Panorama  -1.707054    m  23.0
6310          15764  21.046997  21.540244  Panorama  -2.289888    m  22.0
6622          10966  33.572606  35.012550  Panorama  -4.112651    m  21.0
6630          11292  20.645047  21.799236  Panorama  -5.294630    m  20.0
6423          12621  22.040176  23.676411  Panorama  -6.910827    m  19.0
6431          12959  18.683534  20.083909  Panorama  -6.972622    m  18.0
6591           6668  23.066649  24.898289  Panorama  -7.356490    m  17.0
6365           8624  18.844599  20.824746  Panorama  -9.508625    m  16.0
6666          14600  29.541757  33.330176  Panorama -11.366332    m  15.0
6428          12684  20.075152  22.763418  Panorama -11.809590    m  14.0
6402          10991  21.945971  25.728430  Panorama -14.701477    m  13.0
6414          16894  31.699356  37.309

<ipython-input-59-eaaf6a6ec55c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_w["diff"] = (df_w["LW_m"]-df_w["LW_w"]) / df_w["LW_w"] * 100
<ipython-input-59-eaaf6a6ec55c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m["diff"] = (df_m["LW_w"]-df_m["LW_m"]) / df_m["LW_m"] * 100
<ipython-input-59-eaaf6a6ec55c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

In [61]:
df_all.to_csv("Rangliste_Panorama_Geschlechter.csv", index=False, decimal=",")

In [60]:
df_all = df_all.merge(df[["SplitId", "Artikel", "Platzierungen",
                      "DF", "Datum"]], left_on="TeilartikelId", right_on="SplitId")

In [58]:
df_all

,TeilartikelId,LW_w,LW_m,Ressort,diff,mode,Rang,SplitId,Artikel,Platzierungen,DF,Datum
0,8434,34.078701,33.538020,Panorama,-1.586567,w,1.0,8434,Ohrfeigen gegen Autodieb,TS1,Nachricht,23.04.2021
1,15213,23.958133,23.502231,Panorama,-1.902915,w,2.0,15213,Lena und der Papst,TS1,Komm./Kolumne,20.05.2021
2,17700,45.224002,44.340875,Panorama,-1.952786,w,3.0,17700,Schicksalsberg,AA,Porträt,29.05.2021
3,16701,36.140391,35.359629,Panorama,-2.160356,w,4.0,16701,War der Pfadfinder der Mörder?,AA,Feature,26.05.2021
4,16904,43.815164,42.675089,Panorama,-2.602008,w,5.0,16904,Drei Festnahmen am Lago Maggiore,SK,Bericht,27.05.2021
...,...,...,...,...,...,...,...,...,...,...,...,...
360,17597,27.623993,37.963910,Panorama,-27.236176,m,20.0,17597,Der Wehrmachtspanzer aus der Tiefgarage,SK,Bericht,29.05.2021
361,15218,21.919204,30.627021,Panorama,-28.431814,m,21.0,15218,Aus dem letzten Loch,SK,Feature,20.05.2021
362,6191,13.208263,20.900733,Panorama,-36.804788,m,22.0,6191,Mit Schlagstöcken gegen Bayern-Fans,SK,Bericht,15.04.2021
363,7615,18.716835,31.928575,Panorama,-41.379046,m,23.0,7615,Besser als die Beute,TS2,Porträt,21.04.2021


## Matching SZ-Id und Lesewert-Id

In [20]:
df_id = pd.read_excel("./data/Süddeutsche_Datenabzug.xlsx",
                      sheet_name="NichtKumuliert")

In [22]:
df_id = df_id[["ArtikelId_SZ", "ArtikelId_Lesewert"]]

In [60]:
url_data = "./data/Online-Abgleich_Artikellisten_Panorama_Rubriken.xlsx"
df_data = pd.read_excel(url_data, sheet_name=2)

In [61]:
df_data

,Artikel,Datum,Ressort,Seitentitel,Print-Gruppe,Online-Gruppe,Lesewert,Blickwert,Durchlesewert,Link
0,München,2021-04-23,Panorama,Panorama,3,NaN,27.952616,30.787037,https://mein.lesewert.de/1020/Statistic/Page/D...,NaN
1,Thun,2021-04-23,Panorama,Panorama,2,NaN,22.602426,25.231481,https://mein.lesewert.de/1020/Statistic/Page/D...,NaN
2,Pfalzpaint,2021-04-23,Panorama,Panorama,2,NaN,20.900754,23.148148,https://mein.lesewert.de/1020/Statistic/Page/D...,NaN
3,Hamburg,2021-04-30,Panorama,Panorama,3,NaN,27.748899,30.703625,https://mein.lesewert.de/1020/Statistic/Page/D...,NaN
4,München,2021-04-30,Panorama,Panorama,3,NaN,26.622472,29.424307,https://mein.lesewert.de/1020/Statistic/Page/D...,NaN
5,Lenggries,2021-04-30,Panorama,Panorama,2,NaN,24.933866,27.078891,https://mein.lesewert.de/1020/Statistic/Page/D...,NaN
6,Murnauer Moos,2021-05-07,Panorama,Panorama,3,NaN,30.099087,33.413462,https://mein.lesewert.de/1020/Statistic/Page/D...,NaN
7,Berlin,2021-05-07,Panorama,Panorama,3,NaN,28.429835,31.490385,https://mein.lesewert.de/1020/Statistic/Page/D...,NaN
8,Incheon,2021-05-07,Panorama,Panorama,3,NaN,25.076467,28.125000,https://mein.lesewert.de/1020/Statistic/Page/D...,NaN
9,Berlin,2021-05-14,Panorama,Panorama,3,NaN,26.430485,29.503916,https://mein.lesewert.de/1020/Statistic/Page/D...,NaN


In [62]:
df_data["ArtikelId"] = df_data["Durchlesewert"].str.split("/")
df_data["ArtikelId"] = df_data["ArtikelId"].apply(lambda x: x[-1])
df_data["ArtikelId"] = df_data["ArtikelId"].astype(int)

In [63]:
df_merge = pd.merge(df_data, df_id, left_on="ArtikelId",
                    right_on="ArtikelId_Lesewert")

In [64]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 17
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Artikel             18 non-null     object        
 1   Datum               18 non-null     datetime64[ns]
 2   Ressort             18 non-null     object        
 3   Seitentitel         18 non-null     object        
 4   Print-Gruppe        18 non-null     int64         
 5   Online-Gruppe       0 non-null      float64       
 6   Lesewert            18 non-null     float64       
 7   Blickwert           18 non-null     float64       
 8   Durchlesewert       18 non-null     object        
 9   Link                0 non-null      float64       
 10  ArtikelId           18 non-null     int32         
 11  ArtikelId_SZ        18 non-null     int64         
 12  ArtikelId_Lesewert  18 non-null     int64         
dtypes: datetime64[ns](1), float64(4), int32(1), int64(3)

In [65]:
df_final = df_merge[["ArtikelId_SZ", "ArtikelId_Lesewert", 'Artikel', 'Datum', 'Ressort', 'Seitentitel', 'Print-Gruppe',
                     'Online-Gruppe', 'Lesewert', 'Blickwert', 'Durchlesewert', 'Link']]

In [66]:
df_final.to_clipboard(decimal=",", index=False)

# Ressortwerte nach WE / WT

Wochenende ist immer ET Samstag, bis auf den 30.04., da ist der ET Freitag schon die WE-Ausgabe (wg. 1. Mai). 


In [5]:
liste_ressort_neu = df.Ressort_neu.unique()

In [42]:
sort_ressorts = pd.read_csv("sort_ressorts.csv")

In [14]:
ids = pd.read_csv("ressorts_neu3.csv")

In [25]:
df_neu = df.merge(ids, on="SplitId")

In [26]:
df_neu.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11321 entries, 0 to 11320
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   SplitId             11321 non-null  int64  
 1   ArtikelId           11321 non-null  int64  
 2   Artikel             11318 non-null  object 
 3   ErscheinungsId      11321 non-null  int64  
 4   AusgabeId           11321 non-null  int64  
 5   Datum               11321 non-null  object 
 6   Seite               11321 non-null  int64  
 7   Ausgabeleser        11321 non-null  int64  
 8   Artikelleser        11321 non-null  int64  
 9   BW                  11321 non-null  float64
 10  DW                  11321 non-null  float64
 11  LW                  11321 non-null  float64
 12  Seitentitel         11321 non-null  object 
 13  Platzierungen       11321 non-null  object 
 14  Darstellungsformen  11321 non-null  object 
 15  Beschreibung        4683 non-null   object 
 16  Hand

In [32]:
df_neu["wochenende"] = df_neu["Wochentag"].apply(
    lambda x: "ja" if x == "Samstag" else "nein")

In [34]:
df_neu.loc[df_neu["Datum"] == "30.04.2021", "wochenende"] = "ja"

In [35]:
t = df_neu[df_neu["Datum"] == "30.04.2021"]
t

,SplitId,ArtikelId,Artikel,ErscheinungsId,AusgabeId,Datum,Seite,Ausgabeleser,Artikelleser,BW,...,Dachzeile,Wochentag,Worte,Seitenreihung,Optik,Ressort_neu,Ressort_neu2,DF,Ressort_neu3,wochenende
32,10553,33317,Inzidenz abermals unter 100,165,5,30.04.2021,79,70,25,35.71,...,NaN,Freitag,72,1,nein,Landkreis Starnberg,Lok 1 Starnberg,Nachricht,Landkreis Starnberg,ja
33,10555,33343,Auszogne am Maibaumplatz,165,5,30.04.2021,80,70,10,14.29,...,NaN,Freitag,97,2,nein,Landkreis Starnberg,Landkreis Starnberg,Nachricht,Landkreis Starnberg,ja
34,10558,33378,Berg blüht auf,165,5,30.04.2021,80,70,6,8.57,...,NaN,Freitag,104,2,nein,Landkreis Starnberg,Landkreis Starnberg,Nachricht,Landkreis Starnberg,ja
138,10494,32764,Geld für Fahrradständer,163,3,30.04.2021,79,68,5,7.35,...,NaN,Freitag,57,1,nein,Stadtviertel,Lok 1 München,Nachricht,Stadtviertel,ja
139,10505,32795,Impftermine im Seniorenzentrum,163,3,30.04.2021,82,68,2,2.94,...,NaN,Freitag,156,4,nein,Stadtviertel,Stadtviertel,Nachricht,Stadtviertel,ja
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,10391,32328,Bundesverdienstkreuz,161,1,30.04.2021,72,469,12,2.56,...,AKTUELLES LEXIKON,Freitag,62,3,nein,Kinderseiten,Kinderseiten,Nachricht,Kinderseiten,ja
10908,10492,32768,Mit voller Wucht in parkendes Auto,163,3,30.04.2021,80,68,5,7.35,...,WESTEN UND WÜRMTAL,Freitag,106,2,nein,Stadtviertel,Stadtviertel,Nachricht,Stadtviertel,ja
10950,10382,32326,Fatma,161,1,30.04.2021,66,469,2,0.43,...,Jezt im Netz,Freitag,43,1,nein,TV-Programm,TV-Programm,Nachricht,TV-Programm,ja
11173,10511,32877,Radl-Rallye mit Schnitzeljagd,163,3,30.04.2021,80,68,3,4.41,...,WESTEN UND WÜRMTAL,Freitag,132,2,nein,Stadtviertel,Stadtviertel,Nachricht,Stadtviertel,ja


In [52]:
liste_ressort_neu = df_neu.Ressort_neu3.unique()

cols = ["Ressort", "LW_wochentags", "LW_wochenende"]
new_df = pd.DataFrame(columns=cols)


for res in liste_ressort_neu:
    df_ = df_neu[df_neu["Ressort_neu3"] == res]
    df_wt = df_[df_["wochenende"] == "nein"]
    lw_wt = df_wt.LW.mean()
    df_we = df_[df_["wochenende"] == "ja"]
    lw_we = df_we.LW.mean()

    new_ser = pd.Series([res, lw_wt, lw_we], index=cols)

    new_df = new_df.append(new_ser, ignore_index=True)

In [53]:
new_df

,Ressort,LW_wochentags,LW_wochenende
0,Landkreis Starnberg,15.355110,15.926167
1,Stadtviertel,6.795933,6.426687
2,Landkreis Ebersberg,18.427443,16.738077
3,München,13.833225,13.042254
4,Titelseite,22.395621,23.987662
5,Bayern,13.506552,14.287232
6,Wirtschaft,9.880946,10.376497
7,Wirtschaft & Technik,10.755833,NaN
8,Panorama,21.651126,20.418023
9,SZ Extra Stream & Service,1.423214,NaN


In [54]:
df_final = pd.merge(new_df, sort_ressorts,
                    left_on="Ressort", right_on="Ressort_neu")

In [56]:
del df_final["Ressort_neu"]

In [57]:
df_fin = df_final.sort_values(by="Sortierung_Ressort_neu")

In [61]:
df_fin.to_csv("ressorts_wochentag_wochenende.csv", index=False, decimal=",")

In [62]:
df_fin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 4 to 17
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Ressort                 25 non-null     object 
 1   LW_wochentags           22 non-null     float64
 2   LW_wochenende           24 non-null     float64
 3   Sortierung_Ressort_neu  25 non-null     int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 1000.0+ bytes


In [ ]:
Standpunkt - 

# Themen identifzieren

In [7]:
themen_liste = ["Verbraucher", "Politik", "Umwelt", "Gesundheit"]

In [26]:
df_th = df_kum[(df_kum["Ressort"]=="Politik")]


In [27]:
tt = df_th[df_th.Themen.str.contains("Verbraucher")]
tt.LW.mean()

23.77157430869433

In [29]:
tt.Seitentitel

1902    Standpunkte
3793          Thema
3968    Standpunkte
5361          Thema
6883          Thema
7832          Thema
Name: Seitentitel, dtype: object

# Nur Wochenend-Leser

In [3]:
# alle Scans aus der Tabelle TeilnehmerDurchlesewerte
df_scans = pd.read_pickle("./data/df_scans.pkl")
# alle Scans aus der Tabelle ScanAuswertungen
df_scanauswertung = pd.read_csv("./data/df_scanauswertung.csv")
df_nk = pd.read_pickle("./data/NK_20210910.pkl")
df_k = pd.read_pickle("./data/K_20210911.pkl")
df_teilnehmer = pd.read_pickle("./data/teilnehmer.pkl")

In [4]:
# nur Scans mit ausgewerteten Artikeln nutzen

In [4]:
# df_scans nur SplitIds nutzen, die auch ausgewertet wurden
valid_splits = df_nk.SplitId.unique()
df_scans = df_scans[df_scans["TeilartikelId"].isin(valid_splits)]

In [5]:
# jetzt aus dem gereinigten Scans alle Wellenteilnehmer filtern, die teilgenommen haben
valid_reader = df_scans.WellenteilnahmenId.unique()

In [6]:
# Teilnehmer Wochenende herausfiltern, fangen alle mit einer 2014 an
df_teil_we = df_teilnehmer[df_teilnehmer["Teilnehmernummer"].str[:4] == "2014"]
df_teil_woche = df_teilnehmer[df_teilnehmer["Teilnehmernummer"].str[:4] == "2004"]
df_teil_we = df_teil_we[df_teil_we["WellenteilnahmenId"].isin(valid_reader)]
df_teil_woche = df_teil_woche[df_teil_woche["WellenteilnahmenId"].isin(
    valid_reader)]
# WellenIds aller Wellenteilnehmer Wochenendleser
liste_we_ids = df_teil_we.WellenteilnahmenId.unique()
liste_woche_ids = df_teil_woche.WellenteilnahmenId.unique()
df_scans_we = df_scans[df_scans["WellenteilnahmenId"].isin(liste_we_ids)]
df_scans_woche = df_scans[df_scans["WellenteilnahmenId"].isin(liste_woche_ids)]

In [ ]:
df

In [7]:
# df_scans_we mit Erscheinungsdatum und eventuell Ausgabe anreichen über nichtkum-Daten

df_scans_we = df_scans_we.merge(df_nk[["ArtikelId", "Datum", "Ausgabe", "Wochentag", "Ressort_neu3"]], left_on="TeilartikelVeroeffentlichungenId",
                                right_on="ArtikelId")

df_scans_woche = df_scans_woche.merge(df_nk[["ArtikelId", "Datum", "Ausgabe", "Wochentag", "Ressort_neu3"]], left_on="TeilartikelVeroeffentlichungenId",
                                      right_on="ArtikelId")

In [11]:
df_scans_woche

,Id,TeilartikelVeroeffentlichungenId,WellenteilnahmenId,TeilnehmerDurchlesewert,Bewertung,ScanAuswertungId,TeilartikelId,AusgabeErscheinungId,XPosition,YPosition,ArtikelId,Datum,Ausgabe,Wochentag,Ressort_neu3
0,7536,15929,610,99.025620,0.0,34125,5024,81,None,None,15929,12.04.2021,HF3,Montag,Titelseite
1,8193,15929,463,99.025620,0.0,35162,5024,81,None,None,15929,12.04.2021,HF3,Montag,Titelseite
2,8342,15929,437,99.025620,0.0,35368,5024,81,None,None,15929,12.04.2021,HF3,Montag,Titelseite
3,9743,15929,444,99.169975,0.0,37149,5024,81,None,None,15929,12.04.2021,HF3,Montag,Titelseite
4,10442,15929,18,99.025620,0.0,38225,5024,81,None,None,15929,12.04.2021,HF3,Montag,Titelseite
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130699,478991,60424,361,96.485260,0.0,723635,19240,296,None,None,60424,05.06.2021,HF3,Samstag,Mobiles Leben
130700,479147,60424,208,95.578232,0.0,723830,19240,296,None,None,60424,05.06.2021,HF3,Samstag,Mobiles Leben
130701,479549,60424,543,99.659866,0.0,724355,19240,296,None,None,60424,05.06.2021,HF3,Samstag,Mobiles Leben
130702,479512,60435,543,96.188057,0.0,724308,19246,296,None,None,60435,05.06.2021,HF3,Samstag,Feuilleton


In [8]:
# jeden Tag durchlaufen, Leseranzahl pro Tag feststellen,

def create_bw(df, minus=0):

    cols = ["ArtikelId", "LW", "BW", "DW", "Ressort", "Datum",
            "Wochentag", "Leser_Ausgabe", "Leser_Artikel"]
    new_df = pd.DataFrame(columns=cols)

    for date in df.Datum.unique():
        df_ = df[df["Datum"] == date]
        wochentag = df_.Wochentag.unique()[0]

        # Anzahl Leser am Tag festlegen
        all_reader_per_day = df_.WellenteilnahmenId.nunique() - minus

        liste_all_articles = df_.TeilartikelVeroeffentlichungenId.unique()

        for ari in liste_all_articles:
            df_ari = df_[df_["TeilartikelVeroeffentlichungenId"] == ari]
            ressort = df_ari.Ressort_neu3.unique()[0]
            anzahl_leser_artikel = df_ari.shape[0]

            bw = anzahl_leser_artikel / all_reader_per_day * 100
            dw = df_ari.TeilnehmerDurchlesewert.mean()
            lw = bw * (dw/100)

            new_ser = pd.Series([ari, lw, bw, dw, ressort, date, wochentag,
                                all_reader_per_day, anzahl_leser_artikel], index=cols)

            new_df = new_df.append(new_ser, ignore_index=True)

    return new_df


df_wochenende = create_bw(df_scans_we)
df_woche = create_bw(df_scans_woche)

In [9]:
df_wochenende = df_wochenende[(df_wochenende["Wochentag"] == "Freitag") | (
    df_wochenende["Wochentag"] == "Samstag")]

In [35]:
df_woche.to_clipboard(decimal=",", index=False)

In [14]:
df_wochenende["Abo"] = "Wochenende"

In [12]:
df_woche["Abo"] = "Vollabo"

In [34]:
df_all = df_wochenende.append(df_woche)

In [35]:
df_all = df_all.merge(df_nk[["ArtikelId", "Artikel", "Seitentitel", "Platzierungen", "Darstellungsformen",
                             "ErscheinungsId", "AusgabeId"]], on="ArtikelId")

In [36]:
df_all = df_all[['ArtikelId', 'Artikel', 'Datum', 'LW', 'BW', 'DW', 'Ressort', 'Seitentitel', 'Wochentag', 'Abo',
                 'Leser_Ausgabe', 'Leser_Artikel',
                 'Platzierungen', 'Darstellungsformen', 'ErscheinungsId', 'AusgabeId']]

In [37]:
df_all.to_clipboard(decimal=",", index=False)

In [45]:
def create_lw_tab(df_we, df_wo):
    cols = ["Ressort", "LW_Woche", "LW_Wochenende"]
    new_df = pd.DataFrame(columns=cols)

    liste_res_we = df_we.Ressort.unique()

    for res in liste_res_we:
        df_woche = df_wo[df_wo["Ressort"] == res]
        lw_wo = df_woche["LW"].mean()
        df_wochenende = df_we[df_we["Ressort"] == res]
        lw_we = df_wochenende["LW"].mean()

        new_ser = pd.Series([res, lw_wo, lw_we], index=cols)

        new_df = new_df.append(new_ser, ignore_index=True)

    return new_df


df_fi = create_lw_tab(df_wochenende, df_woche)

In [46]:
df_fi.to_csv("./data/ressorts_woche_we.csv", index=False, decimal=",")

In [39]:
df_all.Wochentag.value_counts()

Samstag       2017
Freitag       1851
Dienstag       906
Mittwoch       885
Montag         795
Donnerstag     731
Name: Wochentag, dtype: int64

# Standpunkte - nur Pressestimmen filtern


In [21]:
df_sp = df_kum[df_kum["Seitentitel"]=="Standpunkte"]

In [26]:
newspapers_list = [
 
 'Allgemeine Zeitung (Mainz)',

 'Augsburger Allgemeine',
 
 'Badische Neueste Nachrichten',
 'Badische Neueste Nachrichten (Karlsruhe)',
 'Badische Zeitung',
 'Badische Zeitung (Freiburg)',
 'Badisches Tagblatt',
 'Badisches Tagblatt (Baden-Baden)',
 
 'Berliner Morgenpost',
 'Berliner Zeitung',
 
 'Corriere della Sera (Rom)',
 'Darmstädter Echo',

 'De Telegraaf (Amsterdam)',

 'Der Standard (Wien)',
 
 'Die Glocke (Oelde)',

 'Die Presse (Wien)',

 'Dithmarscher Landeszeitung',
 'Dithmarscher Landeszeitung (Heide)',
 
 'Frankenpost',
 'Frankfurter Allgemeine',
 'Frankfurter Allgemeine Zeitung',
 'Frankfurter Rundschau',
 'Frankfurter Rundschau:',
 'Freie Fahrt – aber ohne Verbrennungsmotor',
 'Freie Presse (Chemnitz)',
 'Freies Wort (Suhl)',
 
 'Gazeta Wyborcza (Warschau)',
 
 'HAZ (Hannover)',
 'Handelsblatt (Düsseldorf)',

 'Heilbronner Stimme',

 'Irish Times (Dublin)',

 'Kölner Stadt-Anzeiger',
 'Kölnische Rundschau',

 'La Stampa (Turin)',

 'Leipziger Volkszeitung',

 'Ludwigsburger Kreiszeitung',

 'Mittelbayerische Zeitung  (Regensburg)',
 'Mitteldeutsche Zeitung',
 'Mitteldeutsche Zeitung (Halle)',

 'Märkische Oderzeitung',

 'Neue Osnabrücker Zeitung',
 'Neue Presse (Coburg)',
 'Neue Zürcher Zeitung',

 'Nordwest-Zeitung',
 'Nordwest-Zeitung (Oldenburg)',
 'Nürnberger Nachrichten',
 'Nürnberger Zeitung',
 'Oldenburgische Volkszeitung',
 
 'Reutlinger General-Anzeiger',
 'Rhein-Neckar-Zeitung',
 'Rhein-Neckar-Zeitung (Heidelberg)',
 'Rhein-Zeitung (Koblenz)',
 'Rheinpfalz (Ludwigshafen)',
 
 'Schwäbische Zeitung',
 
 'Straubinger Tageblatt',
 'Stuttgarter Nachrichten',
 'Stuttgarter Zeitung',
 'Süddeutsche Zeitung',
 'Süddeutsche Zeitung (München)',
 'Südkurier (Konstanz)',
 'Südwest Presse (Ulm)',
 'Südwest-Presse (Ulm)',
 'Tages-Anzeiger (Zürich)',
 'The Irish Times (Dublin)',
 'The Times (London)',
 'Thüringer Allgemeine',

 'Volksstimme (Magdeburg)',

 'Westfalen-Blatt',
 'Westfälische Nachrichten',

 'Wiesbadener Kurier',

 'de Volkskrant (Amsterdam)',
 
 '„Die Glocke“ (Oelde)',
 '„Münchner Merkur“',
 '„Nordwest-Zeitung“ (Oldenburg)',
 '„Volksstimme“ (Magdeburg)']

In [31]:
df_sp_presse = df_sp[df_sp["Artikel"].isin(newspapers_list)]
df_sp_keinepresse = df_sp[~df_sp["Artikel"].isin(newspapers_list)]

In [38]:
df_sp_keinepresse.Ort.value_counts()

keine Angaben    55
Region           37
Deutschland      37
International    30
Name: Ort, dtype: int64

In [39]:
df_sp.shape

(350, 29)

In [32]:
df_sp_presse.LW.mean()

11.363072651777005

In [42]:
df_ka = df_sp_keinepresse[df_sp_keinepresse["Ort"]=="keine Angaben"]
df_ka.to_clipboard(decimal=",")

In [43]:
df_sp_keinepresse.shape

(159, 29)

In [44]:
55/159*100

34.59119496855346

In [41]:
df_sp_keinepresse[df_sp_keinepresse["Ort"]=="International"].mean()

<ipython-input-41-a7d4ec99aaa8>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sp_keinepresse[df_sp_keinepresse["Ort"]=="International"].mean()


SplitId                  20866.766667
ArtikelId                47111.666667
ErscheinungsId             292.100000
AusgabeId                    1.000000
Seite                        2.000000
BW                          33.020525
DW                          85.317183
LW                          28.301514
Beschreibung                      NaN
Sortierung                  31.000000
Sortierung_Wochentage        3.733333
dtype: float64

In [33]:
df_sp.mean()

<ipython-input-33-c88770dfb6c1>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sp.mean()


SplitId                  20381.888571
ArtikelId                46013.142857
ErscheinungsId             284.845714
AusgabeId                    1.000000
Seite                        2.000000
BW                          21.255645
DW                          87.685106
LW                          18.688652
Sortierung                  43.200000
Sortierung_Wochentage        3.860000
dtype: float64

In [ ]:
['Frankfurter Rundschau', 'Neue Osnabrücker Zeitung',
       'Märkische Oderzeitung', 'Oldenburgische Volkszeitung',
       'Handelsblatt (Düsseldorf)',
       'Mitteldeutsche Zeitung (Halle)',
       'Badische Neueste Nachrichten (Karlsruhe)', 'Rücksichtslos',
       'Rhein-Zeitung (Koblenz)', 'Sicher ist sicher', 'Klare Strukturen',
       'Rheinpfalz (Ludwigshafen)', 'Korrektur', 'Die Glocke (Oelde)',
       'Lerneffekt', 'Wiesbadener Kurier', 'Schwebezustand',
       'Badische Neueste Nachrichten', 'Frankfurter Allgemeine Zeitung',
       'Geteilte Kosten', 'Freies Wort (Suhl)',
       'Allgemeine Zeitung (Mainz)', 'Erschreckend', 'Ein Klimapakt',
       'Balance finden', 'Heilbronner Stimme', 'Badisches Tagblatt',
       'Süddeutsche Zeitung', 'Reutlinger General-Anzeiger',
       'de Volkskrant (Amsterdam)', 'Neue Zürcher Zeitung',
       '„Nordwest-Zeitung“ (Oldenburg)', '„Die Glocke“ (Oelde)',
       'Nordwest-Zeitung', 'Dithmarscher Landeszeitung',
       'Leipziger Volkszeitung', 'Etwas zügiger', 'Was es bedeutet',
       'Die Presse (Wien)', 'The Times (London)', 'Falscher Ansatz',
       'Berliner Zeitung', 'Nachschärfen', 'Ziel verfehlt',
       'Nürnberger Nachrichten', 'Lasst sie helfen', 'La Stampa (Turin)',
       'Einheitlich', 'Nürnberger Zeitung', 'The Irish Times (Dublin)',
       'Berliner Morgenpost', 'Thüringer Allgemeine', 'Düstere Zukunft',
       'Roter Vermittler', 'Ein Ausweg',
       'Rhein-Neckar-Zeitung (Heidelberg)',
       'Wie die Sanitärwertschöpfungskette funktioniert',
       'Die Demokraten müssen jetzt liefern', 'Stuttgarter Zeitung',
       'Straubinger Tageblatt', 'Überfordert', 'Wie Sperrmüll',
       'Höchst sensibel', 'Falsches Signal', 'Höchste Zeit',
       'Eine Ausnahme', 'Die Arktis muss ein Raum des Friedens bleiben',
       'Auf Eis gelegt', 'Neue Presse (Coburg)',
       'Ärmere Menschen müssen stärker entlastet werden',
       'Frieden in der Warteschleife', 'Die Kompromisse reichen nicht',
       'Kräfte müssen für mehr Lebensqualität gebündelt werden',
       'Frühzeitige Kontaktbeschränkungen sind zentral',
       'Südwest-Presse (Ulm)', 'Corriere della Sera (Rom)',
       'Größer denn je', 'Stuttgarter Nachrichten',
       'Glasgow ist nur Zwischenstation', 'Westfalen-Blatt',
       'Süddeutsche Zeitung (München)', 'Kein Geschwätz', 'Ausbaufähig',
       'Schwäbische Zeitung', 'Desillusionierend', 'Irish Times (Dublin)',
       'Tages-Anzeiger (Zürich)', 'Nordwest-Zeitung (Oldenburg)',
       'Südkurier (Konstanz)', 'Zur Abschreckung', 'Der Standard (Wien)',
       'Augsburger Allgemeine', 'Die Insel der Geimpften',
       '„Volksstimme“ (Magdeburg)', 'Künstlich', 'Starker Arm',
       'Südwest Presse (Ulm)', 'Volksstimme (Magdeburg)',
       'De Telegraaf (Amsterdam)', 'Frankenpost', 'Ein Abgrund',
       'Flickenteppich', 'Untragbar', 'Ludwigsburger Kreiszeitung',
       'Badisches Tagblatt (Baden-Baden)', 'Der Plan fehlt',
       'Frankfurter Allgemeine', 'Starker Staat',
       'Westfälische Nachrichten', 'Verräterisch', 'Grüner Hafen',
       'Eine Personalie für den Anschein',
       'Schauprozess gegen eine Militärdiktatur', 'Gebot der Transparenz',
       'Der französische Donald Trump', 'Ein Hauch von Götterdämmerung',
       'Es ist noch nicht vorbei',
       'Der US-Präsident steht unter Zeitdruck',
       'Der Osten als Querschnittsthema', 'Schluss mit Reden',
       'Eine neue Stufe der Polemik',
       'Lukaschenko missbraucht das Elend der Flüchtenden', 'Unter Druck',
       'Freie Presse (Chemnitz)',
       'Erdogan will von seiner Misere ablenken',
       'Mittelbayerische Zeitung  (Regensburg)',
       'Dithmarscher Landeszeitung (Heide)',
       'Warum die Impfpflicht kein Königsweg ist',
       'Warum die Impfpflicht notwendig ist',
       'An der Grenze ist Joe Biden noch nicht angekommen',
       'Autoritäre Politik hat Konsequenzen',
       'Wenn Klimaschutz gegen Umweltschutz steht', 'Darmstädter Echo',
       'Fußballstars sind kein moralischer Kompass',
       'Heftige Bild-Störung in der Medienblase',
       'Badische Zeitung (Freiburg)',
       'Der Kampf um das Nilwasser geht in eine neue Phase',
       'Zwei Frauen auf dem Weg zur Spitze',
       'Nicaragua ist Schauplatz einer Tragödie',
       'Ein Präsident mit loser Zunge',
       'Schief und laut – der Corona-Chor',
       'Wie Lukaschenko die EU vorführt', 'Geliebt und gehasst',
       'Mitschuldig', 'Johnsons Strategie der toten Katze',
       'Die Frage der Freiheit  auf der Intensivstation',
       'Vertraulichkeit schafft nicht automatisch Vertrauen',
       '2G für Sportler',
       'Bremer Flüchtlingspolitik verliert Maß und Mitte',
       'Vor Krieg und Terror ist kaum jemand geflüchtet',
       'Die Täter gehören ins Abseits', 'Beispielhaft',
       'Die Nachfolge-Frage treibt das politische Rom um',
       'Eine schnelle  Entscheidung ist nötig',
       'Der Verein muss um  seinen guten Ruf kämpfen',
       'Frankfurter Rundschau:',
       'Ein Rückschlag für den Fortschritt in Chile',
       'Abtreibungsgegner sehen sich am Ziel ihrer Träume',
       'Italiens banger Blick nach Deutschland',
       'Entscheidungen, die die Gesellschaft verändern',
       'Ein grünes Siegel für Kernenergie und Gas', 'Aufstieg ade',
       'Der Protest auf der Straße  ist zu einer Macht geworden',
       'Freie Fahrt – aber ohne Verbrennungsmotor',
       'Harte Probe für die Ampel', 'Missliche Lage', 'Bund ist gefragt',
       'Späte Einsicht',
       'Die Ampel-Koalition muss  für rasche Entlastung sorgen',
       'Die Gemeinschaft stößt an ihre Grenzen',
       'Der große Schatten der extremen Rechten', 'Ein Wagnis',
       'Fortschritt wagen – so soll es funktionieren', 'Klares Signal',
       'Wladimir Putins großer Aufmarsch',
       'Wenigstens schaut die Welt genau hin', 'Gefährlich',
       'Das Ringen zwischen Demokratien und Autokratien',
       'Gazeta Wyborcza (Warschau)', 'Kaum ein Wort zur Wende',
       'Anreize schaffen', 'Chaotische Kommunikation',
       'Gibt es eine Lernkurve?', 'Verausgabt',
       'Wir müssen ausprobieren und dazulernen', 'Mut ist gefragt',
       'Bärbel Bas – die Eins-w-Lösung', 'Kölnische Rundschau',
       'Ein Gesicht für den Neubeginn wird gesucht', 'Der richtige Mann',
       'Zeit für eine neue Drogenpolitik', 'Es wird kein  einfacher Weg',
       'Wie eine Privatfehde die Arbeit im Senat belastet',
       'Kann die Partei zu neuer Einigkeit finden?',
       'Rot-Grün-Rot stolpert in die Koalition', 'Ein Konfliktfall',
       'Weitsicht', 'Die Würde  des Mandats', 'Kölner Stadt-Anzeiger',
       'Ein erster Test', 'Mitteldeutsche Zeitung', 'Abwegig',
       'HAZ (Hannover)', 'Umsteuern', 'Das reicht nicht',
       'Schwere Aufgabe', 'Badische Zeitung', 'Geringes Risiko',
       'Drei Bewerber, drei Richtungen', 'Die Wähler  ticken anders',
       '„Münchner Merkur“', 'Rhein-Neckar-Zeitung', 'Praxisfern',
       'Gefährlicher Mix', 'Bauernopfer', 'Privilegiert', 'Durcheinander',
       'Eine gute Wahl', 'Keine Häme', 'Bitte setzen', 'Wagnis Ampel',
       'Neuanfang geht anders', 'Korrekturen', nan]

In [23]:
df_sp.Artikel.unique()

array(['Frankfurter Rundschau', 'Neue Osnabrücker Zeitung',
       'Märkische Oderzeitung', 'Oldenburgische Volkszeitung',
       'Fragwürdig', 'Verstörend', 'Handelsblatt (Düsseldorf)',
       'Mitteldeutsche Zeitung (Halle)',
       'Badische Neueste Nachrichten (Karlsruhe)', 'Rücksichtslos',
       'Rhein-Zeitung (Koblenz)', 'Sicher ist sicher', 'Klare Strukturen',
       'Rheinpfalz (Ludwigshafen)', 'Korrektur', 'Die Glocke (Oelde)',
       'Lerneffekt', 'Wiesbadener Kurier', 'Schwebezustand',
       'Badische Neueste Nachrichten', 'Frankfurter Allgemeine Zeitung',
       'Geteilte Kosten', 'Freies Wort (Suhl)',
       'Allgemeine Zeitung (Mainz)', 'Erschreckend', 'Ein Klimapakt',
       'Balance finden', 'Heilbronner Stimme', 'Badisches Tagblatt',
       'Süddeutsche Zeitung', 'Reutlinger General-Anzeiger',
       'de Volkskrant (Amsterdam)', 'Neue Zürcher Zeitung',
       '„Nordwest-Zeitung“ (Oldenburg)', '„Die Glocke“ (Oelde)',
       'Nordwest-Zeitung', 'Dithmarscher Landeszeit

# Darstellungsformen Gesellschaft & Stil neu

In [36]:
li = [1, 2, 3]
lii = [2, 4, 5]
li.extend(lii)

In [42]:
x = set(li)

In [43]:
x

{1, 2, 3, 4, 5}

In [40]:
for x in x:
    print(x)

1
2
3
4
5


In [30]:
df_wochenende.shape

(1837, 9)

In [191]:
dfg = df_k[(df_k["Ressort"] == "Gesellschaft")]
dfgg = df_final[(df_final["Ressort"] == "Gesellschaft")]
#t = dfg[dfg["Darstellungsformen"]=="BB, SE"]
# t["DF"]
dfs = df_k[(df_k["Ressort"] == "Stil")]
dfsg = df_final[(df_final["Ressort"] == "Stil")]

In [192]:
dfs[["DF", "Darstellungsformen", "Ressort"]].to_clipboard()

In [199]:
dfg.loc[dfg["Darstellungsformen"] == "BB, SE", "DF"] = "Bildbericht"
dfg.loc[dfg["Darstellungsformen"] == "BF", "DF"] = "Bericht/Feature/Analyse"
dfgg.loc[dfgg["Darstellungsformen"] == "BB, SE", "DF"] = "Bildbericht"
dfgg.loc[dfgg["Darstellungsformen"] == "BF", "DF"] = "Bericht/Feature/Analyse"

c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)


In [200]:
dfs.loc[dfs["Beschreibung"] == "test", "DF"] = "Test"
dfsg.loc[dfsg["Beschreibung"] == "test", "DF"] = "Test"

c:\users\c_knoop\appdata\local\programs\python\python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [181]:
dfs

,SplitId,ArtikelId,Artikel,ErscheinungsId,AusgabeId,Datum,Seite,Ausgabeleser,Artikelleser,BW,...,Wortzahl,Ressort_neu,Sortierung_Ressort_neu.1,Ressort_neu2,Sortierung_Ressort_neu.2,Wortzahl_neu,Platzierung_neu,Platzierung_neu_Sortierung,Ressort_neu3,Wochenende
2173,15911,49903,Wegweiser für den Lenker,246,1,22.05.2021,57,386,85,"22,02",...,mehr als 1300,Stil,140.0,Stil,140.0,1000 bis 1499,Aufmacher,1,Stil,ja
2187,6717,20797,Fantasstisch,106,1,17.04.2021,57,469,70,"14,93",...,700 bis 899,Stil,140.0,Stil,140.0,500 bis 999,Seitenkeller,2,Stil,ja
2188,8682,26950,Sehnsucht nach Berührung,136,1,24.04.2021,61,465,30,"6,45",...,700 bis 899,Stil,140.0,Stil,140.0,500 bis 999,Seitenkeller,2,Stil,ja
2193,13983,43736,"Oma geht’s gut, danke!",216,1,15.05.2021,61,401,44,"10,97",...,700 bis 899,Stil,140.0,Stil,140.0,500 bis 999,Seitenkeller,2,Stil,ja
2194,14039,43790,Was uns blüht,216,1,15.05.2021,58,401,53,"13,22",...,700 bis 899,Stil,140.0,Stil,140.0,500 bis 999,Seitenkeller,2,Stil,ja
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9214,15898,49929,Für ihn: Volles Rohr!,246,1,22.05.2021,53,386,34,"8,81",...,100 bis 299,Stil,140.0,Stil,140.0,unter 500,Einspalter,3,Stil,ja
9216,17638,55267,Für sie: Sex auf dem roten Teppich,271,1,29.05.2021,53,358,38,"10,61",...,100 bis 299,Stil,140.0,Stil,140.0,unter 500,Einspalter,3,Stil,ja
9217,19196,60343,Scheinbar lässig: Carrie Symonds,296,1,05.06.2021,53,329,47,"14,29",...,100 bis 299,Stil,140.0,Stil,140.0,unter 500,Einspalter,3,Stil,ja
10635,15904,49956,Für sie: Roboterliebe,246,1,22.05.2021,53,386,28,"7,25",...,100 bis 299,Stil,140.0,Stil,140.0,unter 500,Einspalter,3,Stil,ja


In [201]:
all_gesellschaft_stil = dfg.append(dfs)

In [202]:
all_gesellschaft_stil.to_csv(
    "./data/gesell_stil_df_neu.csv", decimal=",", index=False)

In [196]:
all_gesellschaft_stil[["DF", "Darstellungsformen", "Ressort"]].to_clipboard()

In [203]:
all_gesch = dfgg.append(dfsg)

all_gesch.shape

In [186]:
all_gesch.shape

(141, 23)

In [210]:
all_geschlecht.to_csv("./data/gesell_stil_df_neu_gesch.csv",
                      decimal=",", index=False)

In [205]:
all_gesch.Ressort.value_counts()

Stil            71
Gesellschaft    70
Name: Ressort, dtype: int64

In [207]:
all_gesch.DF.value_counts()

Bildbericht                7
Test                       6
Bericht/Feature/Analyse    4
Name: DF, dtype: int64

In [208]:
all_geschlecht = all_gesch.merge(
    all_gesellschaft_stil[["SplitId", "DF"]], on="SplitId")

In [209]:
all_geschlecht.columns

Index(['TeilartikelId', 'Platzierung', 'LW_w', 'BW_w', 'DW_w', 'LW_m', 'BW_m',
       'DW_m', 'SplitId', 'Artikel', 'Datum', 'Seitentitel', 'Platzierungen',
       'Darstellungsformen', 'Beschreibung', 'Handlungsorte', 'Akteure',
       'Themen', 'Ausgabe', 'Layerlink', 'Ressort', 'Ausgabeteil', 'DF_x',
       'DF_y'],
      dtype='object')

In [215]:
#all_gesellschaft_stil["LW"] = all_gesellschaft_stil["LW"].str.replace(",", ".").astype(float)
all_gesellschaft_stil.loc[all_gesellschaft_stil.Darstellungsformen ==
                          "BF", "DF"] = "Bericht/Feature/Analyse"
cols = ["Darstellungsform", "LW Gesellschaft", "LW Stil"]
new_df = pd.DataFrame(columns=cols)
for da in all_gesellschaft_stil.DF.unique():

    df_ = all_gesellschaft_stil[all_gesellschaft_stil["DF"] == da]
    df_g = df_[df_["Ressort"] == "Gesellschaft"]
    df_s = df_[df_["Ressort"] == "Stil"]
    lw_g = df_g.LW.mean()
    lw_s = df_s.LW.mean()

    new_ser = pd.Series([da, lw_g, lw_s], index=cols)
    new_df = new_df.append(new_ser, ignore_index=True)

In [216]:
new_df

,Darstellungsform,LW Gesellschaft,LW Stil
0,Komm./Kolumne,10.759412,8.484516
1,Bildnachricht,2.180000,NaN
2,Interview,25.517000,NaN
3,Geschichtstext,22.345714,NaN
4,Essay,25.033333,14.540000
5,Bericht/Feature/Analyse,11.982500,8.167273
6,Porträt,16.333750,11.827000
7,Reportage,18.673333,20.005000
8,Bildbericht,13.300000,NaN
9,NaN,NaN,NaN


In [217]:
new_df.to_csv("./data/unterschied_df_ges_stil.csv", decimal=",", index=False)